# ***DATA***

## SETUP

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

import re
import time

from selenium.webdriver.safari.service import Service as SafariService

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException

import pandas as pd
import numpy as np

import ipywidgets as widgets
from IPython.display import display, HTML

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

## ALL MOVIES DATA

In [2]:
# last_year = 2023

# years_list = [num for num in range(1975,last_year+1)]
# links_dict = {year : 'https://en.wikipedia.org/wiki/'+str(year)+'_Metro_Manila_Film_Festival' for year in years_list}
# data = {}
    
# for year in years_list:
#     link = links_dict[year]

#     # get html
#     response = requests.get(link)
#     soup = BeautifulSoup(response.text,'html.parser')
#     options_list = list(soup.find_all(['td','h2','h3','h4']))

#     # cut html (start)
#     tag = [element.name for element in options_list]
#     text = [element.get_text().strip() for element in options_list]

#     if 'Entries[edit]' in text:
#         remove_start = text.index('Entries[edit]')
#     elif 'Entries' in text:
#         remove_start = text.index('Entries')
        
#     tag = tag[remove_start+1:]
#     text = text[remove_start+1:]

#     remove_start = tag.index('td')
#     tag = tag[remove_start:]
#     text = text[remove_start:]

#     # cut html (end)
#     for item in text:
#         if '[edit]' in item:
#             remove_end = text.index(item)
#             break
#     tag = tag[: remove_end]
#     text = text[: remove_end]

#     for item in tag:
#         if item != 'td':
#             remove_end = tag.index(item)
#             break
#     tag = tag[: remove_end]
#     text = text[: remove_end]

#     # clean data
#     for index in range(len(text)):
#         item = text[index]
#         while '[' in item:
#             ref_text = item[item.index('[') : item.index(']')+1]
#             item = item.replace(ref_text, '')
#             del text[index]
#             text.insert(index, item)
#         while '\n' in item:
#             ref_text = '\n'
#             item = item.replace(ref_text, '; ')
#             del text[index]
#             text.insert(index, item)

#     for item in text:
#         if item == '':
#             text.remove(item)
            
#     # make dataframe dict
#     data.update({year:
#         {'Year' : [year]*(int(len(text)/5)),
#         'Title' : [text[index] for index in range(len(text)) if index % 5 == 0],
#         'Starring' : [text[index] for index in range(len(text)) if index % 5 == 1],
#         'Studio' : [text[index] for index in range(len(text)) if index % 5 == 2],
#         'Director' : [text[index] for index in range(len(text)) if index % 5 == 3],
#         'Genre' : [text[index] for index in range(len(text)) if index % 5 == 4]}
#     }) 

In [3]:
data = {1975: {'Year': [1975, 1975, 1975, 1975, 1975, 1975, 1975, 1975],
  'Title': ['Alat!',
   'Araw-Araw, Gabi-Gabi',
   "Batu-Bato sa Langit (Ang Tamaa'y Huwag Magagalit..!)",
   'Diligin mo ng Hamog ang Uhaw na Lupa',
   'Kapitan Kulas',
   'Karugtong ang Kahapon',
   'Postcards from China',
   "Siya'y Umalis, Siya'y Dumating"],
  'Starring': ['Tony Ferrer, George Estregan, Chanda Romero, Suzanne Gonzales',
   'Charito Solis, Tony Santos, Sr., Rosanna Ortiz, Dindo Fernando',
   'Nora Aunor, Christopher de Leon, Nida Blanca',
   'Joseph Estrada, Gloria Diaz',
   'Ramon Revilla, Sr., Elizabeth Oropesa, Helen Gamboa, Walter Navarro',
   'Vilma Santos, Edgar Mortiz, Eddie Garcia, Gloria Romero, Celia Rodriguez',
   'Dante Rivero, Boots Anson-Roa, Pilar Pilapil',
   'Marlene Dauden, Nestor de Villa'],
  'Studio': ['Tagalog Ilang-Ilang Productions',
   'Premiere Productions',
   'N.V. Productions',
   'JE Productions',
   'Lea Productions',
   'Roma Films',
   'Lyra Ventures',
   'Mirick Films'],
  'Director': ['Pierre Salas',
   'Danilo Cabreira',
   'Luciano B. Carlos',
   'Augusto Buenaventura',
   'Romy Suzara',
   'Fely Crisostomo',
   'Cesar Gallardo',
   'Mitos Villarreal'],
  'Genre': ['Action',
   'Drama',
   'Romance, Comedy',
   'Action',
   'Action, Drama',
   'Drama, Romance',
   'Romance',
   'Drama']},
 1976: {'Year': [1976, 1976, 1976, 1976, 1976, 1976, 1976, 1976, 1976, 1976],
  'Title': ['Barok',
   'Dateline Chicago: Arrest The Nurse Killer',
   'Ganito Kami Noon, Paano Kayo Ngayon',
   'Insiang',
   'The Interceptors',
   'Kisame Street',
   'Makahiya at Talahib',
   "Minsa'y isang Gamu-gamo",
   'Puede Ako, Puwede Ka Pa Ba',
   'The Rebel Hunter (True-to-Life Story of Brig. Gen. Romeo Gatan)'],
  'Starring': ['Chiquito, Trixia Gomez, Max Alvarado, Tange, Joaquin Fajardo, Palito, Arlene Sison, Ely Roque, Jun Santos, Norma Yumul, Manuel Dagul',
   'Joseph Estrada, Pilar Pilapil, Mailyn Herrs, Garret Blake, J.C. Marshall, Debbie Brinson',
   'Christopher de Leon, Gloria Diaz, Eddie Garcia, Dranreb Belleza, Leopoldo Salcedo, Rosemarie Gil, Johnny Vicar, Tsing-Tong Tsai',
   'Hilda Koronel, Mona Lisa, Ruel Vernal, Rez Cortez, Marlon Ramirez',
   'Tony Ferrer, Ramon Zamora, George Estregan, Edna Diaz, Paquito Diaz, Charlie Davao, Vic Diaz, Max Alvarado',
   'Dolphy, Nida Blanca, Panchito, Rolly Quizon, Katy dela Cruz, Patsy, Georgie Quizon, Bentot, Jr., Margie Braza, Teroy de Guzman',
   'Vilma Santos, Rudy Fernandez, Trixia Gomez, Gloria Romero, Romeo Rivera, Rocco Montalban, Robert Rivera, Max Alvarado',
   'Nora Aunor, Jay Ilagan, Gloria Sevilla, Perla Bautista, Eddie Villamayor, Paquito Salcedo, Lily Miraflor, Leo Martinez, Nanding Fernandez, Luz Fernandez',
   'Amalia Fuentes, Romeo Vasquez, Eddie Gutierrez, Vivian Velez, Rosemarie Gil, Ike Lozada, Inday Badiday, Cloyd Robinson, Lilian Laing, Sandy Garcia',
   "Ramon Revilla, Sr., Boots Anson-Roa, George Estregan, Rodolfo 'Boy' Garcia, Rez Cortez, Max Alvarado"],
  'Studio': ['HPS Film Productions',
   'JE Productions',
   'Hemisphere',
   'Cinemanila',
   'Topaz Film Productions',
   'RVQ Productions',
   'Goodwill Productions',
   'Premiere Productions',
   'AM Productions',
   'Imus Productions'],
  'Director': ['F.H. Constantino',
   'Cesar Gallardo',
   'Eddie Romero',
   'Lino Brocka',
   'Efren C. Piñon',
   'Ading Fernando',
   'Emmanuel Borlaza',
   'Lupita Concio',
   'Elwood Perez & Joey Gosiengfiao',
   'Jose Yandoc'],
  'Genre': ['Comedy',
   'Action',
   'Period Drama, Musical, Romance',
   'Drama',
   'Action',
   'Comedy',
   'Drama',
   'Protest Drama',
   'Drama, Romance',
   'Action']},
 1977: {'Year': [1977, 1977, 1977, 1977, 1977, 1977, 1977, 1977, 1977],
  'Title': ['Babae... Ngayon at Kailanman',
   'Bakya Mo Neneng',
   'Banta ng Kahapon',
   'Mga Bilanggong Birhen',
   'Burlesk Queen',
   'Inay',
   "Kung Mangarap Ka't Magising",
   'Sa Piling ng mga Sugapa',
   'Walang Katapusang Tag-araw'],
  'Starring': ['Charito Solis, Gloria Diaz, Chanda Romero, Vivian Velez',
   "Joseph Estrada, Nora Aunor, Tirso Cruz III, Gloria Sevilla, Angelo Castro, Jr., Ramon D' Salva, Angelo Ventura",
   'Vic Vargas, Rafael Roco Jr., Roland Dantes, Chanda Romero',
   'Alma Moreno, Trixia Gomez, Rez Cortez, Armida Siguion-Reyna, Monang Carvajal, Leroy Salvador',
   'Vilma Santos, Rolly Quizon, Rosemarie Gil, Joonee Gamboa, Leopoldo Salcedo, Roldan Aquino, Chito Ponce Enrile, Dexter Doria, Yolanda Luna',
   'Dindo Fernando, Chanda Romero, Orestes Ojeda, Laurice Guillen, Ace Vergel, Dexter Doria, Alicia Vergel',
   'Christopher de Leon, Hilda Koronel',
   'Mat Ranillo III, Bembol Roco, Chanda Romero',
   'Charito Solis, Mat Ranillo III, Eddie Garcia'],
  'Studio': ['Melros Productions',
   'JE Productions',
   'Hemisphere Films',
   'Pera Films',
   'Ian Film Productions',
   'Lotus Films',
   'LVN Pictures',
   'Sining Silangan',
   'Lea Productions'],
  'Director': ['Joey Gosiengfiao',
   'Augusto Buenaventura',
   'Eddie Romero',
   "Mario O'Hara & Romy Suzara",
   'Celso Ad. Castillo',
   'Lino Brocka',
   'Mike de Leon',
   'Gil Portes',
   'Ishmael Bernal'],
  'Genre': ['Drama',
   'Action, Comedy, Romance',
   'Action',
   'Period Drama',
   'Drama, Romance',
   'Drama, Comedy',
   'Coming-of-age, Drama, Romance. Musical',
   'Drama',
   'Drama, Romance']},
 1978: {'Year': [1978, 1978, 1978, 1978, 1978, 1978, 1978, 1978, 1978],
  'Title': ['Atsay',
   'Garrote: Jai Alai King',
   'Ang Huling Lalaki ng Baluarte',
   "Jack n'Jill of the Third Kind",
   'The Jess Lapid Story',
   'Katawang Alabok',
   'Kid Kaliwete',
   'Rubia Servios',
   'Salonga'],
  'Starring': ['Nora Aunor, Ronald Corveau, Amy Austria, Armida Siguion-Reyna, Roldan Aquino, Renato Robles, Lilian Laing, Angie Ferro, Bella Flores',
   'Christopher de Leon, Marianne dela Riva, Dranreb, Johnny Delgado, Amy Austria, Cynthia Gonzales, Allan Valenzuela, Manny Luna',
   'Rey Malonzo, Tina Monasterio',
   'Dolphy, Nora Aunor, Rolly Quizon, Panchito, Paquito Diaz, Martin Marfil, Georgie Quizon, Max Vera',
   'Lito Lapid, Beth Bautista',
   'Robert Arevalo, Daisy Romualdez, Orestes Ojeda, Janet Bordon, Vic Silayan, Lorna Tolentino, Manny Luna, Anita Linda, Lucita Soriano',
   'Bembol Roco, George Estregan, Trixia Gomez, Jean Saburit, Joonee Gamboa, Cynthia Gonzales, Val Iglesias',
   'Vilma Santos, Mat Ranillo III, Phillip Salvador',
   "Rudy Fernandez, George Estregan, Trixia Gomez, Ruel Vernal, Raul Aragon, Dencio Padilla, Amy Austria, Rodolfo 'Boy' Garcia, Jose Romulo, Veronica Jones"],
  'Studio': ['Ian Film Productions',
   'VP Pictures',
   'SQ Film Productions',
   'RVQ Productions',
   'Mirick Films',
   'Agrix Film Productions',
   'Associated Entertainment Corp.',
   'Sampaguita Pictures',
   'MBM Productions'],
  'Director': ['Eddie Garcia',
   "Manuel 'Fyke' Cinco",
   'Artemio Marquez',
   'Frank Gray Jr.',
   'Gallardo',
   'Emmanuel Borlaza',
   "Manuel 'Fyke' Cinco",
   'Lino Brocka',
   'Romy Suzara'],
  'Genre': ['Drama, Romance',
   'Action, Drama, Sport',
   'Action',
   'Comedy',
   'Action',
   'Drama',
   'Action, Sport',
   'Drama',
   'Action, Drama']},
 1979: {'Year': [1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979, 1979],
  'Title': ['Alabok na Ginto',
   'Ang Alamat ni Julian Makabayan',
   'Bugoy',
   'Ina Ka ng Anak Mo',
   'Kadete',
   'Kasal-Kasalan, Bahay-Bahayan',
   'Ang Lihim ng Guadalupe',
   'Mamang Sorbertero',
   'Modelong Tanso',
   'Ang Sisiw ay Isang Agila'],
  'Starring': ['Roel Vergel de Dios, Veronica Jones, Allan Valenzuela, Anita Linda, Ruben Rustia, Renato Del Prado, Joe Garcia',
   'Christopher de Leon, Charo Santos, Eddie Garcia, Perla Bautista, Johnny Delgado, Lilibeth Ranillo, Celso Ad. Castillo, Tony Santos Sr.',
   'Dolphy, Lotis Key, Panchito, Paquito Diaz, Max Alvarado, Ernie Ortega, Conde Ubaldo, Hero Bautista',
   'Lolita Rodriguez, Nora Aunor, Raul Aragon',
   'Jay Ilagan, Ronald Corveau, Rez Cortez, Anna Marin, Fred Montilla, Rosemarie Gil, Alicia Alonzo, Anita Linda, Romeo Rivera, Dino Kortes, Michael de Mesa, Marissa del Mar',
   'Nora Aunor, Christopher de Leon, Alma Moreno, Rudy Fernandez, Rebecca Gonzales, Lito Anzures, Jose Garcia, Matimtiman Cruz, German Moreno',
   'Fernando Poe, Jr., Tina Monasterio, Yvette Christine, Paquito Diaz, Lito Anzures, Vic Diaz, Romy Diaz, Bentot, Jr., Max Alvarado',
   'Joseph Estrada, Celeste Legaspi, Rod Navarro, Dencio Padilla, Subas Herrero, Quiel Segovia, Vic Sotto, Veronica Palileo',
   'Charito Solis, Vilma Santos',
   'Lito Lapid, George Estregan, Dante Varona, David Aguila, Julie Ann Fortich, Lucita Soriano, Teroy de Guzman, Yoyoy Villame'],
  'Studio': ['MBM Productions',
   'Ian Films',
   'Hemisphere Pictures',
   'Movie Masters',
   'Agrix Films Production',
   'HPS Film Productions',
   'FPJ Productions',
   'JE Productions',
   'Bancom Audiovision',
   'Mirick Films International'],
  'Director': ['Dr. Antonio C. Martinez',
   'Celso Ad. Castillo',
   'Jett C. Espiritu',
   'Lino Brocka',
   'Emmanuel Borlaza',
   'Pablo Santiago',
   'Armando A. Herrera',
   'Augusto Buenaventura',
   'Cirio H. Santiago',
   'Jun Gallardo'],
  'Genre': ['Romance, War',
   'Action',
   'Comedy',
   'Drama',
   'Drama',
   'Romance, Comedy',
   'Action, Adventure',
   'Comedy',
   'Drama',
   'Action, Adventure']},
 1980: {'Year': [1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980, 1980],
  'Title': ['Basag',
   'Bona',
   'Brutal',
   'Dang-Dong',
   "Kape't Gatas",
   "Kung Ako'y Iiwan Mo",
   'Langis at Tubig',
   'Ang Panday',
   'Taga sa Panahon',
   'Tembong'],
  'Starring': ['Mat Ranillo III, Vivian Velez, Alma Moreno',
   'Nora Aunor, Phillip Salvador',
   'Amy Austria, Gina Alajar, Jay Ilagan',
   'Susan Valdez, Eddie Garcia, George Estregan, Andrea Bautista',
   'Chiquito, Julie Vega',
   'Nora Aunor, Christopher de Leon, Rollie Quizon',
   'Vilma Santos, Dindo Fernando, Amy Austria',
   'Fernando Poe, Jr., Liz Alindogan, Paquito Diaz, Max Alvarado, Lito Anzures, Bentot, Jr.',
   'Christopher De Leon, Bembol Roco, Chanda Romero, Cherie Gil, Suzette Ranillo, Michael de Mesa',
   'Niño Muhlach, Ricky Belmonte, Boots Anson-Roa, Azenth Briones, Eddie Garcia'],
  'Studio': ['Showbiz',
   'NV Productions',
   'Bancom Audiovision',
   'Imus Productions',
   'Larry Santiago Productions',
   'Lea Productions',
   'Sining Silangan',
   'FPJ Productions',
   'Premiere Productions',
   "D'Wonder Films"],
  'Director': ['Van Ludor',
   'Lino Brocka',
   'Marilou Diaz-Abaya',
   'Efren Pinon',
   'Ric Santiago',
   'Laurice Guillen',
   'Danny L. Zialcita',
   'Ronwaldo Reyes',
   'Augusto Buenaventura',
   'J. Erastheo Navoa'],
  'Genre': ['Drama',
   'Drama',
   'Drama',
   'Comedy, Family',
   'Comedy',
   'Drama',
   'Drama, Romance',
   'Action, Fantasy',
   'Drama',
   'Comedy']},
 1981: {'Year': [1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981, 1981],
  'Title': ['Ang Babae sa Ulog',
   'Indio',
   "Init O' Lamig",
   'Kamlon',
   'Kapitan Kidlat',
   'Karma',
   'Kisapmata',
   'Pagbabalik ng Panday',
   "Rock n' Roll",
   'Tropang Bulilit'],
  'Starring': ['Ricky Belmonte, Jean Saburit, Maria Victoria, Yasmin Ayesa, Alan Bautista, Karen Lopez',
   "Dante Varona, Chiqui Hollmann, Lirio Vital, Romy Diaz, Subas Herrero, Rodolfo 'Boy' Garcia, Odette Khan, Dave Brodett",
   'Charito Solis, Dindo Fernando, Elizabeth Oropesa, Chanda Romero, Gina Alajar',
   'Ramon Revilla, Anthony Alonzo, Isabel Rivas, Rosemarie de Vera, Eddie Garcia, George Estregan, Nick Romano, Arnold Mendoza, Laarni Enriquez',
   'Elizabeth Oropesa, Chanda Romero, Celia Rodriguez, Dranreb Belleza, Michael de Mesa, Rio Locsin, Carlo Gabriel',
   'Vilma Santos, Ronaldo Valdez, Chanda Romero, Tommy Abuel',
   'Vic Silayan, Jay Ilagan, Charo Santos, Charito Solis',
   'Fernando Poe, Jr., Tina Revilla, Bentot, Jr., Rosemarie Gil, Lito Anzures, Jose Romulo, Lilian Laing, Max Alvarado',
   'Nora Aunor, Lloyd Samartino, Mark Gil, Ricky Davao',
   'Niño Muhlach, Sheryl Cruz, Andrea Bautista, Janice de Belen, Lea Salonga, Donna Villa, Lloyd Samartino, Manny Luna'],
  'Studio': ['Baby Pascual Films & Associates',
   'Lea Productions',
   'Cinex Films & F. Puzon Film Enterprises',
   'Imus Productions',
   'Premiere Productions',
   'Sining Silangan',
   'Bancom Audiovision',
   'FPJ Productions',
   'NV Productions & Annie Batungbakal Productions',
   "D'Wonder Films"],
  'Director': ['Ed Palmos',
   'Carlo J. Caparas',
   'Eddie Rodriguez',
   'Jose Yandoc',
   'Emmanuel Borlaza',
   'Danny Zialcita',
   'Mike de Leon',
   'Ronwaldo Reyes',
   'Maryo J. de los Reyes',
   'J. Erastheo Navoa'],
  'Genre': ['Drama',
   'Action',
   'Drama',
   'Biography, Action',
   'Action, Fantasy, Superhero',
   'Romance, Drama',
   'Psychological Horror',
   'Action, Adventure, Fantasy, Horror',
   'Family, Music',
   'Comedy, Drama, Family']},
 1982: {'Year': [1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982, 1982],
  'Title': ["The Cute...The Sexy n' The Tiny",
   'Desire',
   'Haplos',
   'Himala',
   'Magindanao',
   'Moral',
   'Ang Panday: Ikatlong Yugto',
   'Santa Claus is Coming to Town',
   'Tatlo Silang Tatay Ko',
   'Tulisan ng Pasong Musang'],
  'Starring': ['Bering Labra, Pia Moran, Weng Weng, Dencio Padilla, Beth Sandoval, Bonnie De Jesus',
   'Charito Solis, John Saxon, Tetchie Agbayani, Judith Chapman, Mark Gil',
   'Vilma Santos, Christopher de Leon, Rio Locsin, Delia Razon, J. Eddie Infante, Rez Cortez, Juan Rodrigo',
   'Nora Aunor, Spanky Manikan, Gigi Dueñas, Amable Quiambao, Vangie Labalan, Veronica Panlileo, Peng Medina',
   'Mohamad Faizal, Rhoy Flores, Rex Lapid, Charlie Davao, Philip Gamboa, Laarni Enriquez, Paquito Diaz, Romy Diaz, Dick Israel, Tony Carrion, Joaquin Fajardo, Willy Dado, Amay Bisaya, Max Alvarado',
   'Lorna Tolentino, Gina Alajar, Sandy Andolong, Anna Marin',
   "Fernando Poe, Jr., Criselda 'Dang' Cecilio, Lito Anzures, Bentot, Jr.",
   'Snooky Serna, Maricel Soriano, Gabby Concepcion, William Martinez, Albert Martinez, Robert Arevalo, Liza Lorena, Marissa Delgado, Raul Aragon, Alicia Alonzo, Mila Ocampo, Johnny Wilson, Joel Alano, Aiko Melendez, Kristine Garcia',
   'Tito Sotto, Vic Sotto, Joey de Leon, Susan Bautista, Panchito, Dely Atay-Atayan, Ike Lozada, German Moreno, Niño Muhlach',
   'Ramon Revilla, Susan Valdez, George Estregan, Paquito Diaz, Alicia Alonzo, Johnny Wilson, Janice Jurado'],
  'Studio': ['Liliw Films International',
   'Hemisphere Pictures',
   'Mirick Films International',
   'Experimental Cinema of the Philippines',
   'East West International Films',
   'Seven Stars Productions',
   'FPJ Productions',
   'Regal Films',
   "D'Wonder Films",
   'Lea Productions'],
  'Director': ['Pablo Vergara & Eddie Nicart',
   'Eddie Romero',
   'Antonio Jose Perez',
   'Ishmael Bernal',
   'Diego Cagahastian',
   'Marilou Diaz-Abaya',
   'Ronwaldo Reyes',
   'Elwood Perez',
   'J. Erastheo Navoa',
   'Efren Pinon'],
  'Genre': ['Comedy',
   'Drama',
   'Drama, Horror, Romance, Thriller',
   'Drama',
   'Action, War',
   'Coming-of-age, Drama',
   'Action, Adventure, Fantasy',
   'Drama, Family, Romance',
   'Comedy',
   'Action']},
 1983: {'Year': [1983, 1983, 1983, 1983, 1983, 1983, 1983, 1983],
  'Title': ['Bad Bananas sa Puting Tabing',
   'Bago Kumalat ang Kamandag',
   "D'Godson",
   'Gunfighter',
   'Hot Property',
   'Karnal',
   'Over My Dead Body',
   'Tengteng De Sarapen'],
  'Starring': ['Christopher de Leon, Edgar Mortiz, Jay Ilagan, Johnny Delgado, Nora Aunor, Jaime Fabregas, Larry Lawrence',
   'Anthony Alonzo, Coney Reyes, Eddie Garcia, Perla Bautista, Alicia Alonzo',
   "Niño Muhlach, William Martinez, Susan Bautista, Panchito, Pepe Pimentel, Nanette Inventor, Manny Luna, Rodolfo 'Boy' Garcia, Bayani Casimiro, Jimmy Santos, Palito, Joaquin Fajardo, Fred Panopio",
   'Lito Lapid, Connie Angeles, Chuck Biller, Cole Mackay, Paul Jones, Marlene Chavez, Brad Fletcher',
   'Carmi Martin, Phillip Salvador, Dennis Roldan, Vic Diaz, Tony Santos, Sr.',
   'Charito Solis, Phillip Salvador, Vic Silayan, Joel Torre, Cecille Castillo',
   'Tony Ferrer, Efren Reyes, Jr., Bembol Roco, Raoul Aragon, Anna Marie Gutierrez, Lolita Marquez',
   'Dolphy, Alma Moreno, Panchito, Jaypee de Guzman, Raoul Aragonn, Bayani Casimiro, Teroy de Guzman, Palito, Ben David, Rocco Montalban, Larry Silva, Allan Bautista, Max Vera'],
  'Studio': ['Vanguard Films',
   'Lea Productions',
   "D'Wonder Films",
   'APG Films',
   'Golden Dragon Films',
   'Cine Suerte',
   'Sunfilms International',
   'RVQ Productions'],
  'Director': ['Peque Gallaga',
   'Willie Milan',
   'J. Erastheo Navoa',
   'Romy Suzara',
   'Lino Brocka',
   'Marilou Diaz-Abaya',
   "Arsenio 'Boots' Bautista",
   'Frank Gray, Jr.'],
  'Genre': ['Adventure, Comedy',
   'Action',
   'Action, Comedy',
   'Action, Western',
   'Drama',
   'Drama',
   'Action',
   'Comedy']},
 1984: {'Year': [1984, 1984, 1984, 1984, 1984, 1984, 1984, 1984, 1984],
  'Title': ['Alapaap',
   "Atsay Killer: Buti Nga Sa'yo",
   'Bukas...May Pangarap',
   'Bulaklak sa City Jail',
   'Idol',
   'Misteryo sa Tuwa',
   'Muntinlupa',
   'Ang Panday IV: Ika- Apat Na Aklat',
   'Shake, Rattle & Roll'],
  'Starring': ['William Martinez, Mark Gil, Michael de Mesa, Tanya Gomez, Eva Rose Palma, Isadora',
   'Eddie Garcia, Panchito, Cachupoy, Fauzi Omar, Isabel Rivas, Barbara Luna, Julie Ann Fortich, Donna Villa, Rosemarie de Vera',
   'Gina Alajar, Tommy Abuel, Dante Balaois, Michael Baluyot, Richard Baluyot, Thea Cleofe Salvador, Bes Flores, Ruben Rustia, Bebong Osorio, Josie Galvez',
   'Nora Aunor, Gina Alajar, Celia Rodriguez, Perla Bautista, Maya Valdez, Zenaida Amador, Maritess Gutierrez, Shyr Valdez',
   'Rudy Fernandez, Connie Angeles, Chuckie Dreyfuss, Dencio Padilla, Ruben Rustia, Victor Bravo, Dick Israel, Robert Talabis, Philip Gamboa, Bomber Moran, Rommel Valdez',
   'Tony Santos, Sr., Johnny Delgado, Ronnie Lazaro, Alicia Alonzo, Lito Anzures, Amable Quiambao, Mario Montes',
   "Anthony Alonzo, Marianne dela Riva, Fred Montilla, Tita Muñoz, Alicia Alonzo, Max Alvarado, Joonee Gamboa, Renato del Prado, Rodolfo 'Boy' Garcia",
   'Fernando Poe, Jr., Marianne dela Riva, Lito Anzures, Bentot, Jr., Eddie Infante, Mario Escudero, Robert Rivera, Max Alvarado',
   'Episode 1: "Baso" - Joel Torre, Rey’PJ’ Abellana, Arlene Muhlach; Episode 2: "Pridyider" – Janice de Belen, Charito Solis, William Martinez, Mon Alvir, Lito Gruet; Episode 3: "Manananggal" - Herbert Bautista, Irma Alegre, Mary Walter, Peewee Quiajano, Pen Medina'],
  'Studio': ['Oro Vista Motion Pictures',
   'Sittis Films',
   'Tri-Films',
   'Cherubim Films',
   'Bukang Liwayway Films',
   'Experimental Cinema of the Philippines',
   'Triple A Films',
   'FPJ Productions',
   'Athena Productions'],
  'Director': ['Tata Esteban',
   'Angel Labra',
   'Gil Portes',
   "Mario O'Hara",
   'Romy Suzara',
   'Abbo Q. dela Cruz',
   'Jose Antonio Alonzo',
   'Ronwaldo Reyes',
   'Emmanuel Borlaza; Ishmael Bernal; Peque Gallaga'],
  'Genre': ['Drama, Horror, Thriller',
   'Comedy',
   'Drama',
   'Crime, Drama',
   'Action',
   'Action',
   'Drama, Thriller',
   'Action, Adventure, Fantasy',
   'Horror']},
 1985: {'Year': [1985, 1985, 1985, 1985, 1985, 1985, 1985, 1985, 1985, 1985],
  'Title': ['Ano ang Kulay ng Mukha ng Diyos?',
   'Boboy Tibayan: Tigre ng Cavite',
   'Bomba Arienda',
   'God... Save Me!',
   'Hee Man: Master of None',
   "I Can't Stop Loving You",
   'Menudo At Pandesal',
   'The Moises Padilla Story: The Missing Chapter',
   'Oks na Oks Partner',
   'Paradise Inn'],
  'Starring': ['Gina Alajar, Michael de Mesa, Edu Manzano, Caridad Sanchez and Kristin Gonzales',
   'Ramon Revilla, Jr., Lani Mercado, Lyka Ugarte, George Estregan, Raoul Aragonn, Perla Bautista, Rez Cortez, Conrad Poe, Bomber Moran',
   'Ace Vergel, Charito Solis, William Martinez, Carmi Martin, Tommy Abuel and Coney Reyes',
   'Christopher de Leon, Charito Solis, Rio Locsin, Cherie Gil, Tommy Abuel, Juan Rodrigo, Rose Ann Gonzales',
   'Redford White, Roderick Paulate, Emily Loren, Don Pepot, Tatlong Itlog, Joaquin Fajardo, Maning Bato, Imelda Caparas, Ernie Ortega, Mario Escudero',
   'Nora Aunor, Tirso Cruz III, Rowell Santiago, Jackie Lou Blanco, Lani Mercado, Miguel Rodriguez, Nadia Montenegro, Richard Gomez, Liza Lorena, Marissa Delgado',
   'Gloria Diaz, Carmi Martin, Tanya Gomez, Janice De Belen, Paquito Diaz, Max Alvarado, Lou Veloso & Caridad Sanchez',
   'Anthony Alonzo, Charito Solis, Gina Alajar, Tommy Abuel, George Estregan, Raoul Aragonn, Juan Rodrigo, Rex Lapid, Daria Ramirez, Fred Moro, Roi Vinzon',
   'Niño Muhlach, Janice de Belen, Aga Muhlach',
   "Lolita Rodriguez, Vivian Velez, Dennis Roldan, Michael de Mesa, Robert Arevalo, Armida Siguon Reyna, Lito Anzures, Jinggoy Estrada, Lucita Soriano, Rodolfo 'Boy' Garcia, Bing Davao, Jaime Fabregas, Juan Carlos"],
  'Studio': ['Lea Productions',
   'Imus Productions',
   'Brown Fox Organization',
   'Seiko Films',
   'Cine International Group',
   'Regal Films',
   'Essex Films',
   'Triple A Films',
   "D'Wonder Films",
   'Amazaldy Film Production'],
  'Director': ['Lino Brocka',
   'Pepe Marcos',
   'Mel Chionglo',
   'Carlo J. Caparas',
   'Tony Y. Reyes',
   'Elwood Perez',
   'Danny L. Zialcita',
   'Willie Milan',
   'J. Erastheo Navoa',
   'Celso Ad Castillo'],
  'Genre': ['Drama',
   'Action, Drama',
   'Biography',
   'Drama',
   'Action, Comedy, Fantasy',
   'Drama, Romance',
   'Comedy',
   'Action',
   'Comedy',
   'Drama']},
 1986: {'Year': [1986, 1986, 1986, 1986, 1986, 1986, 1986],
  'Title': ['Bagets Gang',
   'Bangkay Mo Akong Hahakbangan',
   'Captain Barbell',
   'Halimaw',
   'Payaso',
   'Salamangkero: The Magician',
   'Tuklaw'],
  'Starring': ['Jinggoy Estrada, Rio Locsin, ER Ejercito, Robin Padilla, Christopher Ad. Castillo',
   'Dante Varona, Melissa Mendez, Paquito Diaz, Millicent Bautista, Aurora Sevilla, Fred Moro, Usman Hassim, Rocco Montalban and Joko Diaz',
   'Edu Manzano, Herbert Bautista, Lea Salonga, Dennis Da Silva, Bing Loyzaga, Nova Villa, Ruel Vernal, Rez Cortez, Beth Bautista',
   "Liza Lorena, Gina Pareno, Michael de Mesa, Mario O'Hara, Ruel Vernal, Maritess Gutierrez, Ronel Victor, Jaime Fabregas, Ian Kristoffer de Leon & Lotlot de Leon",
   'German Moreno and the All-Star Cast',
   'Michael de Mesa, Gina Alajar, Liza Lorena, Armida Siguion-Reyna, Tanya Gomez, Dick Israel, Odette Khan, Ruben Rustia, Tom-Tom, Sunshine',
   'Charito Solis, Nadia Montenegro, Anjo Yllana, Richard Gomez, Janice Jurado, Zandro Zamora, Lucita Soriano, Jaime Fabregas, Robert Talabis'],
  'Studio': ['Amazaldy Film Production',
   'Bathala Films',
   'Viva Films',
   'NCV Films',
   'Sialina Film Enterprises',
   'Aces Films',
   'Golden Lion Films'],
  'Director': ['Augusto Buenaventura',
   'Dante Varona',
   'Leroy Salvador',
   "Christopher de Leon & Mario O' Hara",
   'Celso Ad. Castillo',
   'Tata Esteban',
   'Carlo J. Caparas'],
  'Genre': ['Action, Drama',
   'Action, War',
   'Action, Adventure, Comedy, Superhero',
   'Action, Comedy, Fantasy, Horror',
   'Comedy, Drama',
   'Fantasy, Horror',
   'Comedy, Drama']},
 1987: {'Year': [1987, 1987, 1987, 1987, 1987, 1987],
  'Title': ['1+1=12+1 (Cheaper By The Dozen)',
   'Action Is Not Missing (Crack Platoon)',
   'Anak Badjao',
   'Huwag Mong Buhayin ang Bangkay',
   'Olongapo...The Great American Dream',
   'The Untold Story of Melanie Marquez'],
  'Starring': ['Susan Roces, Eddie Gutierrez, Roderick Paulate, Lotlot de Leon, Sheryl Cruz, Matet de Leon, Dranreb Belleza, Brylle Mondejar, Jigo Garcia, Jaypee De Guzman, Chuckie Dreyfus, Katrin Gonzales, Isabel Granada, RR Herrera',
   "Dolphy, Paquito Diaz, Carlos 'Sonny' Padilla, Manny Boy Quizon, Freddie Quizon, Edgar Quizon, Rommel Valdez Dick Israel, Renato del Prado, Max Vera, Francis Magalona",
   'Anthony Alonzo, Charito Solis, Tony Santos Sr., Liza Lorena, Alicia Alonzo, Marithes Samson, Isadora',
   'Charito Solis, Ricky Davao, Pinky Suarez, Jestoni Alarcon, Rita Avila, Jojo Alejar, Romnick Sarmenta, Jennifer Sevilla',
   'Jacklyn Jose, Chanda Romero, Joel Torre, Susan Africa, Marilou Sadiua, Pete Cooper, Raoul Aragonn, Joy Virata',
   'Melanie Marquez, Caridad Sanchez, Tony Santos Sr., Rosemarie Gil, Rene Salud, Totoy Marquez, Maya dela Cuesta'],
  'Studio': ['Regal Films',
   'RVQ Productions',
   'Triple A Films',
   'Seiko Films',
   'Asian American Film Institute',
   'Miracle Boy Films International'],
  'Director': ['Mike Relon Makiling',
   'Angel Labra',
   'Jose Antonio Alonzo and Jerry Tirazona',
   'Mauro Gia Samonte|ceb|Mauro Samonte',
   'Chito Roño',
   'Artemio Marquez'],
  'Genre': ['Comedy, Family',
   'Action, Comedy',
   'Action, Drama',
   'Drama, Horror, Thriller',
   'Drama',
   'Drama']},
 1988: {'Year': [1988, 1988, 1988, 1988, 1988, 1988],
  'Title': ['Agila ng Maynila',
   'Celestina Sanchez, Alyas Bubbles – Enforcer: Ativan Gang',
   'Itanong Mo sa Buwan',
   'Magkano ang Iyong Dangal?',
   'Patrolman',
   'Pik Pak Boom'],
  'Starring': ['Fernando Poe, Jr., Vic Vargas, R.R. Herrera, Paquito Diaz, Charlie Davao, Encar Benedicto, Raoul Aragon, Dencio Padilla, Vic Diaz, Lito Anzures, Lucita Soriano, Mario Escudero, Nello Nayo and Max Alvarado',
   'Amy Austria, Miguel Rodriguez, Jacklyn Jose, Isadora, Lola Rodriguez, Gina Pareño',
   'Mark Gil, Anjo Yllana and Jacklyn Jose',
   'Christopher de Leon, Zsa Zsa Padilla, Joel Torre, Jestoni Alarcon, Princess Punzalan, Michael Locsin',
   'Baldo Marro, Melissa Mendez, Sunshine, Raoul Aragonn, Dick Israel, Zandro Zamora, Odette Khan',
   'Herbert Bautista, Lea Salonga, Bing Loyzaga, Regine Velasquez, Lilet, Dingdong Avanzado, Richard Reynoso, Jayjay Salvador, Cathy Moira'],
  'Studio': ['FPJ Productions',
   'Golden Lion Films',
   'Double M Films',
   'Seiko Films',
   'El Niño Films',
   'VIVA Films'],
  'Director': ['Pablo Santiago',
   'Carlo J. Caparas',
   'Chito Roño',
   'Laurice Guillen',
   'Cesar SB. Abella',
   'Leroy Salvador'],
  'Genre': ['Action',
   'Action, Drama',
   'Crime, Drama, Thriller',
   'Drama',
   'Action, Drama',
   'Romance, Comedy, Fantasy']},
 1989: {'Year': [1989, 1989, 1989, 1989, 1989, 1989],
  'Title': ['Ang Bukas ay Akin (Langit ay Uusig)',
   'Imortal',
   'Irosin: Pagputok ng Araw...Babaha ng Dugo!',
   'Last Two Minutes',
   'Ang Mahiwagang Daigdig ni Elias Paniki',
   'Tupang Itim'],
  'Starring': ['Gretchen Barretto, Jestoni Alarcon, Cherie Gil, Cesar Montano, Isabel Rivas',
   'Vilma Santos, Christopher de Leon, Cherie Gil, Liza Lorena, Tommy Abuel, Ricky Davao, Jaclyn Jose, Gelli de Belen, Roi Vinzon, Richard Reynoso, Raoul Aragonn',
   'Anthony Alonzo, Aga Muhlach, Ricky Davao, Paquito Diaz, Chat Silayan, Monica Herrera, Philip Gamboa, Aurora Sevilla',
   'Alvin Patrimonio, Paul Alvarez, Jerry Codinera, Roderick Paulate, Rene Requiestas, Ruffa Gutierrez, Carmina Villaroel, Aiko Melendez, Aljon Jimenez, Jeffrey Santos, Zoren Legaspi, Janice de Belen, Alice Dixson, Maricel Laxa, Benjie Paras',
   'Ramon Revilla, Sr., Maria Isabel Lopez, Atong Redillas, Tina Godinez, Brylle Mondejar',
   'Jess Lapid, Jr., Tetchie Agbayani, Eddie Garcia, Marissa Delgado, Jun Austria, Glaiza Herradura'],
  'Studio': ['Seiko Films',
   'VIVA Films',
   "D'Wonder Films",
   'Regal Films',
   'Golden Lion Films and Pozon Films',
   'Patricia Films International'],
  'Director': ['Laurice Guillen',
   'Eddie Garcia',
   'Augusto Buenaventura',
   'Mike Relon Makiling',
   'Carlo J. Caparas',
   'Joe Balagtas'],
  'Genre': ['Drama',
   'Drama',
   'Action, Drama',
   'Comedy, Fantasy, Sport',
   'Adventure, Action, Fantasy, Horror',
   'Action, Drama']},
 1990: {'Year': [1990, 1990, 1990, 1990, 1990, 1990],
  'Title': ['Ama...Bakit Mo Ako Pinabayaan?',
   'Andrea, Paano Ba ang Maging Isang Ina?',
   'Baril Ko... ang Uusig!',
   'Espadang Patpat',
   'Shake, Rattle & Roll II',
   'Tumakbo Ka... Hanggang May Lupa'],
  'Starring': ['Jestoni Alarcon, Gretchen Barretto, Sheryl Cruz, Ricky Belmonte, Robert Arevalo, Marita Zobel, Suzanne Gonzales, Jovit Moya',
   'Nora Aunor, Gina Alajar, Lloyd Samartino, Dan Alvaro, Perla bautista, RR Herrera, Melissa Mendez, Juan Rodrigo, Susan Africa',
   'Ronnie Ricketts, Eddie Garcia, Michael de Mesa, Beverly Vergel, Sheila Ysrael, Maylene Zapanta',
   'Dolphy, Vandolph, Panchito, Marissa Delgado, Romy Diaz and Eric Quizon, Liz Alindogan, Charmaine Arnaiz, Leni Santos',
   'Episode 1: "Multo"- Cast- Janice de Belen, Eric Quizon, Eddie Gutierrez, Isabel Granada; Episode 2: "Kulam" – Cast- Joey Marquez, Carmina Villaroel, Daisy Romualdez, Sylvia Sanchez; Episode 3: "Aswang"- Cast- Manilyn Reynes, Aljon Jimenez, Ana Roces, Anjo Yllana, Richard Gomez, Vangie Labalan, Rez Cortez',
   'Baldo Marro, Isadora, Rhey Roldan, Dick Israel, Carol Dauden, Bobby Benitez, Robert Talby'],
  'Studio': ['Seiko Films',
   'MRN Film International',
   'Lea Productions',
   'AMS Productions',
   'Regal Films',
   'ATB-4 Films International'],
  'Director': ['Lino Brocka',
   'Gil Portes',
   'Jose N. Carreon',
   'Efren Jarlego',
   'Peque Gallaga and Lore Reyes',
   'Leonardo Pascual and Baldo Marro'],
  'Genre': ['Drama',
   'Drama',
   'Action',
   'Action, Comedy, Drama, Fantasy, Family',
   'Horror, Comedy',
   'Action']},
 1991: {'Year': [1991, 1991, 1991, 1991, 1991, 1991, 1991],
  'Title': ['Contreras Gang',
   'Darna',
   'Juan Tamad at Mister Shooli: Mongolian Barbecue',
   'Magdalena S. Palacol Story',
   'Okay Ka, Fairy Ko!',
   'Ang Totoong Buhay ni Pacita M.',
   'Shake, Rattle & Roll III'],
  'Starring': ['Edu Manzano, Cristina Gonzales, Johnny Delgado, Monsour del Rosario, Rez Cortez, Willie Revillame, Dindo Arroyo, Eric Francisco, Kevin Delgado',
   'Nanette Medved, Edu Manzano, Nida Blanca, Bing Loyzaga, Dennis Padilla, Atong Redillas, Pilar Pilapil and Tonton Gutierrez',
   'Eric Quizon, Jun Urbano, Jackie Lou Blanco, Leo Martinez, Caridad Sanchez, Berting Labra, Lou Veloso',
   'Alma Moreno, Tobi Alejar, Gardo Versoza, Allan Paule, Robert Arevalo, Orestes Ojeda, Alicia Alonzo',
   'Vic Sotto, Aiza Seguerra, Tweetie de Leon, Larry Silva, Tetchie Agbayani, Ruby Rodriguez, Jinky Oda, Bayani Casimiro, Jr., Charito Solis',
   'Nora Aunor, Lotlot de Leon, Armida Siguion-Reyna, Juan Rodrigo, Marissa Delgado, Subas Herrero, Dexter Doria',
   'Kris Aquino, Manilyn Reynes, Janice de Belen, Ogie Alcasid, Rosemarie Gil, Joey Marquez, Gina Alajar, Joel Torre'],
  'Studio': ['Moviestars Production',
   'Viva Films',
   'FLT Films International',
   'Omega Releasing Organization',
   'M-Zet Productions',
   'MRN Film International',
   'Regal Films'],
  'Director': ['Pepe Marcos',
   'Joel Lamangan',
   'Jun Urbano',
   'Junn Cabreira',
   'Tony Y. Reyes and Bert de Leon',
   'Elwood Perez',
   'Peque Gallaga and Lore Reyes'],
  'Genre': ['Action',
   'Action, Comedy, Superhero, Horror',
   'Drama, Comedy',
   'Action, Drama',
   'Action, Comedy, Fantasy',
   'Drama',
   'Horror, Comedy']},
 1992: {'Year': [1992, 1992, 1992, 1992, 1992, 1992],
  'Title': ['Andres Manambit: Angkan ng Matatapang',
   'Bakit Labis Kitang Mahal',
   'Engkanto',
   'Okay Ka, Fairy Ko! Part 2',
   'Shake, Rattle & Roll IV',
   'Takbo... Talon... Tili!!!'],
  'Starring': ['Eddie Garcia, Eddie Gutierrez, Pinky de Leon, Kier Legaspi, Joko Diaz, Mia Pratts, Ramon Christopher',
   'Aga Muhlach, Ariel Rivera, Lea Salonga',
   'Janice de Belen, Francis Magalona, Raul Zaragosa, Maila Gumila, Emon Ramos, Apple Pie Bautista, Suisan Lozada, Cita Astals, Roderick Paulate',
   'Vic Sotto, Aiza Seguerra, Tweetie de Leon, Tetchie Agbayani, Ruby Rodriguez, Jinky Oda, Bayani Casimiro Jr., Charito Solis',
   'Episode 1: "Guro" - Manilyn Reynes, Edu Manzano, Sunshine Cruz, Aljon Jimenez and Nida Blanca; Episode 2: "Kapitbahay" - Aiza Seguerra, Janice de Belen, Al Tantay and Lady Lee; Episode 3: "Madre" - Gina Alajar, Aiko Melendez, Miguel Rodriguez, Ai Ai delas Alas, IC Mendoza, Bella Flores, Lilia Cuntapay',
   'Rene Requiestas, Rita Avila, Raymond Gutierrez, Richard Gutierrez, Sheryl Cruz, Romnick Sarmenta'],
  'Studio': ['VIVA Films',
   'Octoarts Films',
   'Double M Films',
   'Regal Films and M-Zet Productions',
   'Regal Films',
   'Seiko Films'],
  'Director': ['Ike Jarlego, Jr.',
   'Jose Javier Reyes',
   'Tata Esteban',
   'Tony Reyes',
   'Peque Gallaga and Lore Reyes',
   'Efren Jarlego'],
  'Genre': ['Biography, Action',
   'Drama, Romance',
   'Fantasy',
   'Action, Comedy, Fantasy',
   'Horror',
   'Comedy, Fantasy, Horror']},
 1993: {'Year': [1993, 1993, 1993, 1993, 1993, 1993],
  'Title': ['Doring Dorobo: Hagupit ng Batas',
   'Gaano Kita Kamahal',
   'Inay',
   'Kung Mawawala Ka Pa',
   'May Minamahal',
   'Pusoy Dos'],
  'Starring': ["Eddie Garcia, Eddie Gutierrez, Boots Anson-Roa, Sharmaine Arnaiz, Paquito Diaz, Rey 'PJ' Abellana, Ali Sotto, Dick Israel, & Vivian Foz",
   'Christopher de Leon, Lorna Tolentino, Tirso Cruz III, Daniel Fernando, Juan Rodrigo, Marita Zobel, Sylvia Sanchez, & Ali Sotto',
   'Nora Aunor, Tirso Cruz III, Chanda Romero, Tommy Abuel, Tita Munoz, Orestes Ojeda, Caridad Sanchez, Louella de Cordova, Carol Dauden, Jaclyn Jose, & Melanie Marquez',
   'Christopher de Leon, Dawn Zulueta, Sarah Jane Abad, Amy Austria, Pilar Pilapil',
   'Aga Muhlach, Aiko Melendez, Boots Anson-Roa, Ronaldo Valdez, Aljon Jimenez, John Estrada, Agot Isidro, Liza Lorena, Marita Zobel, Nikka Valencia, Claudine Barretto',
   'Janno Gibbs, Dennis Padilla, Gelli de Belen, Leo Martinez, Mat Ranillo III, Jackie Aquino'],
  'Studio': ['LEA Productions',
   'Pioneer Films',
   'OWNI Films',
   'Reyna Films',
   'Star Cinema and Regal Films',
   'VIVA Films'],
  'Director': ['Augusto Salvador',
   'Butch Perez',
   'Artemio Marquez',
   'Jose Mari Avellana',
   'Jose Javier Reyes',
   'Jun Aristorenas'],
  'Genre': ['Action',
   'Drama',
   'Drama',
   'Drama, Romance',
   'Romance, Comedy',
   'Action, Comedy']},
 1994: {'Year': [1994, 1994, 1994, 1994, 1994, 1994],
  'Title': ['Kanto Boy 2: Anak ni Totoy Guapo',
   'Lucas Abelardo',
   "Mama's Boys 2: Let's Go Na!",
   'Ang Pagbabalik ni Pedro Penduko',
   'Shake, Rattle & Roll V',
   'Wanted: Perfect Father'],
  'Starring': ['Ian Veneracion, Kimberly Diaz, Ramon Christopher, Dick Israel, Bob Soler, Max Laurel, Dencio Padilla, Jimmy Fabregas',
   'Roi Vinzon, Dante Rivero, Bembol Roco, Bob Soler, Karen Timbol, Zando Zamora, King Gutierrez, Conrad Poe, Levy Ignacio, Dencio Padilla',
   'Anjo Yllana, Ogie Alcasid, Michael V., Patrick Guzman, Sunshine Cruz, Lailani Navarro, Aileen Angeles, Jennifer Mendoza',
   'Janno Gibbs, Chiquito, Leo Martinez, Jun Aristorenas, Vina Morales, Donita Rose, Arnmel Ignacio, Malu de Guzman',
   'Episode 1: "Maligno" - Ruffa Gutierrez, Monsour del Rosario; Episode 2: "Anino" - Sheryl Cruz, Jacklyn Jose, Dingdong Dantes, Ogie Diaz; Episode 3: "Impakto" - Manilyn Reynes, Tom Taus, Jr., Chuck Perez, Don Pepot',
   'Dolphy, Dawn Zulueta, Edu Manzano, Babalu, Jeffrey Quizon, Rochelle Barrameda'],
  'Studio': ['Moviestars Productions',
   'Levin Films',
   'OctoArts Films',
   'VIVA Films',
   'Regal Films',
   'Star Cinema'],
  'Director': ['Augusto Salvador',
   'Jose "Kaka" Balagtas',
   'Tony Y. Reyes',
   'J. Erastheo Navoa',
   'Manny Castaneda and Jose Javier Reyes',
   'Efren Jarlego'],
  'Genre': ['Action, Crime, Drama',
   'Action, Crime, Drama',
   'Comedy',
   'Action, Adventure, Comedy, Fantasy',
   'Horror, Comedy',
   'Comedy, Drama, Family']},
 1995: {'Year': [1995, 1995, 1995, 1995, 1995, 1995],
  'Title': ['Dahas',
   'Father en Son',
   'Huwag Mong Isuko ang Laban',
   'Isko: Adventures in Animasia',
   'Magic Kombat',
   'Muling Umawit ang Puso'],
  'Starring': ['Maricel Soriano, Richard Gomez, Tonton Gutierrez, Efren Reyes, Jr., Sheila Ysrael, Maila Gumila',
   'Dolphy, Vandolph, Zsa Zsa Padilla, Babalu, Carmi Martin, Panchito, Judy Ann Santos, Manny Boy Q., Jeffrey Quizon, Bernardo Bernardo, Freddie Quizon',
   'Ronnie Ricketts, Vina Morales, Miguel Rodriguez, Ricardo Cepeda, Janine Barredo, Charina Scott, Melissa Mendez, Edgar Mande',
   'Ogie Alcasid, Michelle van Eimeren, Michael V., Candy Pangilinan, Palito',
   'Smokey Manaloto, Eric Fructuoso, Beth Tamayo, Sharmaine Suarez and Joanne Pascual',
   'Nora Aunor, Albert Martinez, Donna Cruz, Ian de Leon, Ricky Davao, Michael de Mesa, Rita Avila, Jennifer Sevilla'],
  'Studio': ['MAQ Productions',
   'RVQ Productions',
   'Rockets Productions, Inc.',
   'OctoArts Films',
   'Mahogany Pictures',
   'VIVA Films'],
  'Director': ['Chito Roño',
   'Rodolfo V. Quizon',
   'Ronnie Ricketts',
   'Mike Relon Makiling and Gerry Garcia',
   'Junn Cabreira',
   'Joel Lamangan'],
  'Genre': ['Crime, Drama, Thriller',
   'Comedy, Drama, Family',
   'Action',
   'Animation, Adventure, Comedy',
   'Fantasy, Action, Adventure',
   'Drama, Musical']},
 1996: {'Year': [1996, 1996, 1996, 1996, 1996, 1996],
  'Title': ['Aring King King: Ang Bodyguard Kong Sexy',
   'Emong Salvacion: Humanda Ka...Oras Mo Na!',
   'Kahit Kailan: Love Forgives',
   'Magic Temple',
   'Rubberman: Up, Up and Away!',
   'Trudis Liit'],
  'Starring': ['Dolphy, Vandolph, Anjanette Abayari, Tirso Cruz III, Babalu, Jean Garcia, Dick Israel',
   'Eddie Garcia, Gardo Versoza, Raymond Keannu, Beth Tamayo, Giorgia Ortega, Ara Mina & Dennis Roldan',
   'Richard Gomez, Aiko Melendez, Jomari Yllana, Giselle Toengi, Roberto Aviles, Melissa Mendez, Johnny Vicar',
   'Jason Salcedo, Junell Hernando, Marc Solis, Jun Urbano, Anna Larrucea, Gina Pareno, Aljon Jimenez and Jackie Lou Blanco',
   'Michael V., Beth Tamayo, Gloria Romero, Dick Israel, Roy Alvarez, Allan K.',
   'Julio Diaz, Amy Austria, Jean Garcia, Jackie Aquino, Paquito Diaz, Suzette Ranillo, Dick Israel and Agatha Tapan & Marijoy Adorable'],
  'Studio': ['Premiere Entertainment Productions',
   'Regal Films',
   'MAQ Productions',
   'Star Cinema',
   'Octoarts Films and Cinemax Studios',
   'RJ Films'],
  'Director': ['Jett Espiritu',
   "Francis 'Jun' Posadas",
   'Maryo J. de los Reyes',
   'Lore Reyes and Peque Gallaga',
   "Edgardo 'Boy' Vinarao",
   'Jett Espiritu'],
  'Genre': ['Comedy',
   'Action',
   'Drama',
   'Action, Adventure, Fantasy',
   'Comedy, Action, Superhero',
   'Drama']},
 1997: {'Year': [1997, 1997, 1997, 1997, 1997, 1997],
  'Title': ['Adarna: The Mythical Bird',
   'Anak ng Bulkan',
   'Babae',
   'Magic Kingdom: Ang Alamat ng Damortis',
   'Nasaan ang Puso',
   'Padre Kalibre'],
  'Starring': ['Voices of: Jolina Magdangal, Marvin Agustin, Boots Anson-Roa, Martin Nievera and Regine Velasquez',
   'Tom Taus, Jr., Beth Tamayo, Amy Austria, Bembol Roco, Lloyd Samartino, Denise Joaquin',
   'Nora Aunor, Judy Ann Santos, Jao Mapa, Nida Blanca, Mark Gil, Luis Gonzales',
   'Jason Salcedo, Junell Hernando, Janus del Prado, Anne Curtis, Mark Gil, William Martinez, Ramon Christopher, Maricel Laxa',
   'Christopher de Leon, Maricel Soriano, Judy Ann Santos, Spencer Reyes, Gina Pareno, Ronaldo Valdez',
   'Eddie Garcia, Monsour Del Rosario, Eddie Gutierrez, Dan Fernandez and Aya Medel'],
  'Studio': ['Guiding Light Productions and FLT Films International',
   'Premiere Entertainment Productions',
   'GEM',
   'VIVA Films and Neo Films',
   'MAQ Productions',
   'Virtual Cinema International'],
  'Director': ['Geirry A. Garccia',
   'Cirio H. Santiago',
   'Lupita Kashiwahara',
   'Peque Gallaga and Lore Reyes',
   'Chito Roño',
   'Val Iglesias'],
  'Genre': ['Musical, fantasy, adventure',
   'Adventure, drama, family, sci-fi',
   'Drama',
   'Action, adventure, fantasy',
   'Drama',
   'Action, drama']},
 1998: {'Year': [1998, 1998, 1998, 1998, 1998, 1998],
  'Title': ['Babae sa Bintana',
   'Hiwaga ng Panday',
   'José Rizal',
   'Kasal-kasalan Sakalan',
   'Puso ng Pasko',
   'Sambahin ang Ngalan Mo'],
  'Starring': ['Richard Gomez, Rosanna Roces, John Estrada, Johnny Manahan, Efren Reyes, Jr., Raymond Keannu',
   'Jinggoy Estrada, Kris Aquino, Robert Arevalo, Gloria Sevilla, Bernadette Allyson, Nards Belen, Benedict Aquino, Martin Gutierrez, Caloy Salvador, Bernard Fabiosa and Celso Ad. Castillo',
   'Cesar Montano, Gloria Diaz, Joel Torre, Jaime Fabregas, Gardo Versoza, Gina Alajar, Tanya Gomez, Ronnie Lazaro, Subas Herrero, Pen Medina, Chinchin Gutierrez, Monique Wilson, Jhong Hilario, Mickey Ferriols',
   'Judy Ann Santos, Wowie de Guzman, Gladys Reyes, Christopher Roxas, Elizabeth Oropesa, Ai-Ai delas Alas, Tiya Pusit, Berting Labra',
   'Jolina Magdangal, Jaclyn Jose, Rita Avila, Cherry Pie Picache, Edu Manzano, Jason Salcedo, Anna Larrucea, Emman Abeleda, Justin Simoy, Korinne Lirio',
   'Eddie Garcia, Christopher de Leon, Jomari Yllana and Alice Dixson'],
  'Studio': ['Regal Films',
   'Golden Lions Films',
   'GMA Films',
   'Solar Films',
   'Star Cinema',
   'MAQ Productions and EG Productions'],
  'Director': ['Chito Roño',
   'Carlo J. Caparas',
   'Marilou Diaz-Abaya',
   'Edgardo "Boy" Vinarao',
   'Peque Gallaga and Lore Reyes',
   'José Carreon'],
  'Genre': ['Action, Drama, Thriller',
   'Action, Fantasy',
   'Biography, Drama, History',
   'Drama, Romance',
   'Comedy, Fantasy',
   'Action, Drama']},
 1999: {'Year': [1999, 1999, 1999, 1999, 1999, 1999],
  'Title': ['Ako ang Lalagot ng Hininga Mo',
   'Bulaklak ng Maynila',
   'Esperanza: The Movie',
   'Muro-Ami',
   'Pepeng Agimat',
   'Sa Piling ng Aswang'],
  'Starring': ['Dan Alvaro, Dante Varona, Rufa Mae Quinto, Daisy Reyes, Erick Esguerra, Bernard Bonnin, Maggie dela Riva, Erie Zarate, Gabriel Romulo, Nick Romano, Romy Diaz, Roberto Gonzales',
   'Christopher de Leon, Elizabeth Oropesa, Angelu de Leon, Jomari Yllana, Bembol Roco, Jake Roxas',
   'Judy Ann Santos, Wowie de Guzman, Piolo Pascual, Dante Rivero, Charo Santos-Concio, Joel Torre, Marvin Agustin, Lito Legaspi, Bembol Roco, Elizabeth Oropesa, Rosa Rosal, Chat Silayan, Angelika dela Cruz',
   'Cesar Montano, Amy Austria, Pen Medina, Jhong Hilario, Rebecca Lusterio',
   "Ramon 'Bong' Revilla, Princess Punzalan, Dennis Padilla, Jess Lapid, Jr., Vanessa del Bianco, Gladys Reyes and LJ Moreno",
   'Maricel Soriano, Gina Alajar, Gardo Versoza and Manilyn Reynes'],
  'Studio': ['MHR Productions',
   'VIVA Films',
   'Star Cinema',
   'GMA Films',
   'Millennium Cinema',
   'Regal Films'],
  'Director': ['Rene Balan',
   'Joel Lamangan',
   'Jerry Lopez Sineneng',
   'Marilou Diaz-Abaya',
   'Felix Dalay',
   'Peque Gallaga and Lore Reyes'],
  'Genre': ['Action, Adventure',
   'Drama',
   'Crime, Drama, Family',
   'Action, Adventure, Drama',
   'Action, Adventure, Fantasy',
   'Horror']},
 2000: {'Year': [2000, 2000, 2000, 2000, 2000, 2000],
  'Title': ['Deathrow',
   'Markova: Comfort Gay',
   'Ping Lacson: Super Cop',
   'Spirit Warriors',
   'Sugatang Puso',
   'Tanging Yaman'],
  'Starring': ['Eddie Garcia, Cogie Domingo, Angelika dela Cruz, Jaclyn Jose, Alan Paule, Pen Medina, Ray Ventura & Ace Espinosa',
   'Dolphy, Eric Quizon, Loren Legarda, Jeffrey Quizon',
   'Rudy Fernandez, Lorna Tolentino, Ricky Davao, Glydel Mercado, Ace Espinosa, Trubador Ramos, Levi Ignacio & Herbert Bautista',
   'Joel Torre, Jhong Hilario, Spencer Reyes, Vhong Navarro, Danilo Barrios, Chris Cruz, Meynard Marcellano, Sherwin Roux, Nikko Manalo, Michael Foz-Sesmundo, Denise Joaquin',
   'Christopher de Leon, Lorna Tolentino, Cherie Gil, Patrick Garcia, Carlo Aquino',
   'Hilda Koronel, Edu Manzano, Dina Bonnevie, Johnny Delgado, Joel Torre, Cherry Pie Picache, Marvin Agustin, Jericho Rosales, Ms. Gloria Romero, Janette McBride, John Prats, Carol Banawa, CJ Ramos, Dominic Ochoa & Shaina Magdayao'],
  'Studio': ['GMA Films',
   'RVQ Productions',
   'Millennium Cinema',
   'MAQ Productions and Roadrunner Network, Inc.',
   'Regal Films',
   'Star Cinema'],
  'Director': ['Joel Lamangan',
   'Gil Portes',
   'Toto Natividad',
   'Chito Rono',
   'Jose Javier Reyes',
   'Laurice Guillen'],
  'Genre': ['Action, Crime, Drama',
   'Biography, Drama, History',
   'Action, Biography',
   'Action, Fantasy, Horror',
   'Drama',
   'Drama, Family']},
 2001: {'Year': [2001, 2001, 2001, 2001, 2001, 2001, 2001],
  'Title': ['Bagong Buwan',
   'Bahay ni Lola',
   'Di Kita Ma-Reach',
   'Hubog',
   'Susmaryosep! Four Fathers',
   'Tatarin',
   "Yamashita: The Tiger's Treasure"],
  'Starring': ['Cesar Montano, Jericho Rosales, Amy Austria, Caridad Sanchez, Jiro Manio, Ronnie Lazaro',
   'Gloria Romero, Aiza Seguerra, Manilyn Reynes, Gina Alajar, James Blanco, Maybelyn dela Cruz, Maxene Magalona, Miko Sotto, Isabella de Leon and Allan K.',
   'Mikey Arroyo, LJ Moreno, Eula Valdez, Tonton Gutierrez, Ana Capri, Raymond Bagatsing',
   'Jay Manalo, Wendell Ramos, Alessandra de Rossi, Assunta de Rossi',
   'Bobby Andrews, Bojo Molina, Polo Ravales, Gerard Madrid',
   'Dina Bonnevie, Edu Manzano, Rica Peralejo, Raymond Bagatsing, Carlos Morales, Patricia Javier',
   'Danilo Barrios, Camille Prats, Armando Goyena, Rustom Padilla, Carlo Munoz, Vic Diaz, Tetsuya Matsui and Albert Martinez'],
  'Studio': ['Star Cinema',
   'Regal Films',
   'ECorp Films',
   'Good Harvest Productions',
   'Maverick Films',
   'Viva Films',
   'MAQ Productions'],
  'Director': ['Marilou Diaz-Abaya',
   'Uro Q. dela Cruz',
   "Wilfredo 'Willy' Milan",
   'Joel Lamangan',
   "Edgardo 'Boy' Vinarao",
   'Tikoy Aguiluz',
   'Chito Rono'],
  'Genre': ['Action, Drama, War',
   'Comedy, Horror, Thriller',
   'Action, Comedy, Drama',
   'Crime, Drama',
   'Comedy, Drama',
   'Drama, Romance',
   'Action, Adventure, Drama, Epic']},
 2002: {'Year': [2002, 2002, 2002, 2002, 2002, 2002, 2002, 2002, 2002],
  'Title': ['Ang Agimat: Anting-Anting ni Lolo',
   'Ang Alamat ng Lawin',
   "Dekada '70",
   'Home Along da Riber',
   'Hula Mo... Huli Ko',
   'Lapu-Lapu',
   'Mano Po',
   'Lastikman',
   'Spirit Warriors: The Shortcut'],
  'Starring': ['Ramon Revilla Sr., Ramon "Jolo" Revilla, Ramon "Bong" Revilla Jr., Mylene Dizon, Carlos Morales, Shaina Magdayao, Nancy Castiglione, Jaime Fabregas, Pen Medina, Goyong, Karylle Padilla & Gina Alajar',
   'Fernando Poe, Jr., Ina Raymundo, Cathy Vilar, Ryan Yamazaki, Franklin Cristobal, Khen Kurillo, Romy Diaz, Augusto Victa, Alex Cunanan, William Romero',
   'Vilma Santos, Christopher de Leon, Piolo Pascual, Marvin Agustin, Carlos Agassi, Danilo Barrios, John Wayne Sace',
   'Dolphy, Jolina Magdangal, Zsa Zsa Padilla, Vandolph, Eddie Gutierrez, Long Mejia, James Blanco, Boy2 Quizon, Michelle Quizon, Palito',
   'Rudy Fernandez and Rufa Mae Quinto',
   'Lito Lapid, Joyce Jimenez, Jeric Raval, Mark Lapid, Roi Vinzon, Jess Lapid Jr., Vic Vargas, Gloria Sevilla, Maria Isabel Lopez, Ian Veneracion, Julio Diaz, Bob Soler, Conrad Poe, Robert Rivera, Dinah Dominguez, Clod Robinson & Dante Rivero',
   'Maricel Soriano, Richard Gomez, Kris Aquino, Ara Mina, Eddie Garcia, Gina Alajar, Boots Anson-Roa, Amy Austria, Tirso Cruz III, Cogie Domingo, Maxene Magalona, Jay Manalo, Eric Quizon',
   'Vic Sotto, Donita Rose, Jeffrey Quizon, Michael V., Michelle Bayle, Ryan Eigenmann, Anne Curtis, Oyo Boy Sotto',
   'Vhong Navarro, Jhong Hilario, Spencer Reyes, Danilo Barrios, Chris Cruz, Gloria Romero, Jaime Fabregas'],
  'Studio': ['Imus Productions',
   'FPJ Productions',
   'Star Cinema',
   'RVQ Productions',
   'Reflection Films',
   'Calinauan Cineworks',
   'Regal Films',
   'Octoarts Films, M-Zet Productions',
   'MAQ Productions, Roadrunner Network, Inc.'],
  'Director': ['Augusto Salvador',
   'Ronwaldo Reyes',
   'Chito Roño',
   'Enrico Quizon',
   "Edgardo 'Boy' Vinarao",
   'William G. Mayo',
   'Joel Lamangan',
   'Tony Y. Reyes',
   'Chito Roño'],
  'Genre': ['Action, adventure, fantasy',
   'Action, adventure, family, fantasy',
   'Period drama',
   'Comedy, drama, musical',
   'Action, comedy',
   'Historical action',
   'Drama',
   'Action, comedy, fantasy, superhero',
   'Action, adventure, fantasy, horror']},
 2003: {'Year': [2003, 2003, 2003, 2003, 2003, 2003, 2003, 2003, 2003],
  'Title': ['Captain Barbell',
   'Crying Ladies',
   'Fantastic Man',
   'Filipinas',
   'Homecoming',
   'Malikmata',
   'Mano Po II: My Home',
   'Bridal Shower',
   'Gagamboy'],
  'Starring': ["Ramon 'Bong' Revilla Jr., Ogie Alcasid, Regine Velasquez, Rufa Mae Quinto, Albert Martinez, Jeffrey Quizon, Snooky Serna, Antonio Aquitania, Sarah Geronimo",
   'Sharon Cuneta, Hilda Koronel, Angel Aquino, Eric Quizon, Ricky Davao, Julio Pacheco, Shamaine Buencamino, Sherry Lara, Raymond Bagatsing',
   'Vic Sotto, Ara Mina, Michael V., Zoren Legaspi, Leo Martinez, Danica Sotto, Alicia Mayer',
   'Maricel Soriano, Richard Gomez, Aiko Melendez, Dawn Zulueta, Victor Neri, Wendell Ramos, Raymond Bagatsing, Andrea del Rosario, Sunshine Dizon, Tanya Garcia, Anne Curtis, Armida Siguion-Reyna and Sarah Geronimo',
   'Elizabeth Oropesa, Bembol Roco, James Blanco and Alessandra de Rossi',
   'Rica Peralejo, Marvin Agustin, Dingdong Dantes, Ricky Davao, Wowie de Guzman, Ana Capri, Nikki Valdez and Ms. Barbara Perez',
   'Susan Roces, Christopher de Leon, Lorna Tolentino, Zsa Zsa Padilla, Kris Aquino, Judy Ann Santos, Carmina Villaroel, Jay Manalo, Cogie Domingo, Richard Gutierrez, Zoren Legaspi, Alessandra de Rossi, Karylle, Chynna Ortaleza, Angel Locsin',
   'Dina Bonnevie, Cherry Pie Picache, Francine Prieto, Christian Vasquez, Rodel Velayo, Boots Anson-Roa, Gina Pareno, Alfred Vargas, Douglas Robinson, Juancho Valentino',
   'Vhong Navarro, Jay Manalo, Aubrey Miles, Long Mejia, Bearwin Meily'],
  'Studio': ['Premiere Entertainment Productions',
   'Unitel Pictures',
   'Octoarts FilmsM-Zet Productions',
   'VIVA Films',
   'Teamwork Productions',
   'Octoarts Films',
   'Regal Films',
   'Seiko Films',
   'MAQ Productions'],
  'Director': ['Mac C. Alejandre',
   'Mark Meily',
   'Tony Y. Reyes',
   'Joel Lamangan',
   'Gil Portes',
   'Jose Javier Reyes',
   'Erik Matti',
   'Jeffrey Jeturian',
   'Erik Matti'],
  'Genre': ['Adventure, Comedy, Fantasy, Superhero',
   'Comedy, Drama',
   'Adventure, Action, Comedy, Superhero',
   'Drama',
   'Drama',
   'Crime, Drama, Horror',
   'Drama',
   'Drama, Comedy, Romance',
   'Action, Comedy, Fantasy, Superhero']},
 2004: {'Year': [2004, 2004, 2004, 2004, 2004, 2004, 2004, 2004],
  'Title': ['Aishite Imasu 1941: Mahal Kita',
   'Enteng Kabisote: Okay ka, Fairy Ko: The Legend',
   'Lastikman: Unang Banat',
   'Mano Po III: My Love',
   'Panaghoy sa Suba',
   'Sigaw',
   'So Happy Together',
   'Spirit of the Glass'],
  'Starring': ['Judy Ann Santos, Raymart Santiago, Jay Manalo, Dennis Trillo, Anita Linda, Ms. Jaclyn Jose, Angelu de Leon, TJ Trinidad, Iya Villania, Marco Alcaraz, Yasmien Kurdi',
   'Vic Sotto, Kristine Hermosa, Michael V., G. Toengi, Jeffrey Quizon, Aiza Seguerra, Bing Loyzaga, Nadine Samonte, Oyo Boy Sotto, January Isaac, Dick Israel, Ruby Rodriguez, Jose Manalo, Wally Bayola, Leila Kuzma',
   'Mark Bautista, Sarah Geronimo, John Estrada, Danilo Barrios, Elizabeth Oropesa, Joel Torre, Mark Gil, Bearwin Meily, Tuesday Vargas, Mikel Campos and Cherie Gil',
   'Vilma Santos, Christopher de Leon, Sheryl Cruz, Jay Manalo, Eddie Garcia, Patrick Garcia, Angel Locsin, Angelica Panganiban, John Prats, Carlo Aquino, Boots Anson-Roa, Jean Garcia, Karylle, Dennis Trillo',
   'Cesar Montano, Juliana Palermo, Jacky Woo, Caridad Sanchez, Joel Torre, Ronnie Lazaro, Daria Ramirez, Rommel Montano, Phil Anthony, Suzette Ranillo and Rebecca Lusterio',
   'Richard Gutierrez, Angel Locsin, Jomari Yllana, James Blanco, Iza Calzado, Ronnie Lazaro, Ella Guevara',
   'Kris Aquino, Eric Quizon, Tonton Gutierrez, Nova Villa, Cogie Domingo, Gloria Diaz, Jennylyn Mercado, Mark Herras, Yasmien Kurdi, Rainier Castillo, Jay-R',
   'Rica Peralejo, Marvin Agustin, Dingdong Dantes, Alessandra de Rossi, Paolo Contis, Ciara Sotto, Ana Capri, Mark Gil, Jay Aquitania, Jake Cuenca, Drew Arellano'],
  'Studio': ['BAS Films',
   'Octoarts Films and M-Zet Productions',
   'VIVA Films',
   'MAQ Productions',
   'CM Films',
   'Regal Films and Megavision Films',
   'Regal Films',
   'OctoArts Films'],
  'Director': ['Joel Lamangan',
   'Tony Y. Reyes',
   'Mac Alejandre',
   'Joel Lamangan',
   'Cesar Montano',
   'Yam Laranas',
   'Joel Lamangan',
   'Jose Javier Reyes'],
  'Genre': ['Drama, Romance, War',
   'Action, Adventure, Comedy, Fantasy',
   'Action, Comedy, Fantasy, Superhero',
   'Drama, Romance',
   'Romance, Drama, War',
   'Horror, Thriller',
   'Comedy, Drama, Romance',
   'Fantasy, Horror, Romance']},
 2005: {'Year': [2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005, 2005],
  'Title': ['Enteng Kabisote 2: Okay Ka Fairy Ko: The Legend Continues',
   'Exodus: Tales from the Enchanted Kingdom',
   'Kutob',
   'Mano Po 4: Ako Legal Wife',
   'Mulawin: The Movie',
   'Shake, Rattle & Roll 2k5',
   'Terrorist Hunter',
   'Blue Moon',
   'Lagot Ka sa Kuya Ko!',
   'Mourning Girls'],
  'Starring': ['Vic Sotto, Kristine Hermosa, Alice Dixson, Oyo Boy Sotto, Bing Loyzaga, Aiza Seguerra, Jeffrey Quizon, Jose Manalo and Victor Neri',
   "Ramon 'Bong' Revilla, Aubrey Miles, Iya Villania, Benjie Paras, Jay-R, Paolo Bediones and BJ 'Tolits' Forbes",
   'Rica Peralejo, Marvin Agustin, Alessandra de Rossi, Andrea del Rosario, Ana Capri, James Blanco, Liza Lorena, Ryan Agoncillo',
   'Rufa Mae Quinto, Cherry Pie Picache, Jay Manalo, Zsa Zsa Padilla, John Prats, JC de Vera, Bianca King',
   'Richard Gutierrez, Angel Locsin, Dingdong Dantes, Dennis Trillo, Iza Calzado, Karylle, Bianca King, Michael de Mesa, Valerie Concepcion, Sunshine Dizon, Carmina Villaroel, Eddie Gutierrez, Zoren Legaspi, Amy Austria',
   'Episode 1: "Poso" - Ai Ai delas Alas, Gloria Romero, Yasmien Kurdi, Rainier Castillo, Marco Alcaraz; Episode 2: "Aquarium" - Ara Mina, Ogie Alcasid, Faisal Daquigan, Wilma Doesnt; Episode 3: "Ang Lihim ng San Joaquin" - Mark Anthony Fernandez, Tanya Garcia, Elizabeth Oropesa, Nonie Buencamino, Ronnie Lazaro',
   'Eddie Garcia, Ronald Gan Ledesma, Dennis Roldan, Maricar de Mesa',
   'Christopher de Leon, Dennis Trillo, Boots Anson-Roa, Eddie Garcia, Jennylyn Mercado, Mark Herras, Pauleen Luna, Polo Ravales',
   'Ronnie Ricketts, Carlos Morales, Angelica Jones, Nadine Samonte, Dinky Doo, Mariz Ricketts, John Apacible',
   'Ricky Davao, Glydel Mercado, Chin Chin Gutierrez, Assunta de Rossi'],
  'Studio': ['Octoarts Films and M-Zet Productions',
   'Imus Productions and Media Asia Distribution',
   'Octoarts Films',
   'Regal Films',
   'Regal Films and GMA Films',
   'Regal Films',
   'Double Impact Films, Leo Films and MMG Entertainment International',
   'Regal Films',
   'Rocketts Productions and RonRick Films',
   'Teamwork Productions and Zennet World Productions'],
  'Director': ['Tony Y. Reyes',
   'Erik Matti',
   'Jose Javier Reyes',
   'Joel Lamangan',
   'Dominic Zapata',
   'Richard Somes, Rico Maria Ilarde',
   'Val Iglesias',
   'Joel Lamangan',
   'Ronnie Ricketts',
   'Gil Portes'],
  'Genre': ['Comedy, family, fantasy',
   'Action, fantasy',
   'Horror, thriller',
   'Comedy, drama',
   'Action, adventure, fantasy',
   'Horror, comedy',
   'Action',
   'Romance, drama',
   'Action',
   'Comedy, drama']},
 2006: {'Year': [2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006, 2006],
  'Title': ['Enteng Kabisote 3: Okay Ka, Fairy Ko: The Legend Goes On and On and On',
   'Kasal, Kasali, Kasalo',
   'Ligalig',
   'Mano Po 5: Gua Ai Di',
   'Matakot Ka sa Karma',
   'Shake, Rattle and Roll 8',
   'Super Noypi',
   'Tatlong Baraha',
   'Zsazsa Zaturnnah Ze Moveeh'],
  'Starring': ['Vic Sotto, Kristine Hermosa, Bing Loyzaga, Oyo Boy Sotto, G. Toengi, Jose Manalo, Aiza Seguerra, Antonio Aquitania and Allan K.',
   'Judy Ann Santos, Ryan Agoncillo, Gina Pareño, Gloria Diaz, Ariel Ureta, AJ Perez',
   'Cesar Montano, Sunshine Cruz, Celia Rodriguez, Johnny Delgado, Katya Santos, Bayani Agbayani, John Regala, Rebecca Lusterio, Rommel Montano, Alvin Anson, Kalila Aguilos & Gwen Garci',
   'Richard Gutierrez, Angel Locsin, Lorna Tolentino, Gina Alajar, Kamilah Daquigan, Boots Anson-Roa, Christian Bautista',
   'Rica Peralejo, Angelica Panganiban, Gretchen Barretto, Tanya Garcia, Bianca King & Ana Capri',
   'Episode 1: "13th Floor" - Bearwin Meily, Keanna Reeves, Roxanne Guinoo, Joseph Bitangcol, Janus Del Prado; Episode 2: "Yaya" - Sheryl Cruz, Iza Calzado, TJ Trinidad, Nene Tamayo, Nash Aguas, Boom Antonio, Debraliz Vallasote; Episode 3: "LRT" - Manilyn Reynes, Keempee de Leon, Eugene Domingo, Miko Palanca, IC Mendoza, Mhyco Aquino, Sergio Garcia, Cassandra Ponti, Ehra Madrigal, Empress Schuck, Dino Imperial',
   'John Prats, Sandara Park, Polo Ravales, Katrina Halili, Jennylyn Mercado, Mark Herras, Monsour del Rosario, Aubrey Miles, Karla Estrada, Jao Mapa, Mon Confiado, Allan Paule',
   'Lito Lapid, Mark Lapid, Maynard Lapid, Phoemela Barranda, Girlie Revilla, Jackie Rice, Bearwin Meily & Monsour del Rosario',
   'Pops Fernandez, Rustom Padilla, Alfred Vargas, Pauleen Luna, Chokoleit, Christian Vasquez, Say Alonzo and Zsa Zsa Padilla'],
  'Studio': ['OctoArts Films and M-Zet Productions',
   'Star Cinema',
   'CM Films',
   'Regal Films',
   'Canary Films and OctoArts Films',
   'Regal Films',
   'Regal Films',
   'Violett Films',
   'Regal Films'],
  'Director': ['Tony Y. Reyes',
   'Jose Javier Reyes',
   'Cesar Montano',
   'Joel Lamangan',
   'Jose Javier Reyes',
   'Mike Tuviera, Rahyan Carlos and Topel Lee',
   'Quark Henares',
   'Toto Natividad',
   'Joel Lamangan'],
  'Genre': ['Action, Comedy, Family, Fantasy',
   'Comedy, Romance',
   'Action, Crime, Drama',
   'Comedy, Drama, Romance',
   'Horror',
   'Horror, Comedy',
   'Action, Adventure, Fantasy, Superhero',
   'Action',
   'Action, Adventure, Comedy, Superhero']},
 2007: {'Year': [2007, 2007, 2007, 2007, 2007, 2007, 2007, 2007, 2007],
  'Title': ['Bahay Kubo: A Pinoy Mano Po!',
   'Enteng Kabisote 4: Okay Ka, Fairy Ko: The Beginning of the Legend',
   'Katas ng Saudi',
   'Resiklo',
   'Sakal, Sakali, Saklolo',
   'Shake, Rattle and Roll 9',
   'Anak ng Kumander',
   'Banal',
   'Desperadas'],
  'Starring': ['Maricel Soriano, Eric Quizon, Marian Rivera, Shaina Magdayao, Yasmien Kurdi, Eugene Domingo, Mark Herras, Rayver Cruz, Jiro Manio, Bearwin Meily and Ms. Gloria Romero',
   'Vic Sotto, Kristine Hermosa, Ian Veneracion, Francine Prieto, Carlos Agassi, Michael de Mesa, Oyo Boy Sotto, G. Toengi, Jose Manalo, Aiza Seguerra, Ruby Rodriguez and Peque Gallaga',
   'Jinggoy Estrada, Lorna Tolentino, Bayani Agbayani, Eugene Domingo, Arron Villaflor, Julian Estrada, Shaina Magdayao, Rayver Cruz',
   "Ramon 'Bong' Revilla, Jr., Dingdong Dantes, Jennylyn Mercado, Bobby Andrews, Jairus Aquino, Paolo Contis, Ella Cruz, Mylene Dizon, King Gutierrez, Michelle Madrigal, Bryan Revilla, Jolo Revilla, Ramjen Revilla, TJ Trinidad, Benjie Paras, Roi Vinzon",
   'Judy Ann Santos, Ryan Agoncillo, Ms. Gloria Diaz, Ms. Gina Pareno, Ariel Ureta, Dominic Ochoa, Juliana Palermo, Derek Ramsay',
   'Episode 1: "Christmas Tree" - Nash Aguas, Gina Alajar, Sophia Baars, Tonton Gutierrez, John Lapus, Lovi Poe, John Prats, Boots Anson-Roa; Episode 2: "Bangungot" - Jason Gainza, Roxanne Guinoo, Jaymee Joaquin, Pauleen Luna, Andrea Torres; Episode 3: "Engkanto" - Jojo Alejar, Sam Concepcion, Mart Escudero, Matt Evans, Katrina Halili, Hector Macaso, Jewel Mische, Melissa Ricks, Felix Roco',
   'Manny Pacquiao, Ara Mina, Valerie Concepcion, Lara Morena, Roi Vinzon, Dante Rivero, Perla Bautista, Efren Reyes Jr.',
   "Christopher de Leon, Paolo Contis, Alfred Vargas, Paolo Paraiso, Cassandra Ponti, Ina Alegre, Joey 'Pepe' Smith, Pen Medina",
   'Ruffa Gutierrez, Rufa Mae Quinto, Iza Calzado, Marian Rivera, Wendell Ramos, Jay-R, TJ Trinidad'],
  'Studio': ['Regal Films',
   'Octoarts Films and M-Zet Productions',
   'Maverick Films',
   'Imus Productions and Media Asia Films',
   'Star Cinema',
   'Regal Films',
   'MP Promotions',
   'ComGuild Productions',
   'Regal Films'],
  'Director': ['Joel Lamangan',
   'Tony Y. Reyes',
   'Jose Javier Reyes',
   'Mark Reyes',
   'Jose Javier Reyes',
   'Paul Daza, Topel Lee and Mike Tuviera',
   'Jose "Kaka" Balagtas',
   'Cesar Apolinario',
   'Joel Lamangan'],
  'Genre': ['Comedy, Drama',
   'Action, Adventure, Comedy',
   'Comedy, Drama',
   'Action, Fantasy, Sci-Fi',
   'Comedy, Romance',
   'Horror',
   'Action',
   'Action, Drama',
   'Comedy, Romance']},
 2008: {'Year': [2008, 2008, 2008, 2008, 2008, 2008, 2008, 2008],
  'Title': ['Baler',
   'Dayo: Sa Mundo ng Elementalia',
   'Desperadas 2',
   'Iskul Bukol 20 Years After: The Ungasis and Escaleras Adventure',
   'Magkaibigan',
   'One Night Only',
   'Shake, Rattle & Roll X',
   "Ang Tanging Ina N'yong Lahat"],
  'Starring': ['Anne Curtis, Jericho Rosales, Rio Locsin, Phillip Salvador, Nikki Bacolod & Carlo Aquino',
   'Nash Aguas, Katrina Legaspi, Michael V., Noel Trinidad, Nova Villa, Pokwang & Johnny Delgado',
   'Ruffa Gutierrez, Rufa Mae Quinto, Iza Calzado, Marian Rivera, Ogie Alcasid, Will Devaughn & Nova Villa',
   'Vic Sotto, Joey de Leon, Tito Sotto, Sharon Cuneta, Nicole Uysiuseng, Keempee de Leon, Bianca King, Francine Prieto & Benjie Paras',
   'Jinggoy Estrada, Christopher de Leon, Dawn Zulueta, AJ Perez, Empress Schuck & Maricel Laxa',
   'Katrina Halili, Alessandra de Rossi, Diana Zubiri, Jennylyn Mercado, Valerie Concepcion, Jon Avila, Ricky Davao, Empoy Marquez, Jayson Gainza, Joross Gamboa, Ogie Diaz & Chokoleit',
   'Marian Rivera, Diana Zubiri, Jennica Garcia, Kim Chiu, Gerald Anderson, Jean Garcia, Mylene Dizon, JC De Vera, Roxanne Guinoo, Denise Laurel & Wendell Ramos',
   'Ai-Ai de las Alas, Eugene Domingo, Shaina Magdayao, Carlo Aquino, Serena Dalrymple, Alwyn Uytingco, and Jiro Manio'],
  'Studio': ['Viva Communications',
   'Cutting Edge Productions',
   'Regal Entertainment',
   'OctoArts Films',
   'Maverick Films',
   'Canary Films',
   'Regal Entertainment',
   'Star Cinema'],
  'Director': ['Mark Meily',
   'Robert Quilao',
   'Joel Lamangan',
   'Tony Y. Reyes',
   'Jose Javier Reyes',
   'Jose Javier Reyes',
   'Mike Tuviera and Topel Lee',
   'Wenn V. Deramas'],
  'Genre': ['Drama, Romance, War',
   'Animation, Adventure, Fantasy',
   'Comedy, Romance',
   'Action, Adventure, Comedy',
   'Drama',
   'Comedy',
   'Horror, Comedy, Fantasy',
   'Comedy']},
 2009: {'Year': [2009, 2009, 2009, 2009, 2009, 2009, 2009],
  'Title': ['Ang Darling Kong Aswang',
   'I Love You, Goodbye',
   'Nobody, Nobody But... Juan',
   "Mano Po 6: A Mother's Love",
   'Ang Panday',
   'Shake, Rattle & Roll 11',
   'Wapakman'],
  'Starring': ['Vic Sotto, Cristine Reyes, Agot Isidro, Rafael Rosell, Jean Garcia',
   'Angelica Panganiban, Gabby Concepcion, Kim Chiu, Derek Ramsay',
   'Dolphy, Gloria Romero, Eddie Garcia, Pokwang, Eugene Domingo, Heart Evangelista, Eric Quizon, Vandolph, Epi Quizon',
   'Sharon Cuneta, Kris Aquino, Zsa Zsa Padilla, Christopher De Leon, Heart Evangelista, Dennis Trillo, Ciara Sotto, Glaiza de Castro',
   'Bong Revilla, Iza Calzado, Rhian Ramos, Buboy Villar, Phillip Salvador, Geoff Eigenmann',
   'Ruffa Gutierrez, Zoren Legaspi, Megan Young, Jennica Garcia, Mart Escudero, Maja Salvador, Iya Villania, Bangs Garcia',
   'Manny Pacquiao, Krista Ranillo, Rufa Mae Quinto, Benjie Paras, Bianca King, Polo Ravales'],
  'Studio': ['OctoArts Films, APT Entertainment and M-Zet TV Productions',
   'Star Cinema',
   'RVQ Productions',
   'Regal Entertainment',
   'GMA Films and Imus Productions',
   'Regal Entertainment',
   'Solar Entertainment Corporation'],
  'Director': ['Tony Y. Reyes',
   'Laurice Guillen',
   'Enrico S. Quizon',
   'Joel Lamangan',
   'Mac Alejandre',
   'Michael Perez, Jessel Monterverde, Rico Gutierrez',
   'Topel Lee'],
  'Genre': ['Comedy, Horror',
   'Drama',
   'Comedy',
   'Drama',
   'Action, Adventure, Comedy, Fantasy',
   'Horror',
   'Action, Comedy, Sci-Fi, Superhero']},
 2010: {'Year': [2010, 2010, 2010, 2010, 2010, 2010, 2010, 2010],
  'Title': ['Si Agimat at si Enteng Kabisote',
   'Dalaw',
   'Father Jejemon',
   'Rosario',
   'RPG: Metanoia',
   'Shake, Rattle and Roll 12',
   'Super Inday and the Golden Bibe',
   "Ang Tanging Ina Mo (Last na 'To!)"],
  'Starring': ['Vic Sotto, Bong Revilla, Gwen Zamora, Sam Pinto, Oyo Boy Sotto, Bing Loyzaga, and Amy Perez',
   'Kris Aquino, Diether Ocampo, Alessandra de Rossi, Karylle, Empress Schuck, Gina Pareño, Ina Feleo, Susan Africa, and Maliksi Morales',
   'Dolphy, Maja Salvador, Ejay Falcon, Efren Reyes, Vandolph, and Jeffrey Quizon',
   'Jennylyn Mercado, Dennis Trillo, Yul Servo, Sid Lucero, Isabel Oli, Phillip Salvador, Tonton Gutierrez, Eula Valdez, Liza Lorena, Dolphy',
   'Vhong Navarro, Aga Muhlach, Eugene Domingo, Mika Dela Cruz & Zaijian Jaranilla',
   'Carla Abellana, Rayver Cruz, Andi Eigenmann, John Lapus, Sid Lucero & Shaina Magdayao',
   'Marian Rivera, John Lapus, Jake Cuenca, Pokwang, Cherry Pie Picache, Jestoni Alarcon, Mylene Dizon, Sheena Halili, Buboy Villar, Sabrina Man, Jairus Aquino, Irma Adlawan, Elijah Alejo',
   'Ai-Ai delas Alas, Eugene Domingo, Marvin Agustin, Carlo Aquino, Shaina Magdayao, Nikki Valdez, Mark Acueza, Yuuki Kadooka, Tonton Gutierrez, Jestoni Alarcon, Dennis Padilla, Serena Dalrymple, Jiro Manio, Alwyn Uytingco, Xyriel Manabat, Empoy Marquez, John Avila, Rafael Rosell IV, Ricci Chan, Andoy Ranay, and Cherry Pie Picache'],
  'Studio': ['GMA Films, OctoArts Films, APT Entertainment, M-Zet Productions & Imus Productions',
   'CineMedia & Star Cinema',
   'RVQ Productions Inc.',
   'Cinemabuhay & Studio5',
   'AmbientMedia & Star Cinema',
   'Regal Entertainment Inc. & Regal Multimedia Inc.',
   'Regal Entertainment',
   'Star Cinema'],
  'Director': ['Tony Y. Reyes',
   'Dondon S. Santos',
   'Frank Grey Jr.',
   'Albert Martinez',
   'Luis C. Suarez',
   'Zoren Legaspi, Jerrold Tarog & Topel Lee',
   'Mike Tuviera',
   'Wenn V. Deramas'],
  'Genre': ['Action, Comedy, Fantasy',
   'Suspense, Thriller',
   'Dramedy',
   'Drama',
   'Comedy, Fantasy, Adventure',
   'Horror',
   'Comedy, Fantasy, Adventure',
   'Dramedy']},
 2011: {'Year': [2011, 2011, 2011, 2011, 2011, 2011, 2011],
  'Title': ['Enteng ng Ina Mo',
   'My House Husband: Ikaw Na!',
   'Ang Panday 2',
   'Manila Kingpin: The Asiong Salonga Story',
   'Segunda Mano',
   'Shake, Rattle & Roll 13',
   'Yesterday, Today, Tomorrow'],
  'Starring': ['Vic Sotto, Ai-Ai delas Alas',
   'Judy Ann Santos, Ryan Agoncillo, and Eugene Domingo',
   'Bong Revilla, Phillip Salvador, Marian Rivera, Iza Calzado, Rhian Ramos, Kris Bernal, and Eddie Garcia',
   'George "E.R" Estregan, Carla Abellana',
   'Kris Aquino, Angelica Panganiban, and Dingdong Dantes',
   'Maricar Reyes, Zanjoe Marudo, Bugoy Cariño, Kathryn Bernardo, Louise delos Reyes, Sam Concepcion, Eugene Domingo, and Edgar Allan Guzman',
   'Maricel Soriano, Gabby Concepcion, Jericho Rosales, Dennis Trillo, Carla Abellana, Lovi Poe, Paulo Avelino, Solenn Heussaff, Agot Isidro, and Eula Caballero'],
  'Studio': ['Star Cinema, OctoArts Films, M-Zet Films, APT Entertainment',
   'OctoArts Films',
   'GMA Films, Imus Productions',
   'VIVA Communications Inc., Scenema Concept International',
   'Star Cinema, MJM Productions, Agostodos Pictures',
   'Regal Entertainment Inc.',
   'Regal Entertainment Inc., Studio5'],
  'Director': ['Tony Y. Reyes',
   'Jose Javier Reyes',
   'Mac Alejandre',
   'Tikoy Aguiluz',
   'Joyce Bernal',
   'Chris Martinez("Rain, Rain Go Away"), Richard Somes ("Tamawo"), Jerrold Tarog ("Parola")',
   'Jun Lana'],
  'Genre': ['ActionFantasy/AdventureComedy-Drama',
   'Comedy-Drama',
   'ActionFantasy/Adventure',
   'ActionDrama',
   'Horror/Suspense',
   'Horror',
   'Drama']},
 2012: {'Year': [2012, 2012, 2012, 2012, 2012, 2012, 2012, 2012],
  'Title': ['Si Agimat, si Enteng Kabisote at si Ako',
   'El Presidente: General Emilio Aguinaldo Story and the First Philippine Republic',
   'One More Try',
   'Shake, Rattle and Roll Fourteen: The Invasion',
   'Sisterakas',
   'Sosy Problems',
   'The Strangers',
   'Thy Womb'],
  'Starring': ['Vic Sotto, Bong Revilla Jr. and Judy Ann Santos',
   'E.R. Ejercito, Cristine Reyes',
   'Angel Locsin, Dingdong Dantes, Angelica Panganiban, and Zanjoe Marudo',
   'Vhong Navarro, Lovi Poe, Dennis Trillo, Paulo Avelino, Mart Escudero, Janice de Belen, and Herbert Bautista',
   'Kris Aquino, Ai-Ai de las Alas, Vice Ganda, Kathryn Bernardo, Daniel Padilla, and Xyriel Manabat',
   'Heart Evangelista, Rhian Ramos, Solenn Heussaff, Bianca King, Benjamin Alves, Mikael Daez and Aljur Abrenica',
   'Julia Montes, Enrique Gil, JM De Guzman and Enchong Dee',
   'Nora Aunor, Bembol Roco, Lovi Poe, and Mercedes Cabral'],
  'Studio': ['Octoarts Films,  M-Zet Productions,  Imus Productions,  APT Entertainment, and  GMA Films',
   'Scenema Concept International,  CMB Films, and  VIVA Films',
   'Star Cinema',
   'Regal Films',
   'Star Cinema and  Viva Films',
   'GMA Films',
   'Quantum Films,  MJM Productions',
   'Centerstage Productions'],
  'Director': ['Tony Y. Reyes',
   'Mark Meily',
   'Ruel S. Bayani',
   'Chito S. Roño',
   'Wenn V. Deramas',
   'Andoy Ranay',
   'Lawrence Fajardo',
   'Brillante Mendoza'],
  'Genre': ['Fantasy, comedy, action',
   'Action, historical drama, thriller',
   'Drama',
   'Horror, Science Fiction',
   'Comedy',
   'Comedy',
   'Horror',
   'Drama']},
 2013: {'Year': [2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013],
  'Title': ['10,000 Hours',
   'Boy Golden',
   'Girl, Boy, Bakla, Tomboy',
   'Kaleidoscope World',
   'Kimmy Dora: Ang Kiyemeng Prequel',
   'My Little Bossings',
   'Pagpag: Siyam na Buhay',
   'Pedro Calungsod: Batang Martir'],
  'Starring': ['Robin Padilla, Alden Richards, Bela Padilla, Mylene Dizon, Carla Humphries, Pen Medina, Joem Bascon, Michael de Mesa',
   'Jeorge Estregan, KC Concepcion',
   'Vice Ganda, Maricel Soriano, Joey Marquez, Ruffa Gutierrez, Cristine Reyes, JC de Vera, Ejay Falcon',
   'Sef Cadayona, Yassi Pressman',
   'Eugene Domingo, Sam Milby',
   'Vic Sotto, Kris Aquino, Ryzza Mae Dizon, James "Bimby" Aquino-Yap',
   'Kathryn Bernardo, Daniel Padilla, Paulo Avelino, Shaina Magdayao',
   'Rocco Nacino, Christian Vasquez, Jestoni Alarcon, Robert Correa, Ryan Eigenmann, Victor Basa, Johan Santos'],
  'Studio': ['Philippine Film Studios, N2 Productions & Viva Films',
   'Viva Films & Scenema Concept International',
   'Star Cinema & Viva Films',
   'iAct Productions',
   'Spring Films, MJM Productions & Quantum Films',
   'OctoArts Films, M-Zet Productions, APT Entertainment & K Productions',
   'Star Cinema & Regal Entertainment',
   'HPI Synergy Group & Wings Entertainment'],
  'Director': ['Joyce Bernal',
   'Chito S. Roño',
   'Wenn Deramas',
   'Eliza Cornejo',
   'Chris Martinez',
   'Marlon Rivera',
   'Frasco Mortiz',
   'Francis O. Villacorta'],
  'Genre': ['Action, drama, thriller',
   'Action, drama',
   'Comedy',
   'Romance, dance film',
   'Comedy, action',
   'Comedy',
   'Horror, suspense, thriller',
   'Biography, epic']},
 2014: {'Year': [2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014],
  'Title': ['The Amazing Praybeyt Benjamin',
   'Bonifacio: Ang Unang Pangulo',
   'English Only, Please',
   'Feng Shui 2',
   'Kubot: The Aswang Chronicles 2',
   'Muslim Magnum .357: To Serve and Protect',
   'My Big Bossing',
   'Shake, Rattle & Roll XV'],
  'Starring': ['Vice Ganda, Richard Yap, Tom Rodriguez, Alex Gonzaga, Bimby Aquino-Yap',
   'Robin Padilla, Vina Morales, Daniel Padilla',
   'Jennylyn Mercado, Derek Ramsay',
   'Kris Aquino, Coco Martin',
   'Dingdong Dantes, Isabelle Daza, Joey Marquez, KC Montero, Lotlot de Leon',
   'ER Ejercito, Sam Pinto',
   'Ryzza Mae Dizon, Vic Sotto, Marian Rivera, Nikki Gil, Alonzo Muhlach',
   'JC de Vera, Erich Gonzales, Dennis Trillo, Carla Abellana, Lovi Poe, Matteo Guidicelli'],
  'Studio': ['Star Cinema & Viva Films',
   'Philippians Productions',
   'Quantum Films, Buchi Boy Films & Tuko Film Productions and MJM Productions',
   'Star Cinema and K Productions',
   'Reality Entertainment, GMA Films and AgostoDos Pictures',
   'Scenema Concept International and Viva Films',
   'OctoArts Films, M-Zet Productions and APT Entertainment',
   'Regal Entertainment'],
  'Director': ['Wenn Deramas',
   'Enzo Williams',
   'Dan Villegas',
   'Chito Roño',
   'Erik Matti',
   'Francis "Jun" Posadas',
   'Joyce Bernal, Marlon Rivera & Tony Reyes',
   'Dondon Santos, Jerrold Tarog and Perci Intalan'],
  'Genre': ['Comedy, action',
   'Historical drama, action',
   'Romantic comedy',
   'Horror',
   'Comedy, horror, action',
   'Action',
   'Comedy, Fantasy',
   'Horror anthology']},
 2015: {'Year': [2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015],
  'Title': ['All You Need Is Pag-ibig',
   'Beauty and the Bestie',
   'Buy Now, Die Later',
   'Haunted Mansion',
   'Honor Thy Father',
   'My Bebe Love: #KiligPaMore',
   'Nilalang',
   '#Walang Forever'],
  'Starring': ['Kris Aquino, Derek Ramsay, Kim Chiu, Xian Lim, Jodi Sta. Maria, Ian Veneracion,   Ronaldo Valdez,  Pokwang, Nova Villa, Bimby Aquino-Yap, Julia Concio, Talia Concio',
   'Vice Ganda, Coco Martin, James Reid, Nadine Lustre',
   'Vhong Navarro, John Lapus, Alex Gonzaga, TJ Trinidad, Rayver Cruz, Lotlot de Leon, Janine Gutierrez',
   'Janella Salvador, Marlo Mortel, Jerome Ponce',
   'John Lloyd Cruz, Meryll Soriano, Tirso Cruz III',
   'Vic Sotto, Ai-Ai delas Alas, Alden Richards, Maine Mendoza',
   'Cesar Montano, Maria Ozawa',
   'Jennylyn Mercado, Jericho Rosales'],
  'Studio': ['Star Cinema',
   'Star Cinema, Viva Films',
   'Quantum Films, MJM Production, Tuko Film Productions, Buchi Boy Films',
   'Regal Entertainment',
   'Reality Entertainment',
   'OctoArts Films, M-Zet Productions, APT Entertainment, GMA Films, MEDA Productions',
   'Haunted Tower Pictures, WeLovePost, Parallax Studios, Viva Films',
   'Quantum Films, MJM Production, Tuko Film Productions, Buchi Boy Films'],
  'Director': ['Antoinette Jadaone',
   'Wenn V. Deramas',
   'Randolph Longjas',
   'Jun Lana',
   'Erik Matti',
   'Jose Javier Reyes',
   'Pedring Lopez',
   'Dan Villegas'],
  'Genre': ['Romantic comedy',
   'Comedy, Action',
   'Horror, Comedy',
   'Horror, Suspense, Thriller',
   'Action, Drama',
   'Romantic comedy',
   'Horror, Action, Thriller, Crime',
   'Romantic comedy']},
 2016: {'Year': [2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016],
  'Title': ['Ang Babae sa Septic Tank 2: #ForeverIsNotEnough',
   'Die Beautiful',
   'Kabisera',
   'Oro',
   'Saving Sally',
   'Seklusyon',
   'Sunday Beauty Queen',
   'Vince & Kath & James'],
  'Starring': ['Eugene Domingo, Jericho Rosales, Kean Cipriano, Cai Cortez, Khalil Ramos, Joel Torre',
   'Paolo Ballesteros, Christian Bables, Joel Torre, Gladys Reyes, Luis Alandy, Albie Casiño,',
   'Nora Aunor, Ricky Davao, JC De Vera, Jason Abalos, RJ Agustin, Victor Neri, Ronwaldo Martin',
   'Irma Adlawan, Joem Bascon, Mercedes Cabral',
   'Rhian Ramos, Enzo Marcos',
   'Rhed Bustamante, Neil Ryan Sese, Ronnie Alonte, Lou Veloso, Phoebe Walker, Dominic Roque, Elora Españo, John Vic De Guzman, JR Versales',
   'Rudelyn Acosta, Cherrie Mae Bretana, Mylyn Jacobo, Hazel Perdido, Leo Selomenio',
   'Julia Barretto, Joshua Garcia, Ronnie Alonte'],
  'Studio': ['Martinez Rivera Films, Quantum Films, Tuko Film Productions, Buchi Boy Films, MJM Production',
   'The IdeaFirst Company, Octobertrain Films',
   'Firestarters Productions, Silver Story Production',
   'Feliz Film Productions',
   'Rocketsheep Studios',
   'Reality Entertainment',
   'Voyage Studios, Tuko Film Productions, Buchi Boy Films',
   'Star Cinema'],
  'Director': ['Marlon Rivera',
   'Jun Lana',
   'Arturo San Agustin, Real Florido',
   'Alvin Yapan',
   'Avid Liongoren',
   'Erik Matti',
   'Baby Ruth Villarama',
   'Theodore Boborol'],
  'Genre': ['Comedy',
   'Comedy',
   'Drama',
   'Political thriller',
   'Live-action/animation, Romance, Sci-fi',
   'Horror',
   'Documentary drama',
   'Romantic comedy']},
 2017: {'Year': [2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017],
  'Title': ['All of You',
   'Gandarrapiddo: The Revenger Squad',
   'Meant to Beh',
   'Ang Panday',
   'Deadma Walking',
   'Haunted Forest',
   'Ang Larawan',
   'Siargao'],
  'Starring': ['Jennylyn Mercado, Derek Ramsay',
   'Vice Ganda, Daniel Padilla, Pia Wurtzbach',
   'Vic Sotto, Dawn Zulueta',
   'Coco Martin',
   'Joross Gamboa, Edgar Allan Guzman',
   'Jane Oineza, Jameson Blake, Maris Racal, Jon Lucas',
   'Joanna Ampil, Rachel Alejandro, Paulo Avelino',
   'Jericho Rosales, Erich Gonzales, Jasmine Curtis-Smith'],
  'Studio': ['Quantum Films, MJM Productions',
   'Star Cinema, Viva Films',
   'OctoArts Films, M-Zet Productions, APT Entertainment',
   'CCM Productions, Star Cinema, Viva Films',
   'T-Rex Entertainment',
   'Regal Entertainment',
   'Culturtain Musicat Productions',
   'Ten17P'],
  'Director': ['Dan Villegas',
   'Joyce Bernal',
   'Chris Martinez',
   'Rodel Nacianceno',
   'Julius Alfonso',
   'Ian Loreños',
   'Loy Arcenas',
   'Paul Soriano'],
  'Genre': ['Romance, Comedy',
   'Action, Comedy',
   'Family, Comedy',
   'Action, Fantasy',
   'Comedy, Drama, Musical',
   'Horror, Thriller',
   'Musical',
   'Romance, Drama']},
 2018: {'Year': [2018, 2018, 2018, 2018, 2018, 2018, 2018, 2018],
  'Title': ['Aurora',
   'Fantastica',
   'The Girl in the Orange Dress',
   'Jack Em Popoy: The Puliscredibles',
   'Mary, Marry Me',
   'One Great Love',
   'Otlum',
   "Rainbow's Sunset"],
  'Starring': ['Anne Curtis',
   'Vice Ganda, Richard Gutierrez, Dingdong Dantes',
   'Jericho Rosales and Jessy Mendiola',
   'Coco Martin, Maine Mendoza and Vic Sotto',
   'Toni Gonzaga, Alex Gonzaga and Sam Milby',
   'Dennis Trillo, Kim Chiu and JC de Vera',
   'Ricci Rivero and Jerome Ponce',
   'Eddie Garcia, Tony Mabesa, Gloria Romero and Sunshine Dizon'],
  'Studio': ['Viva Films, Aliud Entertainment',
   'Star Cinema, Viva Films',
   'Star Cinema, Quantum Films, MJM Production',
   'CCM Film Productions, APT Entertainment, M-Zet Productions',
   'TEN17P, TINCAN',
   'Regal Entertainment',
   'Horseshoe Productions',
   "Heaven's Best Entertainment"],
  'Director': ['Yam Laranas',
   'Barry Gonzales',
   'Jay Abello',
   'Mike Tuviera',
   'RC delos Reyes',
   'Eric Quizon',
   'Joven Tan',
   'Joel Lamangan'],
  'Genre': ['Horror, Thriller, Suspense',
   'Fantasy, Comedy',
   'Romance, Comedy',
   'Action, Thriller, Comedy',
   'Romantic comedy',
   'Romance, Drama',
   'Horror',
   'Family drama']},
 2019: {'Year': [2019, 2019, 2019, 2019, 2019, 2019, 2019, 2019],
  'Title': ['Miracle in Cell No. 7',
   'Mission Unstapabol: The Don Identity',
   'The Mall, The Merrier',
   'Sunod',
   '3pol Trobol: Huli Ka Balbon!',
   'Culion',
   'Mindanao',
   'Write About Love'],
  'Starring': ['Aga Muhlach, Bela Padilla, Xia Vigor',
   'Vic Sotto, Maine Mendoza',
   'Anne Curtis, Vice Ganda',
   'Carmina Villarroel, Mylene Dizon',
   'Jennylyn Mercado, Coco Martin, Ai-Ai delas Alas',
   'Iza Calzado, Jasmine Curtis-Smith, Meryll Soriano',
   'Judy Ann Santos, Allen Dizon',
   'Miles Ocampo, Rocco Nacino, Joem Bascon, Yeng Constantino'],
  'Studio': ['Viva Films',
   'M-ZET TV Productions, APT Entertainment',
   'Star Cinema, Viva Films',
   'Ten17P, Globe Studios',
   'CCM Productions, Quantum Films',
   'iOptions Ventures Corp.',
   'Centre Stage Productions, Solar Pictures',
   'TBA Studios'],
  'Director': ['Nuel Crisostomo Naval',
   'Linnet Zurbano',
   'Barry Gonzalez',
   'Carlo Ledesma',
   'Rodel Nacianceno',
   'Alvin Yapan',
   'Brillante Mendoza',
   'Crisanto B. Aquino'],
  'Genre': ['Drama, Comedy',
   'Comedy',
   'Family, Comedy, Musical',
   'Horror',
   'Action, Comedy, Romance',
   'Historical drama',
   'Drama, Animation',
   'Romance']},
 2020: {'Year': [2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020],
  'Title': ['Coming Home',
   'Fan Girl',
   'Isa Pang Bahaghari',
   'Magikland',
   'Mang Kepweng: Ang Lihim ng Bandanang Itim',
   'Pakboys Takusa',
   'Suarez: The Healing Priest',
   'Tagpuan',
   'The Boy Foretold by the Stars',
   'The Missing'],
  'Starring': ['Jinggoy Estrada, Sylvia Sanchez, Edgar Allan Guzman',
   'Paulo Avelino, Charlie Dizon',
   'Nora Aunor, Phillip Salvador, Michael de Mesa',
   'Miggs Cuaderno, Elijah Alejo, Jun Urbano',
   'Vhong Navarro, Barbie Imperial',
   'Janno Gibbs, Dennis Padilla, Jerald Napoles, Andrew E.',
   'John Arcilla, Alice Dixson',
   'Alfred Vargas, Iza Calzado, Shaina Magdayao',
   'Adrian Lindayag, Keann Johnson',
   'Joseph Marco, Miles Ocampo, Ritz Azul'],
  'Studio': ['Maverick Films, ALV Films',
   'Black Sheep Productions, Globe Studios, Project 8 Corner San Joaquin Projects, Epic Media, Crossword Productions',
   "Heaven's Best Entertainment",
   'Brightlight Leisure Productions, Gallaga Reyes Films',
   'Cineko Productions, Star Cinema',
   'Viva Films',
   'Saranggola Media Productions, Viva Films',
   'Alternative Vision Cinema',
   'Clever Minds Inc., The Dolly Collection, Brainstormers Lab',
   'Regal Entertainment'],
  'Director': ['Adolfo Alix Jr.',
   'Antoinette Jadaone',
   'Joel Lamangan',
   'Christian Acuna',
   'Topel Lee',
   'Al Tantay',
   'Joven Tan',
   'MacArthur Alejandre',
   'Dolly Dulu',
   'Easy Ferrer'],
  'Genre': ['Family drama',
   'Drama',
   'Family drama',
   'Fantasy, Adventure',
   'Comedy',
   'Comedy',
   'Biopic',
   'Romance',
   'Romantic comedy',
   'Horror']},
 2021: {'Year': [2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021],
  'Title': ['A Hard Day',
   'Big Night!',
   'Huling Ulan sa Tag-Araw',
   'Huwag Kang Lalabas',
   'Kun Maupay Man it Panahon',
   'Love at First Stream',
   'Nelia',
   'The Exorsis'],
  'Starring': ['Dingdong Dantes, John Arcilla',
   'Christian Bables',
   'Rita Daniela, Ken Chan',
   'Kim Chiu, Aiko Melendez, Beauty Gonzalez',
   'Charo Santos-Concio, Daniel Padilla, Rans Rifol',
   'Kaori Oinuma, Jeremiah Lisbo, Daniela Stranner, Anthony Jennings',
   'Winwyn Marquez, Raymond Bagatsing, Ali Forbes',
   'Toni Gonzaga, Alex Gonzaga'],
  'Studio': ['Viva Films and AgostoDos Pictures',
   'Cignal Entertainment, The IdeaFirst Company, Octobertrain Films, Quantum Films',
   "Heaven's Best Entertainment",
   'Obra Cinema, Cineko Productions',
   'Globe Studios, Black Sheep Productions, Dreamscape Entertainment',
   'ABS-CBN Film Productions',
   'A and Q Production Films',
   'TINCAN, Viva Films'],
  'Director': ['Lawrence Fajardo',
   'Jun Lana',
   'Louie Ignacio',
   'Adolfo Alix Jr.',
   'Carlo Francisco Manatad',
   'Cathy Garcia-Molina',
   'Lester Dimaranan',
   'Fifth Solomon'],
  'Genre': ['Action',
   'Black Comedy',
   'Romance, Comedy',
   'Horror, Anthology',
   'Drama',
   'Romantic, Comedy',
   'Suspense, Drama',
   'Horror, Comedy']},
 2022: {'Year': [2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022],
  'Title': ['Labyu with an Accent',
   'Nanahimik ang Gabi',
   'Partners in Crime',
   'My Teacher',
   'Deleter',
   'Family Matters',
   'Mamasapano: Now It Can Be Told',
   'My Father, Myself'],
  'Starring': ['Coco Martin, Jodi Sta. Maria',
   'Ian Veneracion, Heaven Peralejo, Mon Confiado',
   'Vice Ganda, Ivana Alawi',
   'Toni Gonzaga, Joey de Leon',
   'Nadine Lustre, Louise delos Reyes, McCoy de Leon, Jeffrey Hidalgo',
   'Noel Trinidad, Liza Lorena',
   'Edu Manzano, Aljur Abrenica, Paolo Gumabao',
   'Dimples Romana, Sean de Guzman, Jake Cuenca'],
  'Studio': ['ABS-CBN Film Productions and CCM Creatives',
   'Rein Entertainment Philippines',
   'ABS-CBN Film Productions and Viva Films',
   'TEN17P,  DepEd Entertainment and APT Entertainment',
   'Viva Films',
   'Cineko Productions, Inc.',
   'Borracho Film Production and Viva Films',
   '3:16 Media Network and Regal Entertainment'],
  'Director': ['Rodel Nacianceno',
   'Shugo Praico',
   'Cathy Garcia-Molina',
   'Paul Soriano',
   'Mikhail Red',
   'Nuel Naval',
   'Lester Dimaranan',
   'Joel Lamangan'],
  'Genre': ['Romance, Comedy',
   'Horror, Thriller, Action',
   'Action, Comedy',
   'Drama, Family',
   'Techno-horror, Psychological thriller',
   'Drama, Family',
   'Action, Drama',
   'Drama']},
 2023: {'Year': [2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023],
  'Title': ['Family of Two',
   'Kampon',
   'Penduko',
   'Rewind',
   'Becky & Badette',
   'Broken Hearts Trip',
   'Firefly',
   'GomBurZa',
   'Mallari',
   'When I Met You in Tokyo'],
  'Starring': ['Sharon Cuneta, Alden Richards',
   'Beauty Gonzalez, Derek Ramsay',
   'Matteo Guidicelli, Kylie Verzosa',
   'Marian Rivera, Dingdong Dantes',
   'Eugene Domingo, Pokwang',
   'Christian Bables, Jaclyn Jose',
   'Alessandra de Rossi, Euwenn Mikaell',
   'Dante Rivero, Cedrick Juan, Enchong Dee, Piolo Pascual',
   'Piolo Pascual, Janella Salvador',
   'Vilma Santos, Christopher de Leon'],
  'Studio': ['Cineko Productions Inc., Myriad Entertainment',
   'Quantum Films, Brightlight Productions, OctoArts Films',
   'Sari Sari Network Films, Viva Films, Studio Viva',
   'ABS-CBN Film Productions, AgostoDos Pictures and APT Entertainment',
   'The IdeaFirst Company, Octoberian Films',
   'BMC Films',
   'GMA Pictures, GMA Public Affairs',
   'Jesuit Communications, MQuest Ventures, CMB Film Services',
   'Mentorque Productions',
   'JG Productions Incorporated'],
  'Director': ['Nuel Naval',
   'King Palisoc',
   'Jason Paul Laxamana',
   'Mae Cruz-Alviar',
   'Jun Robles Lana',
   'Lemuel C. Lorca',
   'Zig Dulay',
   'Pepe Diokno',
   'Derick Cabrido',
   'Rado Peru, Rommel Penesa'],
  'Genre': ['Drama, Family',
   'Horror, Thriller',
   'Superhero',
   'Romance, Drama',
   'Comedy',
   'Comedy, Drama',
   'Fantasy, Drama',
   'Historical, Biopic',
   'Horror, Thriller',
   'Romance, Drama']}}

#### IF INFO NEEDS TO BE UPDATED FOR A NEW YEAR
change `start_year` and `end_year` to the years needed

In [4]:
# start_year = 2023
# last_year = 2023

# years_list = [num for num in range(start_year,last_year+1)]
# links_dict = {year : 'https://en.wikipedia.org/wiki/'+str(year)+'_Metro_Manila_Film_Festival' for year in years_list}
# new_data = {}

# for year in years_list:
#     link = links_dict[year]

#     # get html
#     response = requests.get(link)
#     soup = BeautifulSoup(response.text,'html.parser')
#     options_list = list(soup.find_all(['td','h2','h3','h4']))

#     # cut html (start)
#     tag = [element.name for element in options_list]
#     text = [element.get_text().strip() for element in options_list]

#     if 'Entries[edit]' in text:
#         remove_start = text.index('Entries[edit]')
#     elif 'Entries' in text:
#         remove_start = text.index('Entries')
        
#     tag = tag[remove_start+1:]
#     text = text[remove_start+1:]

#     remove_start = tag.index('td')
#     tag = tag[remove_start:]
#     text = text[remove_start:]

#     # cut html (end)
#     for item in text:
#         if '[edit]' in item:
#             remove_end = text.index(item)
#             break
#     tag = tag[: remove_end]
#     text = text[: remove_end]

#     for item in tag:
#         if item != 'td':
#             remove_end = tag.index(item)
#             break
#     tag = tag[: remove_end]
#     text = text[: remove_end]

#     # clean data
#     for index in range(len(text)):
#         item = text[index]
#         while '[' in item:
#             ref_text = item[item.index('[') : item.index(']')+1]
#             item = item.replace(ref_text, '')
#             del text[index]
#             text.insert(index, item)
#         while '\n' in item:
#             ref_text = '\n'
#             item = item.replace(ref_text, '; ')
#             del text[index]
#             text.insert(index, item)

#     for item in text:
#         if item == '':
#             text.remove(item)
            
#     # make dataframe dict
#     new_data.update({year:
#         {'Year' : [year]*(int(len(text)/5)),
#         'Title' : [text[index] for index in range(len(text)) if index % 5 == 0],
#         'Starring' : [text[index] for index in range(len(text)) if index % 5 == 1],
#         'Studio' : [text[index] for index in range(len(text)) if index % 5 == 2],
#         'Director' : [text[index] for index in range(len(text)) if index % 5 == 3],
#         'Genre' : [text[index] for index in range(len(text)) if index % 5 == 4]}
#     }) 

# new_data_keys = list(new_data.keys())
# new_data_values = list(new_data.values())

# for index in range(len(new_data_keys)):
#     key = new_data_keys[index]
#     value = new_data_values[index]
#     data.update({key:value})

## AWARDS DATA

In [5]:
# awards_data = {}
# last_year = 2023
# years_list = [num for num in range(1975,last_year+1)]
# links_dict = {year : 'https://en.wikipedia.org/wiki/'+str(year)+'_Metro_Manila_Film_Festival' for year in years_list}

# for year in years_list:
#     link = links_dict[year]
#     awards_data[year] = ""
    
#     response = requests.get(link)
#     soup2 = BeautifulSoup(response.text, 'html.parser')
#     awards = soup2.find_all("li", string=re.compile("Best")) + soup2.find_all("li", string=re.compile("Award")) + soup2.find_all("li", string=re.compile("Prize"))
#     winners = {}
    
#     if year <= 2008:
#         if soup2.find('span', {'id': 'Winners_and_nominees'}) is None:
#             awards_header = soup2.find('span', {'id': 'Awards'})
#         else:
#             awards_header = soup2.find('span', {'id': 'Winners_and_nominees'})
        
#         awards_h2 = awards_header.find_parent('h2') 
        
#         next_sibling = awards_h2.find_next_sibling()
#         while next_sibling and next_sibling.name != 'table':
#             next_sibling = next_sibling.find_next_sibling()
#             if next_sibling:
#                 awards_table = next_sibling
#     else:
#         if soup2.find('span', {'id': 'Winners_and_nominees'}) is None:
#             awards_header = soup2.find('span', {'id': 'Awards'})
#         else:
#             awards_header = soup2.find('span', {'id': 'Winners_and_nominees'})
        
#         awards_h2 = awards_header.find_parent('h2') 
        
#         next_sibling = awards_h2.find_next_sibling()
#         table_count = 0 
        
#         while next_sibling:
#             if next_sibling.name == 'table':
#                 table_count += 1
#                 if table_count == 2:
#                     awards_table = next_sibling
#                     break
#             next_sibling = next_sibling.find_next_sibling()
                
#     indiv_awards = awards_table.find_all("th")
#     nominees = awards_table.find_all("td")
    
#     for num in range(0, len(indiv_awards)-1):
#         award_name = indiv_awards[num].text.replace("\n","")
#         if nominees[num].find("b") is None:
#             winner = "None"
#         else:
#             winner = nominees[num].find("b").text.strip()
#             winner = winner.replace("\xa0", " ")

#         winners[award_name] = winner
#         list_awards = list(winners.keys())
    
#         for key in list_awards:
#             if key == "Best Picture" or key == "Best Film" or key == "Dangal ng Bagong Lipunan":
#                 if " – " in winner:
#                     winners[award_name] = winners.get(key)[:winners.get(key).find(" – ")]
#                 elif " - " in winner:
#                     winners[award_name] = winners.get(key)[:winners.get(key).find(" - ")]
#             else:
#                 if " – " in winner:
#                     count_delimiter = winner.count(" – ")
#                     winner_split = list(winner.split(" – "))
#                     if count_delimiter == 1:
#                         winner = {winner.split(" – ")[0]: winner.split(" – ")[1]}
#                     else:
#                         artist = ''
#                         for index in range(len(winner_split)-1):
#                             item = winner_split[index]
#                             artist += item
#                             artist += ' – '
#                         artist = artist[:-3]
#                         winner = {artist: winner.split(" – ")[-1]}
                            
#                 elif " - " in winner:
#                     count_delimiter = winner.count(" - ")
#                     winner_split = list(winner.split(" - "))
#                     if count_delimiter == 1:
#                         winner = {winner.split(" - ")[0]: winner.split(" - ")[1]}
#                     else:
#                         artist = ''
#                         for index in range(len(winner_split)-1):
#                             item = winner_split[index]
#                             artist += item
#                             artist += ' - '
#                         artist = artist[:-3]
#                         winner = {artist: winner.split(" - ")[-1]}
            
#         winners[award_name] = winner
    
#     awards_data[year] = winners

# awards_data

In [6]:
awards_data = {1975: {'Dangal ng Bagong Lipunan': 'Diligin Mo ng Hamog ang Uhaw Na Lupa – JE Productions',
  'Best Director': {'Augusto Buenaventura': 'Diligin Mo ng Hamog ang Uhaw Na Lupa'},
  'Best Actor': {'Joseph Estrada': 'Diligin Mo ng Hamog ang Uhaw Na Lupa'},
  'Best Actress': {'Charito Solis': 'Araw-Araw, Gabi-Gabi'},
  'Best Supporting Actor': {'Vic Silayan': 'Diligin Mo ng Hamog ang Uhaw Na Lupa'},
  'Best Supporting Actress': {'Nida Blanca': 'Batu-Bato sa Langit'},
  'Best Screenplay': {'Luciano Carlos': 'Batu-Bato sa Langit'},
  'Best Story': {'Ophelia San Juan': 'Kapitan Kulas'},
  'Best Cinematography': {'Nonong Rasca': 'Kapitan Kulas'},
  'Best Editing': {'Edgardo Vinarao': 'Diligin Mo ng Hamog ang Uhaw Na Lupa'},
  'Best Music': {'George Canseco': 'Batu-Bato sa Langit'}},
 1976: {'Best Film': 'Ganito Kami Noon, Paano Kayo Ngayon – Hemisphere',
  'Best Director': {'Eddie Romero': 'Ganito Kami Noon, Paano Kayo Ngayon'},
  'Best Actor': {'Christopher de Leon': 'Ganito Kami Noon, Paano Kayo Ngayon'},
  'Best Actress': {'Hilda Koronel': 'Insiang'},
  'Best Supporting Actor': {'Ruel Vernal': 'Insiang'},
  'Best Supporting Actress': {'Mona Lisa': 'Insiang'},
  'Best Screenplay': {'Eddie Romero & Roy C. Iglesias': 'Ganito Kami Noon, Paano Kayo Ngayon'},
  'Best Cinematography': {'Conrado Baltazar': 'Insiang'},
  'Best Editing': {'Edgardo Vinarao': "Minsa'y isang Gamu-gamo"},
  'Best Music': {'Lutgardo Labad': 'Ganito Kami Noon, Paano Kayo Ngayon'},
  'Best Story': {'Marina Feleo-Gonzales': "Minsa'y isang Gamu-gamo"}},
 1977: {'Best Film': 'Burlesk Queen – Ian Film Productions',
  'Best Director': {'Celso Ad Castillo': 'Burlesk Queen'},
  'Best Actor': {'Rolly Quizon': 'Burlesk Queen'},
  'Best Actress': {'Vilma Santos': 'Burlesk Queen'},
  'Best Supporting Actor': {'Joonee Gamboa': 'Burlesk Queen'},
  'Best Supporting Actress': {'Rosemarie Gil': 'Burlesk Queen'},
  'Best Screenplay': {'Mauro Gia Samonte [ceb]': 'Burlesk Queen'},
  'Best Cinematography': {'Romeo Vitug': 'Mga Bilanggong Birhen'}},
 1978: {'Best Film': 'Atsay – Ian Film Productions',
  'Best Director': {'Eddie Garcia': 'Atsay'},
  'Best Performer': {'Nora Aunor': 'Atsay'},
  'Best Editing': {'Jose Tarnate': 'Rubia Servios'},
  'Best Screenplay': {"Mario O'Hara": 'Rubia Servios'},
  'Best Cinematography': {'Romeo Vitug': 'Atsay'}},
 1979: {'Best Film': 'Kasal-kasalan, bahay-bahayan',
  'Best Director': {'Lino Brocka ': 'Ina ka ng Anak Mo'},
  'Best Actor': {'Raul Aragon': 'Ina ka ng Anak Mo'},
  'Best Actress': 'Nora Aunor -Ina ka ng Anak Mo',
  'Best Supporting Actor': {'Johnny Delgado': 'Ang Alamat ni Julian Makabayan'},
  'Best Supporting Actress': {'Rebecca Gonzales': ' Kasal-kasalan, bahay-bahayan'},
  'Best Sound Engineering': {'Gaudencio Barredo': 'Kadete'},
  'Best Cinematography': {'Ben Lobo': 'Ang lihim ng Guadalupe'},
  'Best Editing': {'Edgardo Vinarao': 'Kadete'},
  'Best Music': {'Restie Umali': 'Ang Sisiw ay Isang Agila'}},
 1980: {'Best Film': 'Taga sa Panahon',
  'Best Director': {'Marilou Diaz-Abaya': 'Brutal'},
  'Best Actor': {'Dindo Fernando': 'Langis at Tubig'},
  'Best Actress': {'Amy Austria': 'Brutal'},
  'Best Sound Engineering': {'Rudy Baldovino and Ben Pelayo': "Kung Ako'y Iiwan Mo"},
  'Best Cinematography': {'Rudy Lacap': "Kung Ako'y Iiwan Mo"},
  'Best Music': {'Ernani Cuenco': 'Ang Panday'},
  'Best Story': {'Danny L. Zialcita': 'Jose N. Carreon'},
  'Best Screenplay': {'Jose N. Carreon': 'Taga sa Panahon'}},
 1981: {'Best Film': 'Kisapmata - Bancom Audiovision',
  'Best Director': {'Mike De Leon': 'Kisapmata'},
  'Best Actor': {'Vic Silayan': 'Kisapmata'},
  'Best Actress': {'Vilma Santos': 'Karma'},
  'Best Supporting Actor': {'Jay Ilagan': 'Kisapmata'},
  'Best Supporting Actress': {'Charito Solis': 'Kisapmata'},
  'Best Sound Engineering': 'Kisapmata',
  'Best Cinematography': {'Ver Reyes': 'Ang Pagbabalik ng Panday'},
  'Best Child Performer': {'Bentot, Jr.': 'Ang Pagbabalik ng Panday'},
  'Best Story': {'Clodualdo del Mundo, Jr., Raquel Villavicencio, and Mike De Leon': 'Kisapmata'},
  'Best Screenplay': {'Clodualdo del Mundo, Jr., Raquel Villavicencio, and Mike De Leon': 'Kisapmata'},
  'Best Editing': {'Jess Navarro and Ben Pelayo': 'Kisapmata'}},
 1982: {'Best Film': 'Himala',
  'Best Director': {'Ishmael Bernal ': 'Himala'},
  'Best Actor': {'Christopher de Leon': 'Haplos'},
  'Best Actress': {'Nora Aunor': 'Himala'},
  'Best Supporting Actor': {'Spanky Manikan': 'Himala'},
  'Best Supporting Actress': {'Gigi Dueñas': 'Himala'},
  'Best Sound Engineering': {'Rolly Ruta': 'Himala'},
  'Best Cinematography': {'Sergio Lobo': 'Himala'},
  'Best Editing': {'Ike Jarlego, Jr. and Ben Pelayo': 'Himala'},
  'Best Music': {'Ernani Cuenco': 'Ang Panday: Ikatlong Yugto'},
  'Best Art Direction': {'Raquel Villavicencio': 'Himala'}},
 1983: {'Best Film': 'Karnal',
  'Best Director': {'Willie Milan': 'Bago Kumalat ang Kamandag'},
  'Best Actor': {'Anthony Alonzo': 'Bago Kumalat ang Kamandag'},
  'Best Actress': {'Coney Reyes': 'Bago Kumalat ang Kamandag'},
  'Best Supporting Actor': {'Dennis Roldan': 'Hot Property'},
  'Best Supporting Actress': {'Alicia Alonzo': 'Bago Kumalat ang Kamandag'},
  'Best Sound Engineering': {'Rudy Baldovino': 'Karnal'},
  'Best Cinematography': {'Manolo Abaya': 'Karnal'},
  'Best Editing': {'Augusto Salvador': 'Bago Kumalat ang Kamandag'},
  'Best Music': {'William Yusi': 'Hot Property'},
  'Best Art Direction': {'Fiel Zabat': 'Karnal'},
  'Best Screenplay': {'Willie Milan and Ronnie Paredes': 'Bago Kumalat ang Kamandag'}},
 1984: {'Best Film': 'Bulaklak sa City Jail',
  'Best Director': {"Mario O'Hara": 'Bulaklak sa City Jail'},
  'Best Actor': {'Herbert Bautista': 'Shake, Rattle & Roll'},
  'Best Actress': {'Nora Aunor': 'Bulaklak sa City Jail'},
  'Best Supporting Actor': {'Ed Villapol': 'Alapaap'},
  'Best Supporting Actress': {'Celia Rodriguez': 'Bulaklak sa City Jail'},
  'Best Story': {'Lualhati Bautista': 'Bulaklak sa City Jail'},
  'Best Cinematography': {'Joe Tutanes': 'Alapaap'},
  'Best Editing': {'Jess Navarro': 'Shake, Rattle & Roll'},
  'Best Music': {'Jaime Fabregas': 'Shake, Rattle & Roll'},
  'Best Art Direction': {'Joe Tutanes': 'Alapaap'},
  'Best Screenplay': {'Lualhati Bautista': 'Bulaklak sa City Jail'}},
 1985: {'Best Film': 'Paradise Inn - Amazaldy Productions',
  'Best Director': {'Lino Brocka': 'Ano ang Kulay ng Mukha ng Diyos?'},
  'Best Actor': {'Anthony Alonzo': 'Moises Padilla Story: The Missing Chapter'},
  'Best Actress': {'Vivian Velez': 'Paradise Inn'},
  'Best Supporting Actor': {'Tommy Abuel': 'Moises Padilla Story: The Missing Chapter'},
  'Best Supporting Actress': {'Cherie Gil': ' God... Save Me!'},
  'Best Story': {'Celso Ad.Castillo': 'Paradise Inn'},
  'Best Cinematography': {'Romeo Vitug': 'Paradise Inn'},
  'Best Editing': {'Edgardo Vinarao': 'Paradise Inn'},
  'Best Music': {'Willy Cruz': 'Ano ang kulay ng mukha ng Diyos?'},
  'Best Art Direction': {'Benjie de Guzman': ' Ano ang kulay ng mukha ng Diyos?'},
  'Best Screenplay': {'Tom Adrales': 'Moises Padilla Story: The Missing Chapter'},
  'Best Child Performer': {'Katrin Gonzales': 'Ano ang kulay ng mukha ng Diyos?'}},
 1986: {'Best Film': 'None',
  'Best Director': {"Mario O'Hara": 'Halimaw sa Banga'},
  'Best Actor': {"Mario O'Hara": 'Halimaw sa Banga'},
  'Best Actress': {'Liza Lorena': 'Halimaw sa Banga'},
  'Best Supporting Actor': {'Christopher Ad. Castillo': 'Bagets Gang'},
  'Best Supporting Actress': {'Maritess Gutierrez': 'Halimaw sa Banga'},
  'Best Art Direction': {'Frank G. Rivera': 'Halimaw sa Banga'},
  'Best Cinematography': {'Romeo Vitug': 'Payaso'},
  'Best Sound Engineering': {'Rodel Capule': 'Halimaw sa Banga'},
  'Best Music': {'Jaime Fabregas': 'Halimaw sa Banga'},
  'Best Child Performer': {'Ian de Leon': 'Halimaw sa Banga'}},
 1987: {'Best Film': 'Olongapo, The Great American Dream',
  'Best Director': {'Artemio Marquez': 'The Untold Story of Melanie Marquez'},
  'Best Actor': {'Anthony Alonzo': 'Anak Badjao'},
  'Best Actress': {'Melanie Marquez': 'The Untold Story of Melanie Marquez'},
  'Best Supporting Actor': {'Roderick Paulate': '1+1=12+1'},
  'Best Supporting Actress': {'Susan Africa': 'Olongapo, The Great American Dream'},
  'Best Art Direction': {'Apol Salonga': 'Wag mong Bubuhayin ng Bangkay'},
  'Best Cinematography': {'Apolinario Cuenco': 'Anak Badjao'},
  'Best Sound Engineering': {'Juanito Clemente': 'Olongapo, The Great American Dream'},
  'Best Music': {'Marita Manuel': 'Anak badjao'},
  'Best Child Performer': {'Cheche Sta. Ana': 'Action Is Not Missing (Crack Platoon)'},
  'Best Editing': {'Rogelio Salvador': ' The Untold Story of Melanie Marquez'},
  'Best Story': {'Melanie Marquez': 'The Untold Story of Melanie Marquez'}},
 1988: {'Best Film': 'Patrolman',
  'Best Director': {'Laurice Guillen': 'Magkano ang Iyong Dangal?'},
  'Best Actor': {'Baldo Marro': 'Patrolman'},
  'Best Actress': {'Amy Austria': 'Celestina Sanchez, Alyas Bubbles/ Enforcer: Ativan Gang'},
  'Best Supporting Actor': {'Dick Israel': 'Patrolman'},
  'Best Supporting Actress': {'Jacklyn Jose – Celestina Sanchez, Alyas Bubbles': 'Enforcer: Ativan Gang'},
  'Best Art Direction': {'Manny B. Morfe': 'Pik Pak Boom'},
  'Best Cinematography': {'Romeo Vitug': 'Magkano ang Iyong Dangal?'},
  'Best Sound Engineering': {'Ramon Reyes': 'Itanong Mo sa Buwan'},
  'Best Musical Score': {'Willy Cruz': 'Magkano ang Iyong Dangal?'},
  'Best Child Performer': {'R.R. Herrera': 'Agila ng Maynila'},
  'Best Editing': {'Ike Jarlego Jr.': 'Magkano ang Iyong Dangal?'},
  'Best Story': {'Carlo J. Caparas and Tony Mortel – Celestina Sanchez, Alyas Bubbles': 'Enforcer: Ativan Gang'}},
 1989: {'Best Film': 'Imortal - VIVA Films',
  'Best Director': {'Eddie Garcia': 'Imortal'},
  'Best Actor': {'Christopher de Leon': 'Imortal'},
  'Best Actress': {'Vilma Santos': 'Imortal'},
  'Best Supporting Actor': {'Cesar Montano': 'Ang Bukas ay Akin'},
  'Best Supporting Actress': {'Cherie Gil': 'Imortal'},
  'Best Art Direction': {'Elmer Manapul': 'Imortal'},
  'Best Cinematography': {'Romeo Vitug': 'Ang Bukas ay Akin'},
  'Best Sound Direction': {'Albert Rima': 'Ang Bukas ay Akin'},
  'Best Music': {'George Canseco': 'Imortal'},
  'Best Child Performer': {'Atong Redillas': ' Ang Mahiwagang Daigdig ni Elias Paniki'},
  'Best Editing': {'Ike Jarlego, Jr.': 'Imortal'},
  'Best Story': {'Orlando Nadres': 'Imortal'},
  'Best Screenplay': {'Orlando Nadres': 'Imortal'}},
 1990: {'Best Film': 'Andrea, Paano Ba ang Maging Isang Ina? - MRN Film International',
  'Best Director': {'Gil Portes': 'Andrea, Paano Ba ang Maging Isang Ina?'},
  'Best Actor': {'Dolphy': 'Espadang Patpat'},
  'Best Actress': {'Nora Aunor': 'Andrea, Paano Ba ang Maging Isang Ina?'},
  'Best Supporting Actor': {'Robert Arevalo': 'Ama, Bakit Mo Ako Pinabayaan?'},
  'Best Supporting Actress': {'Gina Alajar': 'Andrea, Paano Ba ang Maging Isang Ina?'},
  'Best Art Direction': {'Don Escudero': 'Shake, Rattle & Roll II'},
  'Best Cinematography': {'Pedro Manding': 'Ama Bakit Mo Ako Pinabayaan'},
  'Best Studio Sound Recording': {'Rolly Ruta': 'Andrea, Paano Ba ang Maging Isang Ina?'},
  'Best Field Sound Recording': 'Conrado Radoban- Ama Bakit Mo Ako Pinabayaan',
  'Best Child Performer': {'Guila Alvarez': 'Ama Bakit Mo Ako Pinabayaan'},
  'Best Editing': {'Boy Vinarao': 'Andrea, Paano Ba ang Maging Isang Ina?'},
  'Best Story': {'Ricky Lee and Gil Portes': 'Andrea, Paano Ba ang Maging Isang Ina?'},
  'Best Screenplay': {'Ricky Lee and Gil Portes': 'Andrea, Paano Ba ang Maging Isang Ina?'},
  'Best Original Song': {'Mon Faustino': 'Andrea, Paano Ba ang Maging Isang Ina?'},
  'Best Music': {'Mon Faustino': 'Andrea, Paano Ba ang Maging Isang Ina?'},
  'Best Visual Effects': {'Sammy Arranzamendez and Benny Batoccoy': 'Shake, Rattle & Roll II'},
  'Best Make-up': {'Denni Tan, Dominique Nazareth and Andrea Manahan': 'Shake, Rattle & Roll II'}},
 1991: {'Best Film': 'Ang Totoong Buhay ni Pacita M. - MRN Film International',
  'Best Director': {'Elwood Perez': 'Ang Totoong Buhay ni Pacita M.'},
  'Best Actor': {'Eric Quizon': 'Juan Tamad at Mister Shooli: Mongolian Barbecue'},
  'Best Actress': {'Nora Aunor': 'Ang Totoong Buhay ni Pacita M.'},
  'Best Supporting Actor': {'Leo Martinez': 'Juan Tamad at Mister Shooli: Mongolian Barbecue'},
  'Best Supporting Actress': {'Tetchie Agbayani': 'Okay Ka, Fairy Ko!'},
  'Best Art Direction': {'Edel Templonuevo': 'Juan Tamad at Mister Shooli: Mongolian Barbecue'},
  'Best Cinematography': {'Johnny Arajo': 'Juan Tamad at Mister Shooli: Mongolian Barbecue'},
  'Best Child Performer': {'Aiza Seguerra': 'Okay Ka, Fairy Ko!'},
  'Best Editing': {'George Jarlego': 'Ang Totoong Buhay ni Pacita M.'},
  'Best Story': {'Ricky Lee': 'Ang Totoong Buhay ni Pacita M.'},
  'Best Screenplay': {'Ricky Lee': 'Ang Totoong Buhay ni Pacita M.'},
  'Best Original Song': {'Lucio San Pedro': 'Ang Totoong Buhay ni Pacita M.'},
  'Best Music': {'Danny Tan': 'Ang Totoong Buhay ni Pacita M.'},
  'Best Visual Effects': {'Carlos Lacap': 'Darna'},
  'Best Make-up': {'Cecille Baun': 'Darna'},
  'Best Sound Recording': {'Gaudencio Barredo': 'Juan Tamad at Mister Shooli: Mongolian Barbecue'}},
 1992: {'Best Film': 'Andres Manambit: Angkan ng Matatapang – VIVA Films',
  'Best Director': {'Enrique "Ike" Jarlego Jr.': 'Andres Manambit: Angkan ng Matatapang'},
  'Best Actor': {'Aga Muhlach': 'Bakit Labis Kitang Mahal'},
  'Best Actress': {'Gina Alajar': 'Shake, Rattle & Roll IV'},
  'Best Supporting Actor': {'Ariel Rivera': 'Bakit Labis Kitang Mahal'},
  'Best Supporting Actress': {'Sylvia Sanchez': 'Takbo... Talon... Tili!!!'},
  'Best Production Design': {'Don Escudero': 'Shake, Rattle & Roll IV'},
  'Best Cinematography': {'Arnold Alvaro and Ramon Marcelino': 'Takbo... Talon... Tili!!!'},
  'Best Child Performer': {'I.C. Mendoza': 'Shake, Rattle & Roll IV'},
  'Best Editor': {'Ike Jarlego Jr.': 'Andres Manambit: Angkan ng Matatapang'},
  'Best Original Story': {'Humilde ‘Meek’ Roxas': 'Andres Manambit: Angkan ng Matatapang'},
  'Best Screenplay': {'Ricky Lee and Jose Bartolome': 'Takbo... Talon... Tili!!!'},
  'Best Theme Song': {'"Bakit Labis Kitang Mahal" by Alex Mallillin': 'Bakit Labis Kitang Mahal'},
  'Best Musical Score': {'Dionisio Buencamino': 'Bakit Labis Kitang Mahal'},
  'Best Visual Special Effects': {'Tony Marbella': 'Okay Ka, Fairy Ko! Part 2'},
  'Best Make-up Artist': {'Andrea Manahan': 'Shake, Rattle & Roll IV'},
  'Best Sound Recording': {'Rolly Ruta': 'Andres Manambit: Angkan ng Matatapang'},
  'Best Float': {'Okay Ka, Fairy Ko! Part 2': 'Regal Films and M-Zet Productions'}},
 1993: {'Best Film': 'Kung Mawawala Ka Pa - Reyna Films',
  'Best Director': {'José Javier Reyes': 'May Minamahal'},
  'Best Actor': {'Aga Muhlach': 'May Minamahal'},
  'Best Actress': {'Dawn Zulueta': 'Kung Mawawala Ka Pa'},
  'Best Supporting Actor': {'Ronaldo Valdez': 'May Minamahal'},
  'Best Supporting Actress': {'Amy Austria': 'Kung Mawawala Ka Pa'},
  'Best Art Direction': {'Benjie de Guzman': 'May Minamahal'},
  'Best Cinematography': {'Romy Vitug': 'Kung Mawawala Ka Pa'},
  'Best Child Performer': {'Sarah Jane Abad': 'Kung Mawawala Ka Pa'},
  'Best Editing': {'Augusto Salvador': 'Doring Dorobo: Hagupit ng Batas'},
  'Best Story': {'Jose Javier Reyes': 'May Minamahal'},
  'Best Screenplay': {'Jose Javier Reyes': 'May Minamahal'},
  'Best Original Song': 'Kung Mawawala Ka Pa',
  'Best Music': {'Ryan Cayabyab': ' Kung Mawawala Ka Pa'},
  'Best Visual Effects': {'Bobby Pineda': 'Doring Dorobo: Hagupit ng Batas'},
  'Best Make-up': {'Teresita Dominguez': 'Gaano Kita Kamahal'},
  'Best Sound Recording': {'Ramon Reyes': 'Kung Mawawala Ka Pa'},
  'Best Float': {'May Minamahal': 'Star Cinema'},
  'Gatpuno Antonio J. Villegas Cultural Awards': {'May Minamahal': 'Star Cinema'}},
 1994: {'Best Film': 'None',
  'Best Director': 'None',
  'Best Actor': {'Roi Vinzon': 'Lucas Abelardo'},
  'Best Actress': {'Kimberly Diaz': 'Kanto Boy 2: Anak ni Totoy Guapo'},
  'Best Supporting Actor': {'Dick Israel': 'Kanto Boy 2: Anak ni Totoy Guapo'},
  'Best Supporting Actress': {'Teresa Loyzaga': 'Lucas Abelardo'},
  'Best Art Direction': {'Ben Payumo': 'Ang Pagbabalik ni Pedro Penduko'},
  'Best Cinematography': {'Ben Lobo': 'Ang Pagbabalik ni Pedro Penduko'},
  'Best Child Performer': {'Tom Taus, Jr.': 'Shake, Rattle & Roll V'},
  'Best Editing': {'Renato de Leon': 'Lucas Abelardo'},
  'Best Story': {'Jose "Kaka" Balagtas': 'Lucas Abelardo'},
  'Best Screenplay': 'None',
  'Best Original Song': {'Rey Magtoto': 'Lucas Abelardo'},
  'Best Music': {'Jaime Fabregas': 'Kanto Boy 2: Anak ni Totoy Guapo'},
  'Best Visual Effects': {'Cinemagic': 'Ang Pagbabalik ni Pedro Penduko'},
  'Best Make-up': {'Rey Salamat': 'Ang Pagbabalik ni Pedro Penduko'},
  'Best Sound Recording': {'Rolly Ruta': 'Lucas Abelardo'},
  'Best Float': {'Ang Pagbabalik ni Pedro Penduko': 'VIVA Films'}},
 1995: {'Best Film': 'Muling Umawit ang Puso - VIVA Films',
  'Best Director': {'Joel Lamangan': 'Muling Umawit ang Puso'},
  'Best Actor': {'Richard Gomez': 'Dahas'},
  'Best Actress': {'Nora Aunor': 'Muling Umawit ang Puso'},
  'Best Supporting Actor': {'Albert Martinez': 'Muling Umawit ang Puso'},
  'Best Supporting Actress': {'Donna Cruz': 'Muling Umawit ang Puso'},
  'Best Production Design': {'Manny Morfe': 'Muling Umawit ang Puso'},
  'Best Cinematography': {'Charlie Peralta': 'Dahas'},
  'Best Child Performer': {'Charina Scott': 'Huwag Mong Isuko ang Laban'},
  'Best Film Editor': {'Jaime Davila': 'Dahas'},
  'Best Original Story': {'Ricky Lee': 'Muling Umawit ang Puso'},
  'Best Screenplay': {'Ricky Lee': 'Muling Umawit ang Puso'},
  'Best Original Song': {'Willy Cruz': 'Muling Umawit ang Puso'},
  'Best Music': {'Vehnee Saturno': 'Muling Umawit ang Puso'},
  'Best Visual Effects': 'None',
  'Best Make-up': 'None',
  'Best Sound Recording': 'None'},
 1996: {'Best Film': 'Magic Temple - Star Cinema',
  'Best Director': {'Peque Gallaga and Lore Reyes': 'Magic Temple'},
  'Best Actor': {'Jomari Yllana': 'Kahit Kailan: Love Forgives'},
  'Best Actress': {'Amy Austria': 'Trudis Liit'},
  'Best Supporting Actor': {'John Arcilla': 'Mulanay'},
  'Best Supporting Actress': {'Jean Garcia': 'Trudis Liit'},
  'Best Production Design': {'Rodell Cruz': 'Magic Temple'},
  'Best Cinematography': {'Joe Tutanes': 'Magic Temple'},
  'Best Child Performer': {'Agatha Tapan': 'Trudis Liit'},
  'Best Editing': {'Danilo Gloria': 'Magic Temple'},
  'Best Original Story': {'Peque Gallaga, Lore Reyes and Erik Matti': 'Magic Temple'},
  'Best Screenplay': {'Peque Gallaga and Lore Reyes and Erik Matti': 'Magic Temple'},
  'Best Original Theme Song': {'Archie Castillo': 'Magic Temple'},
  'Best Musical Score': {'Archie Castillo': 'Magic Temple'},
  'Best Visual Effects': {'Benny Batoctoy': 'Magic Temple'},
  'Best Make-up Artist': {'Siony Tolentino': 'Magic Temple'},
  'Best Sound Recording': {'Michael Idioma and Ronald de Asis': 'Magic Temple'},
  'Best Float': {'Magic Temple': 'Star Cinema'}},
 1997: {'Best Film': 'Nasaan ang Puso - MaQ Productions',
  'Best Director': {'Chito Roño': 'Nasaan ang Puso'},
  'Best Actor': {'Christopher de Leon': 'Nasaan ang Puso'},
  'Best Actress': {'Maricel Soriano': 'Nasaan ang Puso'},
  'Best Supporting Actor': {'Ronaldo Valdez': 'Nasaan ang Puso'},
  'Best Supporting Actress': {'Nida Blanca': 'Babae'},
  'Best Production Design': {'Bradley J. Mayer': 'Magic Kingdom'},
  'Best Cinematography': {'Richard Padernal': 'Magic Kingdom'},
  'Best Child Performer': 'None',
  'Best Editing': {'Jaime Davila': 'Nasaan ang Puso'},
  'Best Original Story': {'Ruel Bayani': 'Babae'},
  'Best Screenplay': {'Roy Iglesias': 'Nasaan ang Puso'},
  'Best Original Theme Song': 'Babae',
  'Best Musical Score': {'Jesse Lasaten': 'Nasaan ang Puso'},
  'Best Visual Effects': {'Roadrunner Network, Inc.': 'Magic Kingdom'},
  'Best Make-up Artist': {'Manuel Benito': 'Magic Kingdom'},
  'Best Sound Recording': {'Albert Idiona, Ronaldo Asis and Arnold Reodica': 'Nasaan ang Puso'},
  'Best Float': {'Magic Kingdom': 'VIVA Films and Neo Films'}},
 1998: {'Best Film': 'José Rizal - GMA Films',
  'Best Director': {'Marilou Diaz-Abaya': 'José Rizal'},
  'Best Actor': {'Cesar Montano': 'José Rizal'},
  'Best Actress': {'Alice Dixson': 'Sambahin ang Ngalan Mo'},
  'Best Supporting Actor': {'Jaime Fabregas': 'José Rizal'},
  'Best Supporting Actress': {'Gloria Diaz': 'José Rizal'},
  'Best Production Design': {'Leo Abaya': 'José Rizal'},
  'Best Cinematography': {'Rody Lacap': 'José Rizal'},
  'Best Child Performer': 'None',
  'Best Editing': {'Jess Navarro and Manet Dayrit': 'José Rizal'},
  'Best Original Story': {'Ricky Lee, Jun Lana and Peter Lim': 'José Rizal'},
  'Best Screenplay': {'Ricky Lee, Jun Lana and Peter Lim': 'José Rizal'},
  'Best Original Theme Song': {'Nonong Buencamino ("Awit ni Maria Clara")': 'José Rizal'},
  'Best Musical Score': {'Nonong Buencamino': 'José Rizal'},
  'Best Visual Effects': {'Optima Digital': 'José Rizal'},
  'Best Make-up Artist': {'Dennis Tan': 'José Rizal'},
  'Best Sound Recording': {'Mike Idioma': 'José Rizal'},
  'Best Float': {'José Rizal': 'GMA Films'}},
 1999: {'Best Film': 'Muro-Ami - GMA Films',
  'Best Director': {'Marilou Diaz-Abaya': 'Muro-Ami'},
  'Best Actor': {'Christopher de Leon': 'Bulaklak ng Maynila'},
  'Best Actress': {'Elizabeth Oropesa': 'Bulaklak ng Maynila'},
  'Best Supporting Actor': {'Pen Medina': 'Muro-Ami'},
  'Best Supporting Actress': {'Angelu de Leon': 'Bulaklak ng Maynila'},
  'Best Cinematography': {'Rody Lacap': 'Muro-Ami'},
  'Best Production Design': {'Leo Abaya': 'Muro-Ami'},
  'Best Child Performer': {'Rebecca Lusterio': 'Muro-Ami'},
  'Best Editing': {'Jess Navarro and Manet Dayrit': 'Muro-Ami'},
  'Best Original Story': {'Marilou Diaz-Abaya, Ricardo Lee and Jun Lana': 'Muro-Ami'},
  'Best Screenplay': {'Ricardo Lee and Jun Lana': 'Muro-Ami'},
  'Best Original Theme Song': {'Vehnee Saturno ("Anong Daling Sabihin")': 'Bulaklak ng Maynila'},
  'Best Musical Score': {'Nonong Buencamino': 'Muro-Ami'},
  'Best Visual Effects': {'Marc Ambat (Optima Digital)': 'Muro-Ami'},
  'Best Make-up Artist': 'None',
  'Best Sound Recording': {'Albert Michael Idioma': 'Muro-Ami'},
  'Best Float': {'Pepeng Agimat': 'Millennium Cinema'}},
 2000: {'Best Film': 'Tanging Yaman - Star Cinema',
  'Best Director': {'Laurice Guillen': 'Tanging Yaman'},
  'Best Actor': {'Johnny Delgado': 'Tanging Yaman'},
  'Best Actress': {'Gloria Romero': 'Tanging Yaman'},
  'Best Supporting Actor': {'Jeffrey Quizon': 'Markova: Comfort Gay'},
  'Best Supporting Actress': {'Cherie Gil': 'Sugatang Puso'},
  'Best Cinematography': {'Videlle Meily': 'Tanging Yaman'},
  'Best Production Design': {'Joey Luna': 'Deathrow'},
  'Best Child Performer': 'None',
  'Best Editing': {'Jess Navarro and Kelly Cruz': 'Deathrow'},
  'Best Original Story': {'Laurice Guillen': 'Tanging Yaman'},
  'Best Screenplay': {'Shaira Mella Salvador, Raymond Lee and Laurice Guillen': 'Tanging Yaman'},
  'Best Original Theme Song': 'None',
  'Best Musical Score': {'Nonong Buencamino': 'Tanging Yaman'},
  'Best Visual Effects': {'Roadrunner Network, Inc.': 'Spirit Warriors'},
  'Best Make-up Artist': {'Cecile Baun, Benny Batoctoy and Warren Munar': 'Spirit Warriors'},
  'Best Sound Recording': {'Albert Michael Idioma and Rudy Gonzales': 'Deathrow'},
  'Best Float': {'Spirit Warriors': 'MAQ Productions and Roadrunner Network, Inc.'}},
 2001: {'Best Film': "Yamashita: The Tiger's Treasure - MAQ Productions",
  'Best Director': {'Chito Roño': "Yamashita: The Tiger's Treasure"},
  'Best Actor': {'Cesar Montano': 'Bagong Buwan'},
  'Best Actress': {'Assunta de Rossi': 'Hubog'},
  'Best Supporting Actor': {'Ronnie Lazaro': 'Bagong Buwan'},
  'Best Supporting Actress': {'Alessandra de Rossi': 'Hubog'},
  'Best Cinematography': {'Neil Daza': "Yamashita: The Tiger's Treasure"},
  'Best Production Design': {'Fernan Santiago and Max Paglinawan': "Yamashita: The Tiger's Treasure"},
  'Best Child Performer': {'Jiro Manio': 'Bagong Buwan'},
  'Best Editing': {'Manet Dayrit': "Yamashita: The Tiger's Treasure"},
  'Best Original Story': {'Roselle Monteverde-Teo, Roy Iglesias and Chito Rono': "Yamashita: The Tiger's Treasure"},
  'Best Screenplay': {'Roy Iglesias and Chito Rono': "Yamashita: The Tiger's Treasure"},
  'Best Original Theme Song': {'Joey Ayala ("Walang Hanggang Paalam")': 'Bagong Buwan'},
  'Best Musical Score': {'Nonong Buencamino': 'Bagong Buwan'},
  'Best Visual Effects': {'Roadrunner Network, Inc.': "Yamashita: The Tiger's Treasure"},
  'Best Make-up Artist': {'Warren Munar': "Yamashita: The Tiger's Treasure"},
  'Best Sound Recording': {'Albert Michael Idioma': "Yamashita: The Tiger's Treasure"},
  'Best Float': {"Yamashita: The Tiger's Treasure": 'MAQ Productions'}},
 2002: {'Best Film': 'Mano Po - Regal Films',
  'Best Director': {'Joel Lamangan': 'Mano Po'},
  'Best Actor': {'Eddie Garcia': 'Mano Po'},
  'Best Actress': {'Ara Mina': 'Mano Po'},
  'Best Supporting Actor': {'Piolo Pascual': "Dekada '70"},
  'Best Supporting Actress': {'Kris Aquino': 'Mano Po'},
  'Best Cinematography': {'Leslie Garchitorena': 'Mano Po'},
  'Best Production Design': {'Tatus Aldana': 'Mano Po'},
  'Best Child Performer': {'John Wayne Sace': "Dekada '70"},
  'Best Editing': {'Tara Illenberger': 'Mano Po'},
  'Best Original Story': {'Lily Monteverde and Roy Iglesias': 'Mano Po'},
  'Best Screenplay': {'Roy Iglesias': 'Mano Po'},
  'Best Original Theme Song': {'("Nasaan Ka Man")': 'Home Alone Da Riber'},
  'Best Musical Score': {'Von de Guzman': 'Mano Po'},
  'Best Visual Effects': {'Roadrunner Network, Inc.': 'Spirit Warriors: The Shortcut'},
  'Best Make-up Artist': {'Warren Munar': 'Spirit Warriors: The Shortcut'},
  'Best Sound Recording': {'Nestor Mutya': 'Hula Mo... Huli Ko'},
  'Best Float': {'Ang Agimat: Anting-Anting ni Lolo': 'Imus Productions'}},
 2003: {'Best Film': 'Crying Ladies - Unitel Pictures',
  'Best Director': {'Mark Meily': 'Crying Ladies'},
  'Best Actor': {'Eric Quizon': 'Crying Ladies'},
  'Best Actress': {'Maricel Soriano': 'Filipinas'},
  'Best Supporting Actor': {'Victor Neri': 'Filipinas'},
  'Best Supporting Actress': {'Hilda Koronel ': 'Crying Ladies'},
  'Best Cinematography': {'J. A. Tadena': 'Mano Po II: My Home'},
  'Best Production Design': {'Rodell Cruz': 'Mano Po II: My Home'},
  'Best Child Performer': {'Julio Pacheco': 'Crying Ladies'},
  'Best Editing': {'Vito Cajili': 'Malikmata'},
  'Best Original Story': {'Lily Monteverde and Roy Iglesias': 'Mano Po II: My Home'},
  'Best Screenplay': {'Roy Iglesias': 'Filipinas'},
  'Best Original Theme Song': {'("Isang Pagtanaw")': 'Homecoming'},
  'Best Musical Score': {'Francis Guevarra and Ferdie Marquez': 'Malikmata'},
  'Best Visual Effects': {'Dodge Ledesma Roadrunner Network, Inc.': 'Malikmata'},
  'Best Make-up Artist': {'Florencio Pinero, Gretta Estoesta, and Oscar Buara': 'Fantastic Man'},
  'Best Sound Recording': {'Mike Idioma': 'Malikmata'},
  'Best Float': {'Mano Po II: My Home': 'Regal Films'},
  'Most Gender-Sensitive Film': 'Homecoming'},
 2004: {'Best Picture': 'Mano Po III: My Love - MAQ Productions',
  'Best Director': {'Cesar Montano': 'Panaghoy sa Suba'},
  'Best Actor': {'Christopher de Leon': 'Mano Po III: My Love'},
  'Best Actress': {'Vilma Santos': 'Mano Po III: My Love'},
  'Best Supporting Actor': {'Dennis Trillo': 'Aishite Imasu 1941: Mahal Kita'},
  'Best Supporting Actress': {'Rebecca Lusterio': 'Panaghoy sa Suba'},
  'Best Cinematography': {'Ely Cruz': 'Panaghoy sa Suba'},
  'Best Production Design': {'Rodell Cruz': 'Mano Po III: My Love'},
  'Best Child Performer': {'Ella Guevara ': 'Sigaw'},
  'Best Editing': {'Manet Dayrit': 'Sigaw'},
  'Best Original Story': {'Joel Lamangan, Roy Iglesias & Lily Monteverde': 'Mano Po III: My Love'},
  'Best Screenplay': {'Cris Vertido': 'Panaghoy sa Suba'},
  'Best Original Theme Song': {'"Pagbigyan ang Puso Ko" from Mano Po III: My Love': 'Karylle and Jerome John Hughes'},
  'Best Musical Score': {'Nonong Buencamino': 'Panaghoy sa Suba'},
  'Best Visual Effects': {'Fel Rodolfo': 'Lastikman: Unang Banat'},
  'Best Make-up Artist': {'Alex Vicencio': 'Enteng Kabisote: Okay ka, Fairy Ko: The Legend'},
  'Best Sound Recording': {'Albert Michael Idioma & Arnold Reodica': 'Sigaw'},
  'Best Float': {'Mano Po III: My Love': 'MAQ Productions'},
  'Most Gender-Sensitive Film': {'Aishite Imasu 1941: Mahal Kita': 'Regal Films'}},
 2005: {'Best Picture': 'Blue Moon - Regal Films',
  'Best Director': {'José Javier Reyes': 'Kutob'},
  'Best Actor': {'Marvin Agustin': 'Kutob'},
  'Best Actress': {'Zsa Zsa Padilla': 'Mano Po 4: Ako Legal Wife'},
  'Best Supporting Actor': {'Jose Manalo': 'Enteng Kabisote 2: Okay Ka Fairy Ko: The Legend Continues'},
  'Best Supporting Actress': {'Cherry Pie Picache': 'Mano Po 4: Ako Legal Wife'},
  'Best Cinematography': {'Lyle Sacris': 'Exodus: Tales from the Enchanted Kingdom'},
  'Best Production Design': {'Richard Somes': 'Exodus: Tales from the Enchanted Kingdom'},
  'Best Child Performer': {'Paul Salas ': 'Shake, Rattle & Roll 2k5'},
  'Best Editing': {'Vito Cajili ': 'Kutob'},
  'Best Original Story': {'Allan Tijamo': 'Blue Moon'},
  'Best Screenplay': {'Allan Tijamo': 'Blue Moon'},
  'Best Original Theme Song': {'Thor ("Kasalanan Nga Ba?")': 'Kutob'},
  'Best Musical Score': {'Jaime Fabregas': 'Kutob'},
  'Best Visual Effects': {'Ignite Media': 'Exodus: Tales from the Enchanted Kingdom'},
  'Best Make-up Artist': {'Baby Lucero': 'Exodus: Tales from the Enchanted Kingdom'},
  'Best Sound Recording': {'Ditoy Aguila': 'Exodus: Tales from the Enchanted Kingdom'},
  'Best Float': 'None',
  'Most Gender-Sensitive Film': {'Blue Moon': 'Regal Films'}},
 2006: {'Best Picture': 'Enteng Kabisote 3: Okay Ka, Fairy Ko: The Legend Goes On and On and On - Octoarts Films and M-Zet Productions',
  'Best Director': {'José Javier Reyes': 'Kasal, Kasali, Kasalo'},
  'Best Actor': {'Cesar Montano': 'Ligalig'},
  'Best Actress': {'Judy Ann Santos': 'Kasal, Kasali, Kasalo'},
  'Best Supporting Actor': {'Johnny Delgado': 'Ligalig'},
  'Best Supporting Actress': {'Gina Pareño': 'Kasal, Kasali, Kasalo'},
  'Best Cinematography': {'Charlie Peralta': 'Mano Po 5: Gua Ai Di'},
  'Best Production Design': {'Edgar Martin Littaua': 'Mano Po 5: Gua Ai Di'},
  'Best Child Performer': {'Nash Aguas ': 'Shake, Rattle and Roll 8'},
  'Best Editing': {'Jason Cahapay ': 'Ligalig'},
  'Best Original Story': {'José Javier Reyes': 'Kasal, Kasali, Kasalo'},
  'Best Screenplay': {'José Javier Reyes': 'Kasal, Kasali, Kasalo'},
  'Best Original Theme Song': {'Yeng Constantino ("Hawak Kamay")': 'Kasal, Kasali, Kasalo'},
  'Best Musical Score': {'Von De Guzman': 'Mano Po 5: Gua Ai Di'},
  'Best Visual Effects': {'Imaginary Friends': 'Tatlong Baraha'},
  'Best Make-up Artist': {'Baby Lucero': 'Tatlong Baraha'},
  'Best Sound Recording': {'Ditoy Aguila': 'Mano Po 5: Gua Ai Di'},
  'Best Float': {'Tatlong Baraha': 'Violett Films'},
  'Most Gender-Sensitive Film': {'Kasal, Kasali, Kasalo': 'Star Cinema'}},
 2007: {'Best Picture': 'Resiklo - Imus Productions and Media Asia Films',
  'Best Director': {'Cesar Apolinario': 'Banal'},
  'Best Actor': {'Jinggoy Estrada': 'Katas ng Saudi'},
  'Best Actress': {'Maricel Soriano': 'Bahay Kubo: A Pinoy Mano Po!'},
  'Best Supporting Actor': {'Roi Vinzon': 'Resiklo'},
  'Best Supporting Actress': {'Eugene Domingo': 'Bahay Kubo: A Pinoy Mano Po!'},
  'Best Cinematography': {'Jay Linao': 'Resiklo'},
  'Best Production Design': {'Rodel Cruz': 'Resiklo'},
  'Best Child Performer': {'Nash Aguas': 'Shake, Rattle and Roll 9'},
  'Best Editing': {'Jay Halili': 'Resiklo'},
  'Best Original Story': {'Mac Cruz and Cesar Apolinario': 'Banal'},
  'Best Screenplay': {'José Javier Reyes': 'Katas ng Saudi'},
  'Best Original Theme Song': {'Rusty Fernandez ("Wala Na Bang Pag-Asa?")': 'Anak ng Kumander'},
  'Best Musical Score': {'Von De Guzman': 'Bahay Kubo: A Pinoy Mano Po!'},
  'Best Visual Effects': {'Ignite Media': 'Resiklo'},
  'Best Make-up Artist': {'Rosalinda Lopez': 'Desperadas'},
  'Best Sound Recording': {'Ditoy Aguila': 'Resiklo'},
  'Best Float': {'Resiklo': 'Imus Productions and Media Asia Films'},
  'Most Gender-Sensitive Film': {'Desperadas ': 'Regal Films'}},
 2008: {'Best Picture': 'Baler - Viva Communications',
  'Best Director': {'Mark Meily': 'Baler'},
  'Best Actor': {'Christopher de Leon': 'Magkaibigan'},
  'Best Actress': {'Anne Curtis': 'Baler'},
  'Best Supporting Actor': {'Phillip Salvador': 'Baler'},
  'Best Supporting Actress': {'Manilyn Reynes': 'One Night Only'},
  'Best Cinematography': {'Lee Meily': 'Baler'},
  'Best Production Design': {'Aped Santos': 'Baler'},
  'Best Child Performer': {'Robert Villar': 'Shake, Rattle & Roll X'},
  'Best Editing': {'Danny Anonuevo': 'Baler'},
  'Best Original Story': {'Jose Javier Reyes': 'One Night Only'},
  'Best Screenplay': {'Roy Iglesias': 'Baler'},
  'Best Original Theme Song': {'Jessie Lasaten and Artemio Abad Jr. ("Lipad" - performed by Lea Salonga)': 'Dayo: Sa Mundo ng Elementalia'},
  'Best Musical Score': {'Jessie Lazatin': 'Dayo: Sa Mundo ng Elementalia'},
  'Best Visual Effects': {'Robert Quilao': 'Dayo: Sa Mundo ng Elementalia'},
  'Best Make-up Artist': {'Noli Villalobos': 'Desperadas 2'},
  'Best Sound Recording': {'Albert Idioma and Whannie Dellosa': 'Dayo: Sa Mundo ng Elementalia'},
  'Best Float': 'None',
  'Most Gender-Sensitive Film': {'Baler ': 'Viva Communications'}},
 2009: {'Best Picture': 'Ang Panday - GMA Films and Imus Productions',
  'Best Director': {'Joel Lamangan': "Mano Po 6: A Mother's Love"},
  'Best Actor': {'Bong Revilla': 'Ang Panday'},
  'Best Actress': {'Sharon Cuneta': "Mano Po 6: A Mother's Love"},
  'Best Supporting Actor': {'Phillip Salvador': 'Ang Panday'},
  'Best Supporting Actress': {'Heart Evangelista': "Mano Po 6: A Mother's Love"},
  'Best Cinematography': {'Lee Briones-Meily': 'I Love You, Goodbye'},
  'Best Production Design': {'Richard Somes': 'Ang Panday'},
  'Best Child Performer': {'Robert Villar ': 'Ang Panday'},
  'Best Editing': {'Manet Dayrit and Efren Jarlego ': 'I Love You, Goodbye'},
  'Best Original Story': {'Vanessa Valdez, Anna Karenina Ramos and Kriz Gazmen': 'I Love You, Goodbye'},
  'Best Screenplay': {'Roy Iglesias': "Mano Po 6: A Mother's Love"},
  'Best Original Theme Song': {'Ogie Alcasid ("Tanging Ikaw Lamang" - performed by Regine Velasquez)': 'Ang Panday'},
  'Best Musical Score': {'Von de Guzman': "Mano Po 6: A Mother's Love"},
  'Best Visual Effects': {'Jay Santiago': 'Ang Panday'},
  'Best Make-up Artist': {'Noel Flores': 'Shake, Rattle & Roll 11'},
  'Best Sound Recording': {'Mike Idioma': 'Ang Darling Kong Aswang'},
  'Best Float': 'None',
  'Most Gender-Sensitive Film': {"Mano Po 6: A Mother's Love ": 'Regal Entertainment'}},
 2010: {'Best Picture': "Ang Tanging Ina Mo (Last na 'To!) - Star Cinema",
  'Best Director': {'Wenn V. Deramas': "Ang Tanging Ina Mo (Last na 'To!)"},
  'Best Actor': {'Dolphy': 'Father Jejemon'},
  'Best Actress': {'Ai-Ai delas Alas': "Ang Tanging Ina Mo (Last na 'To!)"},
  'Best Supporting Actor': {'Dolphy': 'Rosario'},
  'Best Supporting Actress': {'Eugene Domingo': "Ang Tanging Ina Mo (Last na 'To!)"},
  'Best Cinematography': {'Carlo Mendoza': 'Rosario'},
  'Best Production Design': {'Joey Luna': 'Rosario'},
  'Best Child Performer': {'Xyriel Manabat ': "Ang Tanging Ina Mo (Last na 'To!)"},
  'Best Editing': {'John Wong': 'Rosario'},
  'Best Original Story': {'Mel del Rosario': "Ang Tanging Ina Mo (Last na 'To!)"},
  'Best Screenplay': {'Mel del Rosario': "Ang Tanging Ina Mo (Last na 'To!)"},
  'Best Original Theme Song': {'Ria Osorio and Gerard Salonga - ("Kaya Mo" - performed by Protein Shake ft. Ney and Kean Cipriano)': 'RPG Metanoia'},
  'Best Musical Score': {'Jessie Lazatin': "Ang Tanging Ina Mo (Last na 'To!)"},
  'Best Visual Effects': {'Rico Gutierrez': 'Si Agimat at si Enteng Kabisote'},
  'Best Make-up Artist': 'Si Agimat at si Enteng Kabisote',
  'Best Sound Recording': {'Ditoy Aguila': 'Super Inday and the Golden Bibe'},
  'Best Float': {'Rosario ': 'Cinemabuhay and Studio5'},
  'Most Gender-Sensitive Film': {"Ang Tanging Ina Mo (Last na 'To!) ": 'Star Cinema'},
  'Best Indie Film': {'Presa': 'Adolf Alix Jr.'}},
 2011: {'Best Picture': 'Manila Kingpin: The Asiong Salonga Story',
  'Best Director': {'Tikoy Aguiluz': 'Manila Kingpin: The Asiong Salonga Story'},
  'Best Actor': {'Dingdong Dantes': 'Segunda Mano'},
  'Best Actress': {'Maricel Soriano': 'Yesterday, Today, Tomorrow'},
  'Best Supporting Actor': {'John Regala': 'Manila Kingpin: The Asiong Salonga Story'},
  'Best Supporting Actress': {'Eugene Domingo': 'My House Husband: Ikaw Na!'},
  'Best Child Performer': {'Bugoy Carino': 'Shake, Rattle & Roll 13'},
  'Best Original Story': {'Chris Martinez and Marlon Rivera ("Rain Rain Go Away" episode)': 'Shake, Rattle & Roll 13'},
  'Best Screenplay': {'Roy Iglesias': 'Manila Kingpin: The Asiong Salonga Story'},
  'Best Production Design': {'Fritz Siloria, Mona Soriano, and Ronaldo Cadapan': 'Manila Kingpin: The Asiong Salonga Story'},
  'Best Editing': {'Jason Canapay and Ryan Orduna': 'Manila Kingpin: The Asiong Salonga Story'},
  'Best Musical Score': {'Jessie Lazaten': 'Manila Kingpin: The Asiong Salonga Story'},
  'Best Sound Recording': {'Mike Idioma': 'Manila Kingpin: The Asiong Salonga Story'},
  'Best Original Theme Song': {'("La Paloma" - performed by Ely Buendia)': 'Manila Kingpin: The Asiong Salonga Story'},
  'Best Visual Effects': {'Riot Incorporated': 'Ang Panday 2'},
  'Best Make-up': {'Florencia Penero, Niccolo Medina, Jayvee Flores': 'Enteng ng Ina Mo'},
  'Best Cinematography': {'Carlo Mendoza': 'Manila Kingpin: The Asiong Salonga Story'},
  'Best Float': 'Ang Panday 2',
  'Most Gender- Sensitivity Film': 'My House Husband: Ikaw Na!'},
 2012: {'Best Picture': 'One More Try - Star Cinema',
  'Best Director': {'Brillante Mendoza': 'Thy Womb'},
  'Best Actor': {'Dingdong Dantes': 'One More Try'},
  'Best Actress': {'Nora Aunor': 'Thy Womb'},
  'Best Supporting Actor': {'Cesar Montano': 'El Presidente'},
  'Best Supporting Actress': {'Wilma Doesnt': 'Sisterakas'},
  'Best Child Performer': {'Miguel Vergara': 'One More Try'},
  'Best Original Story': {'Henry Burgos': 'Thy Womb'},
  'Best Screenplay': {'Anna Karenina and Jay Gazmin': 'One More Try'},
  'Best Production Design': {'Dante Mendoza': 'Thy Womb'},
  'Best Editing': {'Vito Cahilig': 'One More Try'},
  'Best Musical Score': {'Jessie Lazatin': 'El Presidente'},
  'Best Sound Recording': {'Albert Michael Idioma': 'El Presidente'},
  'Best Original Theme Song': {'Apl.de.ap and Jamir Garcia': 'El Presidente'},
  'Best Visual Effects': {'Imaginary Friends Studios and Blackburst Inc.': 'Shake, Rattle and Roll Fourteen: The Invasion'},
  'Best Make-up': {'Warren Munar, Benny Batoctoy and Virginia Apolinario': 'El Presidente'},
  'Best Cinematography': {'Odyssey Flores': 'Thy Womb'},
  'Best Float': 'El Presidente',
  'Most Gender- Sensitivity Film': {'Thy Womb': 'Centerstage Productions'},
  'Gatpuno Antonio J. Villegas Cultural Awards': {'Thy Womb': 'Centerstage Productions'}},
 2013: {'Best Picture': '10,000 Hours – Philippine Film Studios, Inc. and Viva Films',
  'Best Director': {'Joyce E. Bernal': '10,000 Hours'},
  'Best Actor': {'Robin Padilla': '10,000 Hours as Gabriel Molino Alcaraz'},
  'Best Actress': {'Maricel Soriano': 'Girl, Boy, Bakla, Tomboy as Pia Jackstone'},
  'Best Supporting Actor': {'Pen Medina': '10,000 Hours as Sebastian Jago'},
  'Best Supporting Actress': {'Aiza Seguerra': 'My Little Bossings as Ice'},
  'Gatpuno Antonio J. Villegas Cultural Award': '10,000 Hours',
  'Fernando Poe Jr. Memorial Award for Excellence': '10,000 Hours',
  'Best Screenplay': {'10,000 Hours': 'Ryllah Epifania Berica and Keiko Aquino'},
  'Best Original Story': {'10,000 Hours': 'Neil Arce, Peter Serrano, Bela Padilla and Joyce Bernal'},
  'Gender Sensitivity Award': 'Girl, Boy, Bakla, Tomboy',
  'Best Cinematography': {'10,000 Hours': 'Marissa Floirendo and Gilberto Vistan'},
  'Best Editor': {'Marya Ignacio': '10,000 Hours'},
  'Best Production Design': {'10,000 Hours': 'Joey Luna'},
  'Best Visual Effects': {'10,000 Hours': 'Central Digital'},
  'Best Make-up Artist': {'Mountain Rock Productions and Leslie Lucero': 'Pagpag: Siyam na Buhay'},
  'Best Original Theme Song': {'"My Little Bossings" from My Little Bossings': 'Jan K. Ilacad'},
  'Best Musical Scorer': {'Teresa Barrozo': '10,000 Hours'},
  'Best Sound Engineer': {'Emmanuel Clemente': '10,000 Hours'},
  'Best Child Performer': {'Ryzza Mae Dizon': 'My Little Bossings'},
  'Youth Choice Award': 'Pagpag: Siyam na Buhay'},
 2014: {'Best Picture': 'Bonifacio: Ang Unang Pangulo – Philippians Productions',
  'Best Director': {'Dan Villegas': 'English Only, Please'},
  'Best Actor': {'Derek Ramsay': 'English Only, Please as Julian Parker'},
  'Best Actress': {'Jennylyn Mercado': 'English Only, Please as Tere Madlansacay'},
  'Best Supporting Actor': {'Joey Marquez': 'Kubot: The Aswang Chronicles 2 as Nestor'},
  'Best Supporting Actress': {'Lotlot de Leon': 'Kubot: The Aswang Chronicles 2 as Nieves'},
  'Best Child Performer': {'Ryzza Mae Dizon': 'My Big Bossing as Jessa, Angel and Victoria'},
  'Best Cinematography': {'Carlo Mendoza': 'Bonifacio: Ang Unang Pangulo'},
  'Best Screenplay': {'Antoinette Jadaone & Anjeli Pessumal': 'English Only, Please'},
  'Best Original Story': {'Antoinette Jadaone & Dan Villegas': 'English Only, Please'},
  'Best Production Design': {'Ericson Navarro': 'Kubot: The Aswang Chronicles 2'},
  'Best Editing': {'Marya Ignacio': 'English Only, Please'},
  'Best Visual Effects': {'Mothership': 'Kubot: The Aswang Chronicles 2'},
  'Best Make-up Artist': {'Juvan Bermil & Charles Albert Alabado': 'Kubot: The Aswang Chronicles 2'},
  'Best Original Theme Song': {'"Hindi Pa Tapos" from Bonifacio: Ang Unang Pangulo': 'Gloc-9'},
  'Best Musical Score': {'Von de Guzman': 'Bonifacio: Ang Unang Pangulo'},
  'Best Sound Engineering': {'Wild Sound': 'Bonifacio: Ang Unang Pangulo'},
  'Gatpuno Antonio J. Villegas Cultural Award': 'Bonifacio: Ang Unang Pangulo',
  'Best Float': 'Bonifacio: Ang Unang Pangulo',
  'Fernando Poe Jr. Memorial Award for Excellence': 'Bonifacio: Ang Unang Pangulo'},
 2015: {'Best Picture': '#Walang Forever – Quantum Films, MJM Production, Tuko Film Productions, Buchi Boy Films',
  'Best Director': {'Erik Matti': 'Honor Thy Father'},
  'Best Actor': {'Jericho Rosales': '#Walang Forever'},
  'Best Actress': {'Jennylyn Mercado': '#Walang Forever'},
  'Best Supporting Actor': {'Tirso Cruz III': 'Honor Thy Father'},
  'Best Supporting Actress': {'Maine Mendoza': 'My Bebe Love: #KiligPaMore'},
  'Best Child Performer': {'Krystal Brimner': 'Honor Thy Father'},
  'Best Cinematography': {'Pao Orendain': 'Nilalang'},
  'Best Screenplay': {'Paul Sta. Ana': '#Walang Forever'},
  'Best Original Story': {'Dan Villegas & Antoinette Jadaone': '#Walang Forever'},
  'Best Production Design': {'Angel Diesta': 'Buy Now, Die Later'},
  'Best Editing': {'Jason Cahapay': 'Nilalang'},
  'Best Visual Effects': {'Pedro Chuidian & Rommel Pambid': 'Nilalang'},
  'Best Make-up Artist': {'Ryan Panaligan & Erika Racela': 'Honor Thy Father'},
  'Best Original Theme Song': '"Tao" from Honor Thy Father',
  'Best Musical Score': {'Jessie Lasaten': 'Nilalang'},
  'Best Sound Engineering': {'Ditoy Aguila': 'Nilalang'},
  'Gatpuno Antonio J. Villegas Cultural Award': 'My Bebe Love: #KiligPaMore',
  'Best Float': 'Buy Now, Die Later'},
 2016: {'Best Picture': 'Sunday Beauty Queen – Voyage Studios, Tuko Film Productions, Buchi Boy Films',
  'Best Director': {'Erik Matti': 'Seklusyon'},
  'Best Actor': {'Paolo Ballesteros': 'Die Beautiful'},
  'Best Actress': {'Irma Adlawan': 'Oro'},
  'Best Supporting Actor': {'Christian Bables': 'Die Beautiful'},
  'Best Supporting Actress': {'Phoebe Walker': 'Seklusyon'},
  'Best Ensemble Cast': 'Oro',
  'Best Screenplay': {'Anton Santamaria': 'Seklusyon'},
  'Best Cinematography': {'Neil Bion': 'Seklusyon'},
  'Best Production Design': {'Ericson Navarro': 'Seklusyon'},
  'Best Editing': {'Chuck Gutierrez': 'Sunday Beauty Queen'},
  'Best Sound Design': {'Lamberto Casas, Jr. and Albert Michael Idioma': 'Seklusyon'},
  'Best Original Theme Song': {'"Dominus Miserere" from Seklusyon': 'Francis de Veyra'},
  'Best Musical Score': {'Pablo Pico': 'Saving Sally'},
  'Gatpuno Antonio J. Villegas Cultural Award': 'Sunday Beauty Queen',
  'Fernando Poe Jr. Memorial Award for Excellence': 'Oro',
  'Special Jury Prize': {'Rhed Bustamante': 'Seklusyon'},
  'Best Float': 'Die Beautiful',
  'My Most Favorite Film': 'Die Beautiful'},
 2017: {'Best Picture': 'Ang Larawan – Culturtain Musicat Productions',
  'Best Director': {'Paul Soriano ': 'Siargao'},
  'Best Actor': {'Derek Ramsay ': 'All of You'},
  'Best Actress': {'Joanna Ampil ': 'Ang Larawan'},
  'Best Supporting Actor': {'Edgar Allan Guzman ': 'Deadma Walking'},
  'Best Supporting Actress': {'Jasmine Curtis-Smith ': 'Siargao'},
  'Best Child Performer': {'Baeby Baste ': 'Meant to Beh'},
  'Best Screenplay': {'Carl Chavez, Dan Villegas and Mae Chua ': 'All of You'},
  'Best Cinematography': {'Odyssey Flores ': 'Siargao'},
  'Best Production Design': {'Gino Gonzales ': 'Ang Larawan'},
  'Best Editing': {'Mark Victor ': 'Siargao'},
  'Best Sound': {'Mark Locsin, Mikko Quizon, Jason Conanan and Kat Salinas ': 'Siargao'},
  'Best Original Theme Song': {'"Alon" from Siargao ': 'Hale'},
  'Best Musical Score': {'Ryan Cayabyab ': 'Ang Larawan'},
  'Best Visual Effects': 'Ang Panday',
  'Best Float': 'Deadma Walking',
  'Gatpuno Antonio J. Villegas Cultural Award': 'Ang Larawan',
  'Fernando Poe Jr. Memorial Award for Excellence': 'Ang Panday',
  "People's Choice Award": 'Gandarrapiddo: The Revenger Squad',
  'Special Jury Prize': {'Nick Joaquin (posthumous) ': 'Ang Larawan'}},
 2018: {'Best Picture': "Rainbow's Sunset – Heaven's Best Entertainment",
  'Best Director': {'Joel Lamangan': "Rainbow's Sunset"},
  'Best Actor': {'Dennis Trillo': 'One Great Love'},
  'Best Actress': {'Gloria Romero': "Rainbow's Sunset"},
  'Best Supporting Actor': {'Tony Mabesa': "Rainbow's Sunset"},
  'Best Supporting Actress': {'Aiko Melendez': "Rainbow's Sunset"},
  'Best Child Performer': {'Phoebe Villamor': 'Aurora'},
  'Best Screenplay': {'Eric Ramos': "Rainbow's Sunset"},
  'Best Cinematography': {'Yam Laranas': 'Aurora'},
  'Best Production Design': {'Jay Custodio': "Rainbow's Sunset"},
  'Best Editing': {'Tara Illenberger and Renewin Alano': 'Jack Em Popoy: The Puliscredibles'},
  'Best Sound': {'Albert Michael Idioma': 'Aurora'},
  'Best Original Theme Song': {'"Sa\'yo Na" from Rainbow\'s Sunset': 'Ice Seguerra'},
  'Best Musical Score': {'Miguel Mendoza': 'One Great Love'},
  'Best Visual Effects': {'Ernest Villanueva and Gem Garcia': 'Aurora'},
  'Best Float': 'Jack Em Popoy: The Puliscredibles',
  'Gatpuno Antonio J. Villegas Cultural Award': "Rainbow's Sunset",
  'Fernando Poe Jr. Memorial Award for Excellence': 'Jack Em Popoy: The Puliscredibles',
  'Special Jury Prize': {'Max Collins': "Rainbow's Sunset"}},
 2019: {'Best Picture': 'Mindanao – Centre Stage Productions',
  'Best Director': {'Brillante Mendoza': 'Mindanao'},
  'Best Actor': {'Allen Dizon': 'Mindanao'},
  'Best Actress': {'Judy Ann Santos': 'Mindanao'},
  'Best Supporting Actor': {'Joem Bascon': 'Write About Love'},
  'Best Supporting Actress': {'Yeng Constantino': 'Write About Love'},
  'Best Child Performer': {'Yuna Tangog': 'Mindanao'},
  'Best Screenplay': {'Crisanto B. Aquino': 'Write About Love'},
  'Best Cinematography': {'Mycko David': 'Sunod'},
  'Best Production Design': {'Ericson Navarro': 'Sunod'},
  'Best Editing': {'Vanessa De Leon': 'Write About Love'},
  'Best Sound': {'Hiroyuki Ishizaka': 'Mindanao'},
  'Best Original Theme Song': {'"Ikaw ang Akin" from Write About Love': 'Written by Crisanto B. Aquino, Performed by Yeng Constantino'},
  'Best Musical Score': {'Jerrold Tarog': 'Write About Love'},
  'Best Visual Effects': {'Team App': 'Mindanao'},
  'Best Float': 'Mindanao',
  'Gatpuno Antonio J. Villegas Cultural Award': 'Mindanao',
  'Fernando Poe Jr. Memorial Award for Excellence': 'Mindanao',
  'Special Jury Prize': {'Cast Ensemble': 'Culion'}},
 2020: {'Best Picture': 'Fan Girl – Black Sheep Productions, Globe Studios',
  'Best Director': {'Antoinette Jadaone': 'Fan Girl'},
  'Best Actor': {'Paulo Avelino': 'Fan Girl'},
  'Best Actress': {'Charlie Dizon': 'Fan Girl'},
  'Best Supporting Actor': {'Michael de Mesa': 'Isa Pang Bahaghari'},
  'Best Supporting Actress': {'Shaina Magdayao': 'Tagpuan'},
  'Best Child Performer': {'Seiyo Masunaga': 'The Missing'},
  'Best Screenplay': {'Antoinette Jadaone': 'Fan Girl'},
  'Best Cinematography': {'Neil Daza': 'Fan Girl'},
  'Best Production Design': {'Ericson Navarro': 'Magikland'},
  'Best Editing': {'Benjamin Tolentino': 'Fan Girl'},
  'Best Sound': {'Vincent Villa': 'Fan Girl'},
  'Best Original Theme Song': {'"Ulan" from The Boy Foretold by the Stars': 'Written by Jhay Cura/Pau Protacio'},
  'Best Musical Score': {'Emerzon Texon': 'Magikland'},
  'Best Visual Effects': {'Richard Francia, Ryan Grimarez': 'Magikland'},
  'Best Virtual Float': 'Magikland',
  'Gatpuno Antonio J. Villegas Cultural Award': 'Suarez: The Healing Priest',
  'Fernando Poe Jr. Memorial Award for Excellence': 'Magikland',
  'Special Jury Prize': 'Peque Gallaga'},
 2021: {'Best Picture': 'Big Night! – Cignal Entertainment, The IdeaFirst Company',
  'Best Director': {'Jun Lana': 'Big Night!'},
  'Best Actor': {'Christian Bables': 'Big Night!'},
  'Best Actress': {'Charo Santos-Concio': 'Kun Maupay Man it Panahon'},
  'Best Supporting Actor': {'John Arcilla': 'Big Night!'},
  'Best Supporting Actress': {'Rans Rifol': 'Kun Maupay Man it Panahon'},
  'Best Screenplay': {'Jun Lana': 'Big Night!'},
  'Best Cinematography': {'Carlo Mendoza': 'Big Night!'},
  'Best Production Design': {'Juan Manuel Alcazaren': 'Kun Maupay Man it Panahon'},
  'Best Editing': {'Lawrence Fajardo': 'A Hard Day'},
  'Best Sound': {'Albert Michael Idioma': 'A Hard Day'},
  'Best Original Theme Song': {'"Umulan Man O Umaraw" from Huling Ulan sa Tag-Araw': 'Written by Louie Ignacio'},
  'Best Musical Score': {'Teresa Barrozo': 'Big Night!'},
  'Best Visual Effects': {'Ogie Tiglao': 'Kun Maupay Man it Panahon'},
  'Best Float': 'Huwag Kang Lalabas',
  'Gatpuno Antonio J. Villegas Cultural Award': 'Kun Maupay Man it Panahon',
  'Fernando Poe Jr. Memorial Award for Excellence': 'A Hard Day',
  'Special Jury Prize': {'Daniel Padilla': 'Kun Maupay Man it Panahon'}},
 2022: {'Best Picture': 'Deleter – Pelikula Red, Top Story',
  'Best Director': {'Mikhail Red': 'Deleter'},
  'Best Actor': {'Ian Veneracion': 'Nanahimik ang Gabi'},
  'Best Actress': {'Nadine Lustre': 'Deleter'},
  'Best Supporting Actor': {'Mon Confiado': 'Nanahimik ang Gabi'},
  'Best Supporting Actress': {'Dimples Romana': 'My Father, Myself'},
  'Best Screenplay': {'Eric Ramos': 'Mamasapano: Now It Can Be Told'},
  'Best Cinematography': {'Ian Guevarra': 'Deleter'},
  'Best Production Design': {'Mariel Hizon': 'Nanahimik ang Gabi'},
  'Best Editing': {'Nikolas Red': 'Deleter'},
  'Best Sound': 'Deleter',
  'Best Original Theme Song': {'"Aking Mahal" from Mamasapano: Now It Can Be Told': 'by Ferdinand Topacio and Cristy Fermin'},
  'Best Musical Score': {'Greg Rodriguez III': 'Nanahimik ang Gabi'},
  'Best Visual Effects': {'Gaspar Mangalin': 'Deleter'},
  'Best Child Performer': {'Shawn Niño Gabriel': 'My Father, Myself'},
  'Gatpuno Antonio J. Villegas Cultural Award': 'Family Matters',
  'Fernando Poe Jr. Memorial Award for Excellence': 'Mamasapano: Now It Can Be Told',
  'Best Float': 'My Father, Myself',
  'Special Jury Prize': 'Mamasapano: Now It Can Be Told'},
 2023: {'Best Picture': 'Firefly – GMA Pictures',
  'Best Director': {'Pepe Diokno': 'GomBurZa'},
  'Best Actor': {'Cedrick Juan': 'GomBurZa'},
  'Best Actress': {'Vilma Santos': 'When I Met You in Tokyo'},
  'Best Supporting Actor': {'JC Santos': 'Mallari'},
  'Best Supporting Actress': {'Miles Ocampo': 'Family of Two'},
  'Best Screenplay': {'Angeli Atienza': 'Firefly'},
  'Best Cinematography': {'Carlo Mendoza': 'GomBurZa'},
  'Best Production Design': {'Ericson Navarro': 'GomBurZa'},
  'Best Editing': {'Benjamin Tolentino': 'Kampon'},
  'Best Sound': {'Melvin Rivera and Louie Boy Bauson': 'GomBurZa'},
  'Best Original Theme Song': {'"Finggah Lickin" from Becky & Badette': 'by Eugene Domingo'},
  'Best Musical Score': {'Von de Guzman': 'Mallari'},
  'Best Visual Effects': {'Gaspar Mangalin': 'Mallari'},
  'Best Child Performer': {'Euwenn Mikaell': 'Firefly'},
  'Gatpuno Antonio J. Villegas Cultural Award': 'GomBurZa',
  'Fernando Poe Jr. Memorial Award for Excellence': 'When I Met You in Tokyo',
  'Best Float': 'When I Met You in Tokyo',
  'Gender Sensitivity Award': 'Becky & Badette'}}

#### IF INFO NEEDS TO BE UPDATED FOR A NEW YEAR
change `start_year` and `end_year` to the years needed

In [7]:
# new_awards_data = {}
# start_year = 2023
# last_year = 2023
# years_list = [num for num in range(start_year,last_year+1)]
# links_dict = {year : 'https://en.wikipedia.org/wiki/'+str(year)+'_Metro_Manila_Film_Festival' for year in years_list}

# for year in years_list:
#     link = links_dict[year]
#     new_awards_data[year] = ""
    
#     response = requests.get(link)
#     soup2 = BeautifulSoup(response.text, 'html.parser')
#     awards = soup2.find_all("li", string=re.compile("Best")) + soup2.find_all("li", string=re.compile("Award")) + soup2.find_all("li", string=re.compile("Prize"))
#     winners = {}
    
#     if year <= 2008:
#         if soup2.find('span', {'id': 'Winners_and_nominees'}) is None:
#             awards_header = soup2.find('span', {'id': 'Awards'})
#         else:
#             awards_header = soup2.find('span', {'id': 'Winners_and_nominees'})
        
#         awards_h2 = awards_header.find_parent('h2') 
        
#         next_sibling = awards_h2.find_next_sibling()
#         while next_sibling and next_sibling.name != 'table':
#             next_sibling = next_sibling.find_next_sibling()
#             if next_sibling:
#                 awards_table = next_sibling
#     else:
#         if soup2.find('span', {'id': 'Winners_and_nominees'}) is None:
#             awards_header = soup2.find('span', {'id': 'Awards'})
#         else:
#             awards_header = soup2.find('span', {'id': 'Winners_and_nominees'})
        
#         awards_h2 = awards_header.find_parent('h2') 
        
#         next_sibling = awards_h2.find_next_sibling()
#         table_count = 0 
        
#         while next_sibling:
#             if next_sibling.name == 'table':
#                 table_count += 1
#                 if table_count == 2:
#                     awards_table = next_sibling
#                     break
#             next_sibling = next_sibling.find_next_sibling()
                
#     indiv_awards = awards_table.find_all("th")
#     nominees = awards_table.find_all("td")
    
#     for num in range(0, len(indiv_awards)-1):
#         award_name = indiv_awards[num].text.replace("\n","")
#         if nominees[num].find("b") is None:
#             winner = "None"
#         else:
#             winner = nominees[num].find("b").text.strip()
#             winner = winner.replace("\xa0", " ")

#         winners[award_name] = winner
#         list_awards = list(winners.keys())
    
#         for key in list_awards:
#             if key == "Best Picture" or key == "Best Film" or key == "Dangal ng Bagong Lipunan":
#                 if " – " in winner:
#                     winners[award_name] = winners.get(key)[:winners.get(key).find(" – ")]
#                 elif " - " in winner:
#                     winners[award_name] = winners.get(key)[:winners.get(key).find(" - ")]
#             else:
#                 if " – " in winner:
#                     count_delimiter = winner.count(" – ")
#                     winner_split = list(winner.split(" – "))
#                     if count_delimiter == 1:
#                         winner = {winner.split(" – ")[0]: winner.split(" – ")[1]}
#                     else:
#                         artist = ''
#                         for index in range(len(winner_split)-1):
#                             item = winner_split[index]
#                             artist += item
#                             artist += ' – '
#                         artist = artist[:-3]
#                         winner = {artist: winner.split(" – ")[-1]}
                            
#                 elif " - " in winner:
#                     count_delimiter = winner.count(" - ")
#                     winner_split = list(winner.split(" - "))
#                     if count_delimiter == 1:
#                         winner = {winner.split(" - ")[0]: winner.split(" - ")[1]}
#                     else:
#                         artist = ''
#                         for index in range(len(winner_split)-1):
#                             item = winner_split[index]
#                             artist += item
#                             artist += ' - '
#                         artist = artist[:-3]
#                         winner = {artist: winner.split(" - ")[-1]}
            
#         winners[award_name] = winner
    
#     new_awards_data[year] = winners

# new_awards_data_keys = list(new_awards_data.keys())
# new_awards_data_values = list(new_awards_data.values())

# for index in range(len(new_awards_data_keys)):
#     key = new_awards_data_keys[index]
#     value = new_awards_data_values[index]
#     awards_data.update({key:value})

## ALL MOVIES DATAFRAME

In [8]:
last_year = 2023
years_list = [num for num in range(1975,last_year+1)]

all_movies_list = [pd.DataFrame(data[year]) for year in years_list]
all_movies = pd.concat(all_movies_list, ignore_index=True)

### get links attempt 1

In [9]:
# driver = webdriver.Chrome()
# driver.get(r'https://www.imdb.com/')

# movies_list = list(all_movies['Title'].to_dict().values())
# all_years_list = list(all_movies['Year'].to_dict().values())

# links_list = []

# for index in range(len(movies_list)):
#     movie = movies_list[index]
#     year = all_years_list[index]
#     search = f'{movie} {year}'
    
#     driver.find_element(By.NAME, 'q').send_keys(search)    
#     driver.find_element(By.ID, 'suggestion-search-button').click()

#     try:
#         driver.find_element(By.CSS_SELECTOR, '.ipc-metadata-list-summary-item__t').click()
#         current_link = driver.current_url
#         print(current_link)
#         links_list.append(current_link)
    
#     except NoSuchElementException:
#         print('No Link')
#         links_list.append('No Link')

# driver.quit()

In [10]:
links_list = ['https://www.imdb.com/title/tt1147439/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0358943/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0122991/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0402134/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0370836/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0434119/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1178155/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1174736/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0487554/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0401234/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0159456/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0077740/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0487619/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0487669/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0188872/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0125408/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2137215/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2137385/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1143963/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0358980/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0300979/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0292105/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0185968/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0076189/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0138623/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0313549/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0302015/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0124988/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0359311/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1156137/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0445474/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0360696/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0496967/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0359554/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0078184/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0496994/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2258251/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0081039/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0438794/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0125232/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1345483/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0314261/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0359602/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0401630/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0206946/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0499568/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1176930/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0082093/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0278989/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0427121/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1176904/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0125326/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0186286/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0427626/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0474950/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1167681/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0472287/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0474740/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0427386/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0474761/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1360793/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0186265/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0221322/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0436659/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0079813/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0335493/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1135484/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0083821/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0186192/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0125144/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1175042/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0279238/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0424341/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0465758/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1133608/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1133950/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0445203/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1063595/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0905984/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0089241/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0087426/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0312863/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1407268/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1345520/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0374488/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1452834/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0464792/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0125025/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0428661/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0300202/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1874632/?ref_=fn_al_tt_1',
 'No Link',
 'https://www.imdb.com/title/tt0156066/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0398692/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0374545/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0088836/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0478158/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0985617/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0312790/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0417960/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1063633/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0374912/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0313455/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0440076/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2573762/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0346694/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0291972/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0371071/?ref_=fn_al_tt_1',
 'No Link',
 'https://www.imdb.com/title/tt1321494/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1170346/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0266224/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1058059/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0452257/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0940744/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1063662/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0361291/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0408618/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0311349/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0412953/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0940750/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0349964/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0360367/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0186234/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2738426/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0422616/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0498182/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0453238/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0466671/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0124982/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0377457/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0409938/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt6080024/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1318991/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0349237/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0239326/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0437337/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0378294/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0422876/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0106277/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0418150/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0437814/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0343398/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0424068/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0422877/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0418151/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0441516/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0411303/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0437200/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0125233/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0403211/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0318434/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0457693/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0408947/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1318907/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt30781262/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0347726/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0383631/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0370054/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0218920/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt27135873/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0378075/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0441319/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0378297/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0125415/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0377388/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0377852/?ref_=fn_al_tt_1',
 'No Link',
 'https://www.imdb.com/title/tt0155844/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0211174/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0429195/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1245450/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0120466/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0216559/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0378296/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0216937/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0378513/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0325037/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0335158/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0186257/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0410154/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0216988/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0350086/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0346486/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0260769/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0343640/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0256213/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0347760/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0332364/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0294514/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0292097/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0347777/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0348087/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0313659/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0283037/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0325043/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0309331/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0346860/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0309694/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0348154/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0320574/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0310368/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0346482/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0346489/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0325233/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0360270/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0345412/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0345547/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0345623/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt27486829/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0348088/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0389833/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0391892/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0391147/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0392005/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0392121/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0393450/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0392413/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0393110/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0374328/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0421572/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0422175/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0422617/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0422716/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0422909/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0423195/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0423023/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0421879/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0479659/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0479660/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0479693/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0479631/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0479722/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0479750/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0494867/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0479725/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0494744/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0494764/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0877612/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0877651/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0877590/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0877658/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0877675/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0877709/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0877717/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0860548/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0877732/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1043426/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1047868/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1038140/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1047877/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1042547/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1042910/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1091987/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0825236/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1056421/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1305656/?ref_=fn_al_tt_1',
 'No Link',
 'https://www.imdb.com/title/tt1308642/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1320341/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1322337/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0091682/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1314843/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1099228/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1555901/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1559344/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1555991/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1488579/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1366419/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt6080024/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1470719/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1782526/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1783258/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1789759/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1612716/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1735221/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1789902/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1781909/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1781768/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2104022/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2118688/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2118570/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2101393/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2118720/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2118722/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0057171/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2529956/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2477732/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2590280/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2538176/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2590214/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2591860/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2529924/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2395459/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2995148/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2229167/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2996804/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt3342852/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2996906/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2997004/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2996950/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2995836/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt3828220/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt4304364/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt4304418/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt3837198/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt3748466/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt4304426/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt4304440/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt4304450/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt4697134/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt4842270/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt5293828/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt3875444/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt4498800/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt5201424/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt5216674/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt5293858/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt6271404/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt6186430/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt6271638/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt6271528/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt5569560/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt6271180/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt6102288/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt6271264/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt18504930/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt7736488/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt7736492/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt6947630/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt7736550/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt7248324/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt6099554/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt7736564/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt8821182/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt9270416/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt9493038/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt9139664/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt9337248/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt9395384/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt9494300/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt9389924/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt10845262/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt10827078/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt10619522/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt11531410/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt11531234/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt11531248/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt11519610/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt11531544/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt13724106/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt11091664/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt13572612/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt13582504/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt10845704/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt13723564/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt13723386/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt13702138/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt13397632/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt12867916/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt16024152/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt16024312/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt16024362/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt16024722/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt14962622/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt15006444/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt16024470/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt16729508/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt22170224/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt25147662/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt22178922/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt25147606/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt21237578/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt25145532/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt25146782/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt23771510/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt28946952/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt29544860/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt7248350/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt28152455/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt29544878/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt29544938/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt29173414/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt27203116/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt29544868/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt28929276/?ref_=fn_al_tt_1']

In [11]:
# # RE-SEARCH ITEMS WITHOUT LINKS

# driver = webdriver.Chrome()
# driver.get(r'https://www.imdb.com/')

# movies_list = list(all_movies['Title'].to_dict().values())
# all_years_list = list(all_movies['Year'].to_dict().values())
# links_list_clean = links_list.copy()

# for index in range(len(movies_list)):
#     movie = movies_list[index]
#     year = all_years_list[index]
#     link = links_list_clean[index]

#     if link == 'No Link':
#         movie = movie[:movie.find(':')]
#         search = f'{movie} {year}'
        
#         driver.find_element(By.NAME, 'q').send_keys(search)    
#         driver.find_element(By.ID, 'suggestion-search-button').click()
    
#         try:
#             driver.find_element(By.CSS_SELECTOR, '.ipc-metadata-list-summary-item__t').click()
#             current_link = driver.current_url
#             del links_list_clean[index]
#             links_list_clean.insert(index, current_link)
        
#         except NoSuchElementException:
#             del links_list_clean[index]
#             links_list_clean.insert(index, 'No Link')

# driver.quit()

In [12]:
links_list_clean = ['https://www.imdb.com/title/tt1147439/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0358943/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0122991/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0402134/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0370836/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0434119/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1178155/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1174736/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0487554/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0401234/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0159456/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0077740/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0487619/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0487669/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0188872/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0125408/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2137215/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2137385/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1143963/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0358980/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0300979/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0292105/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0185968/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0076189/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0138623/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0313549/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0302015/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0124988/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0359311/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1156137/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0445474/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0360696/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0496967/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0359554/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0078184/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0496994/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2258251/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0081039/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0438794/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0125232/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1345483/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0314261/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0359602/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0401630/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0206946/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0499568/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1176930/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0082093/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0278989/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0427121/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1176904/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0125326/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0186286/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0427626/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0474950/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1167681/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0472287/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0474740/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0427386/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0474761/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1360793/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0186265/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0221322/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0436659/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0079813/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0335493/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1135484/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0083821/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0186192/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0125144/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1175042/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0279238/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0424341/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0465758/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1133608/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1133950/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0445203/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1063595/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0905984/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0089241/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0087426/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0312863/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1407268/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1345520/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0374488/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1452834/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0464792/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0125025/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0428661/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0300202/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1874632/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0428937/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0156066/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0398692/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0374545/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0088836/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0478158/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0985617/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0312790/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0417960/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1063633/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0374912/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0313455/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0440076/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2573762/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0346694/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0291972/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0371071/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0944119/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1321494/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1170346/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0266224/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1058059/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0452257/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0940744/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1063662/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0361291/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0408618/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0311349/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0412953/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0940750/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0349964/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0360367/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0186234/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2738426/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0422616/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0498182/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0453238/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0466671/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0124982/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0377457/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0409938/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt6080024/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1318991/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0349237/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0239326/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0437337/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0378294/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0422876/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0106277/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0418150/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0437814/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0343398/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0424068/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0422877/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0418151/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0441516/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0411303/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0437200/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0125233/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0403211/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0318434/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0457693/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0408947/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1318907/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt30781262/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0347726/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0383631/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0370054/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0218920/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt27135873/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0378075/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0441319/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0378297/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0125415/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0377388/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0377852/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0335221/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0155844/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0211174/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0429195/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1245450/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0120466/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0216559/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0378296/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0216937/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0378513/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0325037/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0335158/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0186257/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0410154/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0216988/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0350086/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0346486/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0260769/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0343640/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0256213/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0347760/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0332364/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0294514/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0292097/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0347777/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0348087/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0313659/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0283037/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0325043/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0309331/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0346860/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0309694/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0348154/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0320574/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0310368/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0346482/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0346489/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0325233/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0360270/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0345412/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0345547/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0345623/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt27486829/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0348088/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0389833/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0391892/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0391147/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0392005/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0392121/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0393450/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0392413/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0393110/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0374328/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0421572/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0422175/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0422617/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0422716/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0422909/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0423195/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0423023/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0421879/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0479659/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0479660/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0479693/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0479631/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0479722/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0479750/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0494867/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0479725/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0494744/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0494764/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0877612/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0877651/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0877590/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0877658/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0877675/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0877709/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0877717/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0860548/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0877732/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1043426/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1047868/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1038140/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1047877/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1042547/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1042910/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1091987/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0825236/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1056421/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1305656/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1245468/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1308642/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1320341/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1322337/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0091682/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1314843/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1099228/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1555901/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1559344/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1555991/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1488579/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1366419/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt6080024/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1470719/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1782526/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1783258/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1789759/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1612716/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1735221/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1789902/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1781909/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt1781768/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2104022/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2118688/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2118570/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2101393/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2118720/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2118722/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt0057171/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2529956/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2477732/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2590280/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2538176/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2590214/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2591860/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2529924/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2395459/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2995148/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2229167/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2996804/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt3342852/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2996906/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2997004/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2996950/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt2995836/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt3828220/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt4304364/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt4304418/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt3837198/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt3748466/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt4304426/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt4304440/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt4304450/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt4697134/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt4842270/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt5293828/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt3875444/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt4498800/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt5201424/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt5216674/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt5293858/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt6271404/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt6186430/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt6271638/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt6271528/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt5569560/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt6271180/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt6102288/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt6271264/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt18504930/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt7736488/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt7736492/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt6947630/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt7736550/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt7248324/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt6099554/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt7736564/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt8821182/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt9270416/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt9493038/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt9139664/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt9337248/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt9395384/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt9494300/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt9389924/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt10845262/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt10827078/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt10619522/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt11531410/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt11531234/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt11531248/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt11519610/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt11531544/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt13724106/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt11091664/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt13572612/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt13582504/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt10845704/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt13723564/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt13723386/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt13702138/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt13397632/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt12867916/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt16024152/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt16024312/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt16024362/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt16024722/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt14962622/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt15006444/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt16024470/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt16729508/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt22170224/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt25147662/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt22178922/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt25147606/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt21237578/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt25145532/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt25146782/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt23771510/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt28946952/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt29544860/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt7248350/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt28152455/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt29544878/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt29544938/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt29173414/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt27203116/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt29544868/?ref_=fn_al_tt_1',
 'https://www.imdb.com/title/tt28929276/?ref_=fn_al_tt_1']

#### IF INFO NEEDS TO BE UPDATED FOR A NEW YEAR
change `start_year` and `end_year` to the years needed

In [13]:
# driver = webdriver.Chrome()
# driver.get(r'https://www.imdb.com/')

# movies_list = list(all_movies['Title'].to_dict().values())
# all_years_list = list(all_movies['Year'].to_dict().values())

# start_year = 2023
# cut_off_index = all_years_list.index(start_year)
# movies_list = movies_list[cut_off_index:]
# all_years_list = all_years_list[cut_off_index:]

# new_links_list = []

# for index in range(len(movies_list)):
#     movie = movies_list[index]
#     year = all_years_list[index]
#     search = f'{movie} {year}'
    
#     driver.find_element(By.NAME, 'q').send_keys(search)    
#     driver.find_element(By.ID, 'suggestion-search-button').click()

#     try:
#         driver.find_element(By.CSS_SELECTOR, '.ipc-metadata-list-summary-item__t').click()
#         current_link = driver.current_url
#         print(current_link)
#         new_links_list.append(current_link)
    
#     except NoSuchElementException:
#         print('No Link')
#         new_links_list.append('No Link')

# driver.quit()

In [14]:
# # RE-SEARCH ITEMS WITHOUT LINKS

# driver = webdriver.Chrome()
# driver.get(r'https://www.imdb.com/')

# movies_list = list(all_movies['Title'].to_dict().values())
# all_years_list = list(all_movies['Year'].to_dict().values())

# start_year = 2023
# cut_off_index = all_years_list.index(start_year)
# movies_list = movies_list[cut_off_index:]
# all_years_list = all_years_list[cut_off_index:]

# new_links_list_clean = new_links_list.copy()

# for index in range(len(movies_list)):
#     movie = movies_list[index]
#     year = all_years_list[index]
#     link = new_links_list_clean[index]

#     if link == 'No Link':
#         movie = movie[:movie.find(':')]
#         search = f'{movie} {year}'
        
#         driver.find_element(By.NAME, 'q').send_keys(search)    
#         driver.find_element(By.ID, 'suggestion-search-button').click()
    
#         try:
#             driver.find_element(By.CSS_SELECTOR, '.ipc-metadata-list-summary-item__t').click()
#             current_link = driver.current_url
#             del new_links_list_clean[index]
#             new_links_list_clean.insert(index, current_link)
        
#         except NoSuchElementException:
#             del new_links_list_clean[index]
#             new_links_list_clean.insert(index, 'No Link')

# driver.quit()

# links_list_clean = links_list_clean + new_links_list_clean

In [15]:
all_movies['Link 1'] = links_list_clean

### get links attempt 2

In [16]:
# # First Part

# CHROMEDRIVER_PATH = '/path/to/chromedriver'

# chrome_options = Options()
# chrome_options.add_argument('--headless')

# service = Service(CHROMEDRIVER_PATH)
# driver = webdriver.Chrome()

# years_list = [num for num in range(1975,2023+1)]
# links_dict = {year : 'https://www.imdb.com/event/ev0000431/'+str(year)+'/1/?ref_=ev_eh' for year in years_list}

# titledict = {}
# filmdict = {}
# links = []
# titles = []

# for year in years_list:
#     url = links_dict[year]

#     driver.get(url)
#     driver.implicitly_wait(3)
    
#     #COLLECTING THE LIST OF LINKS
#     link_elem = driver.find_elements(By.TAG_NAME, 'a')

#     for link in link_elem:
#         links.append(str(link.get_attribute('href')))
#     for link in links:
#         if 'title' in link:
#             titles.append(link)
#     del titles[0]

#     for title in titles:
#         titledict[title] = ''
    
#     filmdict[year] = titledict.copy()

#     print(filmdict)
    
#     links.clear()
#     titles.clear()
#     titledict.clear()

#     #linkset = set(titles)
#     #linksetlist = list(linkset)

# driver.quit()

In [17]:
# # Second Part

# #DICTIONARY NA

# CHROMEDRIVER_PATH = '/path/to/chromedriver'

# chrome_options = Options()
# chrome_options.add_argument('--headless')

# service = Service(CHROMEDRIVER_PATH)
# driver = webdriver.Chrome()

# for year in filmdict:
#     smalldict = {}
#     for film in filmdict[year]:
#         driver.get(film)
#         driver.implicitly_wait(5)
#         film_elem = driver.find_element(By.CSS_SELECTOR,'.hero__primary-text')
#         smalldict[film_elem.text] = film
#     filmdict[year] = smalldict

# driver.quit()

# print(filmdict)

In [18]:
filmdict = {1975: {'Diligin mo ng hamog ang uhaw na lupa': 'https://www.imdb.com/title/tt0402134/?ref_=ev_nom',
  'Araw-araw, gabi-gabi': 'https://www.imdb.com/title/tt0358943/?ref_=ev_nom',
  'Kapitan Kulas': 'https://www.imdb.com/title/tt0370836/?ref_=ev_nom',
  "Batu-bato sa langit: Ang tamaa'y huwag magagalit": 'https://www.imdb.com/title/tt0122991/?ref_=ev_nom',
  'Alat': 'https://www.imdb.com/title/tt1147439/?ref_=ev_nom',
  'Karugtong ang kahapon': 'https://www.imdb.com/title/tt0434119/?ref_=ev_nom',
  'Postcards from China': 'https://www.imdb.com/title/tt1178155/?ref_=ev_nom',
  "Siya'y Umalis, Siya'y Dumating": 'https://www.imdb.com/title/tt1174736/?ref_=ev_nom'},
 1976: {'This Is How We Were Before, How Are You Doing Now?': 'https://www.imdb.com/title/tt0159456/?ref_=ev_nom',
  'Insiang': 'https://www.imdb.com/title/tt0077740/?ref_=ev_nom',
  'Once a Moth': 'https://www.imdb.com/title/tt0125408/?ref_=ev_nom',
  'Barok': 'https://www.imdb.com/title/tt0487554/?ref_=ev_nom',
  'Dateline Chicago: Arrest the Nurse Killer': 'https://www.imdb.com/title/tt0401234/?ref_=ev_nom',
  'Kisame Street': 'https://www.imdb.com/title/tt0487669/?ref_=ev_nom',
  'Makahiya at talahib': 'https://www.imdb.com/title/tt0188872/?ref_=ev_nom',
  'Puwede ako, puwede ka pa ba?': 'https://www.imdb.com/title/tt2137215/?ref_=ev_nom',
  'The Interceptors': 'https://www.imdb.com/title/tt0487619/?ref_=ev_nom',
  'The Rebel Hunter': 'https://www.imdb.com/title/tt2137385/?ref_=ev_nom'},
 1977: {'Burlesk Queen': 'https://www.imdb.com/title/tt0185968/?ref_=ev_nom',
  'Bakya mo Neneng': 'https://www.imdb.com/title/tt0358980/?ref_=ev_nom',
  'The Captive Virgins': 'https://www.imdb.com/title/tt0292105/?ref_=ev_nom'},
 1978: {'Katawang alabok': 'https://www.imdb.com/title/tt0496967/?ref_=ev_nom',
  'Atsay': 'https://www.imdb.com/title/tt0124988/?ref_=ev_nom',
  'Rubia Servios': 'https://www.imdb.com/title/tt0078184/?ref_=ev_nom'},
 1979: {'Ina ka ng anak mo': 'https://www.imdb.com/title/tt0125232/?ref_=ev_nom',
  'Ang lihim ng Guadalupe': 'https://www.imdb.com/title/tt0359602/?ref_=ev_nom',
  'Kadete': 'https://www.imdb.com/title/tt1345483/?ref_=ev_nom',
  'Kasal-kasalan, bahay-bahayan': 'https://www.imdb.com/title/tt0314261/?ref_=ev_nom',
  'Ang sisiw ay isang agila': 'https://www.imdb.com/title/tt0499568/?ref_=ev_nom',
  'Ang alamat ni Julian Makabayan': 'https://www.imdb.com/title/tt0081039/?ref_=ev_nom'},
 1980: {'Langis at tubig': 'https://www.imdb.com/title/tt0186286/?ref_=ev_nom',
  'Brutal': 'https://www.imdb.com/title/tt0278989/?ref_=ev_nom',
  "Kung ako'y iiwan mo": 'https://www.imdb.com/title/tt0125326/?ref_=ev_nom',
  'Bona': 'https://www.imdb.com/title/tt0082093/?ref_=ev_nom',
  "Kape't gatas": 'https://www.imdb.com/title/tt1176904/?ref_=ev_nom',
  'Taga sa panahon': 'https://www.imdb.com/title/tt0474950/?ref_=ev_nom',
  'Ang panday': 'https://www.imdb.com/title/tt0427626/?ref_=ev_nom'},
 1981: {'In the Wink of an Eye': 'https://www.imdb.com/title/tt0221322/?ref_=ev_nom',
  'Karma': 'https://www.imdb.com/title/tt0186265/?ref_=ev_nom',
  "Rock n' Roll": 'https://www.imdb.com/title/tt0313536/?ref_=ev_nom',
  'Ang babae sa ulog': 'https://www.imdb.com/title/tt0472287/?ref_=ev_nom'},
 1982: {'Miracle': 'https://www.imdb.com/title/tt0125144/?ref_=ev_nom',
  'Ang panday: Ikatlong yugto': 'https://www.imdb.com/title/tt0424341/?ref_=ev_nom',
  'Moral': 'https://www.imdb.com/title/tt0279238/?ref_=ev_nom',
  'The House in the Woods': 'https://www.imdb.com/title/tt0186192/?ref_=ev_nom',
  'Desire': 'https://www.imdb.com/title/tt0085429/?ref_=ev_nom',
  'Magindanao': 'https://www.imdb.com/title/tt1175042/?ref_=ev_nom',
  'Santa Claus Is Coming to Town!': 'https://www.imdb.com/title/tt0465758/?ref_=ev_nom',
  'Tatlo silang tatay ko': 'https://www.imdb.com/title/tt1133608/?ref_=ev_nom',
  "The Cute... the Sexy n' the Tiny": 'https://www.imdb.com/title/tt1135484/?ref_=ev_nom',
  'Tulisan ng pasong musang': 'https://www.imdb.com/title/tt1133950/?ref_=ev_nom'},
 1983: {'Bago kumalat ang kamandag': 'https://www.imdb.com/title/tt1063595/?ref_=ev_nom',
  'Tengteng de sarapen': 'https://www.imdb.com/title/tt1345520/?ref_=ev_nom',
  'Of the Flesh': 'https://www.imdb.com/title/tt0312863/?ref_=ev_nom',
  'Hot Property': 'https://www.imdb.com/title/tt0087426/?ref_=ev_nom',
  'Bad Bananas on the Silver Screen': 'https://www.imdb.com/title/tt0445203/?ref_=ev_nom',
  "D' Godson": 'https://www.imdb.com/title/tt0905984/?ref_=ev_nom',
  'Over My Dead Body': 'https://www.imdb.com/title/tt1407268/?ref_=ev_nom',
  'The Gunfighter': 'https://www.imdb.com/title/tt0089241/?ref_=ev_nom'},
 1984: {'Shake, Rattle & Roll': 'https://www.imdb.com/title/tt0156066/?ref_=ev_nom',
  'Bulaklak sa City Jail': 'https://www.imdb.com/title/tt0125025/?ref_=ev_nom',
  'Alapaap': 'https://www.imdb.com/title/tt0374488/?ref_=ev_nom',
  'Idol': 'https://www.imdb.com/title/tt0428661/?ref_=ev_nom',
  'Ang panday IV': 'https://www.imdb.com/title/tt0428937/?ref_=ev_nom',
  "Atsay killer buti nga sa'yo": 'https://www.imdb.com/title/tt1452834/?ref_=ev_nom',
  'Bukas... May pangarap': 'https://www.imdb.com/title/tt0464792/?ref_=ev_nom',
  'Misteryo sa tuwa': 'https://www.imdb.com/title/tt0300202/?ref_=ev_nom',
  'Muntinlupa': 'https://www.imdb.com/title/tt1874632/?ref_=ev_nom'},
 1985: {'Moises Padilla Story: The Missing Chapter': 'https://www.imdb.com/title/tt1063633/?ref_=ev_nom',
  'Paradise Inn': 'https://www.imdb.com/title/tt0313455/?ref_=ev_nom',
  "I Can't Stop Loving You": 'https://www.imdb.com/title/tt0312790/?ref_=ev_nom',
  'Ano ang kulay ng mukha ng Diyos?': 'https://www.imdb.com/title/tt0398692/?ref_=ev_nom',
  'Boboy Tibayan: Tigre ng Cavite': 'https://www.imdb.com/title/tt0374545/?ref_=ev_nom',
  'Bomba Arienda': 'https://www.imdb.com/title/tt0088836/?ref_=ev_nom',
  'God ...Save Me!': 'https://www.imdb.com/title/tt0478158/?ref_=ev_nom',
  'Hee-Man: Master of None': 'https://www.imdb.com/title/tt0985617/?ref_=ev_nom',
  'Menudo at pandesal': 'https://www.imdb.com/title/tt0417960/?ref_=ev_nom',
  'Oks na oks pakner': 'https://www.imdb.com/title/tt0374912/?ref_=ev_nom'},
 1986: {'Halimaw': 'https://www.imdb.com/title/tt0291972/?ref_=ev_nom',
  'Payaso': 'https://www.imdb.com/title/tt0371071/?ref_=ev_nom',
  'Bagets Gang': 'https://www.imdb.com/title/tt0440076/?ref_=ev_nom'},
 1987: {'Anak Badjao': 'https://www.imdb.com/title/tt1058059/?ref_=ev_nom',
  'The Untold Story of Melanie Marquez': 'https://www.imdb.com/title/tt1063662/?ref_=ev_nom',
  'Action Is Not Missing': 'https://www.imdb.com/title/tt0266224/?ref_=ev_nom',
  'Olongapo... The Great American Dream': 'https://www.imdb.com/title/tt0940744/?ref_=ev_nom',
  '1 + 1 = 12 + 1': 'https://www.imdb.com/title/tt1170346/?ref_=ev_nom',
  'Huwag mong buhayin ang bangkay': 'https://www.imdb.com/title/tt0452257/?ref_=ev_nom'},
 1988: {'Patrolman': 'https://www.imdb.com/title/tt0940750/?ref_=ev_nom',
  'Magkano ang iyong dangal?': 'https://www.imdb.com/title/tt0412953/?ref_=ev_nom',
  "Bubble's Ativan Gang": 'https://www.imdb.com/title/tt0408618/?ref_=ev_nom',
  'Agila ng Maynila': 'https://www.imdb.com/title/tt0361291/?ref_=ev_nom',
  'Itanong mo sa buwan': 'https://www.imdb.com/title/tt0311349/?ref_=ev_nom',
  'Pik pak boom': 'https://www.imdb.com/title/tt0349964/?ref_=ev_nom'},
 1989: {'Imortal': 'https://www.imdb.com/title/tt0186234/?ref_=ev_nom'},
 1990: {'Espadang patpat': 'https://www.imdb.com/title/tt0409938/?ref_=ev_nom',
  'Andrea, paano ba ang maging isang ina?': 'https://www.imdb.com/title/tt0124982/?ref_=ev_nom',
  'Ama... Bakit mo ako pinabayaan?': 'https://www.imdb.com/title/tt0466671/?ref_=ev_nom',
  'Tumakbo ka hanggang may lupa': 'https://www.imdb.com/title/tt1318991/?ref_=ev_nom',
  'Baril ko ang uusig': 'https://www.imdb.com/title/tt0377457/?ref_=ev_nom',
  'Shake, Rattle & Roll 2': 'https://www.imdb.com/title/tt0318683/?ref_=ev_nom'},
 1991: {'Juan Tamad at Mr. Shooli: Mongolian Barbecue': 'https://www.imdb.com/title/tt0437337/?ref_=ev_nom',
  'Ang totoong buhay ni Pacita M.': 'https://www.imdb.com/title/tt0106277/?ref_=ev_nom',
  'Okay ka, fairy ko!': 'https://www.imdb.com/title/tt0422876/?ref_=ev_nom',
  'Darna': 'https://www.imdb.com/title/tt0239326/?ref_=ev_nom',
  'Contreras Gang': 'https://www.imdb.com/title/tt0349237/?ref_=ev_nom',
  'Magdalena S. Palacol Story': 'https://www.imdb.com/title/tt0378294/?ref_=ev_nom'},
 1992: {'Bakit labis kitang mahal': 'https://www.imdb.com/title/tt0343398/?ref_=ev_nom',
  'Shake Rattle & Roll IV': 'https://www.imdb.com/title/tt0418151/?ref_=ev_nom',
  'Takbo... Talon... Tili!!!': 'https://www.imdb.com/title/tt0441516/?ref_=ev_nom',
  'Andres Manambit: Angkan ng matatapang': 'https://www.imdb.com/title/tt0437814/?ref_=ev_nom',
  'Engkanto': 'https://www.imdb.com/title/tt0424068/?ref_=ev_nom',
  'Okay ka, fairy ko! Part 2': 'https://www.imdb.com/title/tt0422877/?ref_=ev_nom'},
 1993: {'Loving Someone': 'https://www.imdb.com/title/tt0318434/?ref_=ev_nom',
  'Kung mawawala ka pa': 'https://www.imdb.com/title/tt0403211/?ref_=ev_nom',
  'Inay': 'https://www.imdb.com/title/tt0125233/?ref_=ev_nom',
  'Doring Borobo': 'https://www.imdb.com/title/tt0411303/?ref_=ev_nom',
  'Gaano kita kamahal': 'https://www.imdb.com/title/tt0437200/?ref_=ev_nom',
  'Pusoy dos': 'https://www.imdb.com/title/tt0457693/?ref_=ev_nom'},
 1994: {},
 1995: {'Rage': 'https://www.imdb.com/title/tt0218920/?ref_=ev_nom',
  'Muling umawit ang puso': 'https://www.imdb.com/title/tt0125415/?ref_=ev_nom'},
 1996: {'Magic Temple': 'https://www.imdb.com/title/tt0155844/?ref_=ev_nom'},
 1997: {'Rizal in Dapitan': 'https://www.imdb.com/title/tt0125475/?ref_=ev_nom',
  'Nasaan ang puso': 'https://www.imdb.com/title/tt0216937/?ref_=ev_nom',
  'Babae': 'https://www.imdb.com/title/tt0216559/?ref_=ev_nom'},
 1998: {'José Rizal': 'https://www.imdb.com/title/tt0186257/?ref_=ev_nom',
  'Kasal-kasalan (Sakalan)': 'https://www.imdb.com/title/tt0410154/?ref_=ev_nom'},
 1999: {'Muro-ami': 'https://www.imdb.com/title/tt0256213/?ref_=ev_nom',
  'Bulaklak ng Maynila': 'https://www.imdb.com/title/tt0260769/?ref_=ev_nom',
  'José Rizal': 'https://www.imdb.com/title/tt0186257/?ref_=ev_nom'},
 2000: {'A Change of Heart': 'https://www.imdb.com/title/tt0283037/?ref_=ev_nom',
  'Deathrow': 'https://www.imdb.com/title/tt0294514/?ref_=ev_nom',
  'Sugatang puso': 'https://www.imdb.com/title/tt0313659/?ref_=ev_nom',
  'Markova: Comfort Gay': 'https://www.imdb.com/title/tt0292097/?ref_=ev_nom',
  'Ping Lacson: Super Cop': 'https://www.imdb.com/title/tt0347777/?ref_=ev_nom',
  'Spirit Warriors': 'https://www.imdb.com/title/tt0348087/?ref_=ev_nom'},
 2001: {"Yamashita: The Tiger's Treasure": 'https://www.imdb.com/title/tt0310368/?ref_=ev_nom'},
 2002: {'Mano po': 'https://www.imdb.com/title/tt0345623/?ref_=ev_nom'},
 2003: {'Crying Ladies': 'https://www.imdb.com/title/tt0391892/?ref_=ev_nom'},
 2004: {'Panaghóy sa subâ: The Call of the River': 'https://www.imdb.com/title/tt0422909/?ref_=ev_nom',
  'Kulimlim': 'https://www.imdb.com/title/tt0418810/?ref_=ev_nom',
  'Mano po III: My Love': 'https://www.imdb.com/title/tt0422716/?ref_=ev_nom',
  'Aishite imasu (Mahal kita) 1941': 'https://www.imdb.com/title/tt0421572/?ref_=ev_nom',
  'Lastikman': 'https://www.imdb.com/title/tt0422617/?ref_=ev_nom'},
 2005: {'Kutob': 'https://www.imdb.com/title/tt0479693/?ref_=ev_nom',
  'Ako legal wife: Mano po 4?': 'https://www.imdb.com/title/tt0479631/?ref_=ev_nom',
  'Shake Rattle & Roll 2k5': 'https://www.imdb.com/title/tt0479750/?ref_=ev_nom',
  'Exodus: Tales from the Enchanted Kingdom': 'https://www.imdb.com/title/tt0479660/?ref_=ev_nom',
  'Blue Moon': 'https://www.imdb.com/title/tt0479725/?ref_=ev_nom',
  'Enteng Kabisote 2: Okay ka fairy ko... The legend continues': 'https://www.imdb.com/title/tt0479659/?ref_=ev_nom',
  'Mulawin: The Movie': 'https://www.imdb.com/title/tt0479722/?ref_=ev_nom'},
 2006: {'Ligalig': 'https://www.imdb.com/title/tt0877590/?ref_=ev_nom',
  'Magic Mirror': 'https://www.imdb.com/title/tt0472521/?ref_=ev_nom',
  'Kasal, kasali, kasalo': 'https://www.imdb.com/title/tt0877651/?ref_=ev_nom',
  'Shake Rattle and Roll 8': 'https://www.imdb.com/title/tt0877709/?ref_=ev_nom',
  'Mano po 5: Gua ai di (I love you)': 'https://www.imdb.com/title/tt0877658/?ref_=ev_nom',
  'Tatlong baraha': 'https://www.imdb.com/title/tt0860548/?ref_=ev_nom',
  'Enteng Kabisote 3: Okay ka fairy ko... The legend goes on and on and on': 'https://www.imdb.com/title/tt0877612/?ref_=ev_nom'},
 2007: {'Katas ng Saudi': 'https://www.imdb.com/title/tt1038140/?ref_=ev_nom',
  'Bahay kubo: A Pinoy mano po!': 'https://www.imdb.com/title/tt1043426/?ref_=ev_nom',
  'Shake, Rattle & Roll 9': 'https://www.imdb.com/title/tt1042910/?ref_=ev_nom',
  'Transmutators': 'https://www.imdb.com/title/tt1047877/?ref_=ev_nom',
  'Banal': 'https://www.imdb.com/title/tt1077074/?ref_=ev_nom',
  'Desperadas': 'https://www.imdb.com/title/tt1056421/?ref_=ev_nom',
  'Anak ng Kumander': 'https://www.imdb.com/title/tt1091987/?ref_=ev_nom',
  'Sakal, sakali, saklolo': 'https://www.imdb.com/title/tt1042547/?ref_=ev_nom',
  'Enteng Kabisote 4: Okay ka fairy ko... The beginning of the legend': 'https://www.imdb.com/title/tt1047868/?ref_=ev_nom'},
 2008: {'Magkaibigan': 'https://www.imdb.com/title/tt1322337/?ref_=ev_nom',
  'Baler': 'https://www.imdb.com/title/tt1305656/?ref_=ev_nom',
  'Shake Rattle & Roll X': 'https://www.imdb.com/title/tt1314843/?ref_=ev_nom',
  'Desperadas 2': 'https://www.imdb.com/title/tt1308642/?ref_=ev_nom',
  'Dayo': 'https://www.imdb.com/title/tt1245468/?ref_=ev_nom',
  'One Night Only': 'https://www.imdb.com/title/tt1339339/?ref_=ev_nom',
  'The Only Mother to You All': 'https://www.imdb.com/title/tt1099228/?ref_=ev_nom',
  'Iskul Bukol: 20 Years After (The Ungasis and Escaleras Adventure)': 'https://www.imdb.com/title/tt1320341/?ref_=ev_nom'},
 2009: {'Ang panday': 'https://www.imdb.com/title/tt1366419/?ref_=ev_nom',
  "Mano po 6: A Mother's Love": 'https://www.imdb.com/title/tt1488579/?ref_=ev_nom',
  'I Love You Goodbye': 'https://www.imdb.com/title/tt1559344/?ref_=ev_nom',
  'Shake Rattle & Roll XI': 'https://www.imdb.com/title/tt1488593/?ref_=ev_nom',
  'Ang darling kong aswang': 'https://www.imdb.com/title/tt1555901/?ref_=ev_nom'},
 2010: {'Father Jejemon': 'https://www.imdb.com/title/tt1789759/?ref_=ev_nom',
  'Si Agimat at si Enteng Kabisote': 'https://www.imdb.com/title/tt1782526/?ref_=ev_nom',
  'Rosario': 'https://www.imdb.com/title/tt1733275/?ref_=ev_nom',
  "Ang tanging ina mo: Last na 'to!": 'https://www.imdb.com/title/tt1781768/?ref_=ev_nom',
  'Shake Rattle and Roll 12': 'https://www.imdb.com/title/tt1789902/?ref_=ev_nom',
  'Super Inday and the Golden Bibe': 'https://www.imdb.com/title/tt0479546/?ref_=ev_nom',
  'Dalaw': 'https://www.imdb.com/title/tt1783258/?ref_=ev_nom',
  'RPG Metanoia': 'https://www.imdb.com/title/tt1735221/?ref_=ev_nom',
  'Presa': 'https://www.imdb.com/title/tt1783362/?ref_=ev_nom',
  'Nasaan si Hefte?': 'https://www.imdb.com/title/tt1785489/?ref_=ev_nom',
  'Rindido': 'https://www.imdb.com/title/tt1826835/?ref_=ev_nom',
  'Senior Year': 'https://www.imdb.com/title/tt1770732/?ref_=ev_nom',
  'Slow Fade': 'https://www.imdb.com/title/tt1783374/?ref_=ev_nom'},
 2011: {'Segunda mano': 'https://www.imdb.com/title/tt2118720/?ref_=ev_nom',
  'Yesterday Today Tomorrow': 'https://www.imdb.com/title/tt2118771/?ref_=ev_nom',
  'Manila Kingpin: The Asiong Salonga Story': 'https://www.imdb.com/title/tt2101393/?ref_=ev_nom',
  'My Househusband: Ikaw na!': 'https://www.imdb.com/title/tt2118688/?ref_=ev_nom',
  'Enteng ng Ina mo': 'https://www.imdb.com/title/tt2104022/?ref_=ev_nom',
  'Shake Rattle Roll 13': 'https://www.imdb.com/title/tt2118722/?ref_=ev_nom',
  'Ang panday 2': 'https://www.imdb.com/title/tt2118570/?ref_=ev_nom',
  'Pintakasi': 'https://www.imdb.com/title/tt2133306/?ref_=ev_nom',
  'HIV: Si Heidi, si Ivy at si V': 'https://www.imdb.com/title/tt1794800/?ref_=ev_nom',
  'Dyagwar: Havey o waley': 'https://www.imdb.com/title/tt2133215/?ref_=ev_nom',
  'Haruo': 'https://www.imdb.com/title/tt2136975/?ref_=ev_nom',
  'Ritwal: The Faithfools': 'https://www.imdb.com/title/tt1998304/?ref_=ev_nom',
  'Ang payaso at ang prinsesa': 'https://www.imdb.com/title/tt2165593/?ref_=ev_nom',
  'Máte': 'https://www.imdb.com/title/tt1854557/?ref_=ev_nom',
  'Adivino': 'https://www.imdb.com/title/tt2165563/?ref_=ev_nom',
  'Bagong ligo': 'https://www.imdb.com/title/tt2165623/?ref_=ev_nom',
  'Biyahe ni Barbie': 'https://www.imdb.com/title/tt2165643/?ref_=ev_nom',
  'I See Everything': 'https://www.imdb.com/title/tt2165843/?ref_=ev_nom',
  'Oras': 'https://www.imdb.com/title/tt2166035/?ref_=ev_nom',
  'Sanayan lang ang pagpatay': 'https://www.imdb.com/title/tt2166113/?ref_=ev_nom',
  'Speechless': 'https://www.imdb.com/title/tt2166133/?ref_=ev_nom',
  'Ulan': 'https://www.imdb.com/title/tt2166281/?ref_=ev_nom'},
 2012: {'One More Try': 'https://www.imdb.com/title/tt2590280/?ref_=ev_nom',
  'Thy Womb': 'https://www.imdb.com/title/tt2395459/?ref_=ev_nom',
  'Shake Rattle and Roll Fourteen: The Invasion': 'https://www.imdb.com/title/tt2538176/?ref_=ev_nom',
  'El Presidente': 'https://www.imdb.com/title/tt2477732/?ref_=ev_nom',
  'Sisterakas': 'https://www.imdb.com/title/tt2590214/?ref_=ev_nom',
  'Si Agimat si Enteng Kabisote at si ako': 'https://www.imdb.com/title/tt2529956/?ref_=ev_nom',
  'Sosy Problems': 'https://www.imdb.com/title/tt2591860/?ref_=ev_nom',
  'The Strangers': 'https://www.imdb.com/title/tt2529924/?ref_=ev_nom',
  'Gayak': 'https://www.imdb.com/title/tt2590162/?ref_=ev_nom',
  'In nomine matris': 'https://www.imdb.com/title/tt2590186/?ref_=ev_nom',
  'The Grave Bandits': 'https://www.imdb.com/title/tt2557172/?ref_=ev_nom',
  'Ad ignorantiam': 'https://www.imdb.com/title/tt2590110/?ref_=ev_nom',
  'Paglaya sa Tanikala': 'https://www.imdb.com/title/tt2431988/?ref_=ev_nom',
  'Pukpok': 'https://www.imdb.com/title/tt2779828/?ref_=ev_nom',
  'Kinse': 'https://www.imdb.com/title/tt2780026/?ref_=ev_nom',
  'Lugaw': 'https://www.imdb.com/title/tt2780024/?ref_=ev_nom',
  'Manibela': 'https://www.imdb.com/title/tt2779796/?ref_=ev_nom',
  'Obsesyon': 'https://www.imdb.com/title/tt2779838/?ref_=ev_nom',
  'Ritwal': 'https://www.imdb.com/title/tt2779996/?ref_=ev_nom',
  'Rolyo': 'https://www.imdb.com/title/tt2779966/?ref_=ev_nom',
  'Sonata': 'https://www.imdb.com/title/tt2780028/?ref_=ev_nom',
  'Tagad': 'https://www.imdb.com/title/tt2780022/?ref_=ev_nom',
  'Tsansa': 'https://www.imdb.com/title/tt2780020/?ref_=ev_nom'},
 2013: {'10000 Hours': 'https://www.imdb.com/title/tt2995148/?ref_=ev_nom',
  'Pagpag: Nine Lives': 'https://www.imdb.com/title/tt2996950/?ref_=ev_nom',
  'Boy Golden: Shoot to Kill': 'https://www.imdb.com/title/tt3419966/?ref_=ev_nom',
  'Pedro Calungsod: Batang martir': 'https://www.imdb.com/title/tt2995836/?ref_=ev_nom',
  'My Little Bossings': 'https://www.imdb.com/title/tt2997004/?ref_=ev_nom',
  'Female, Male, Gay, Lesbian': 'https://www.imdb.com/title/tt2996804/?ref_=ev_nom',
  'Kimmy Dora: Ang kiyemeng prequel': 'https://www.imdb.com/title/tt2996906/?ref_=ev_nom',
  'Kaleidoscope World': 'https://www.imdb.com/title/tt3342852/?ref_=ev_nom',
  'Dukit': 'https://www.imdb.com/title/tt3411758/?ref_=ev_nom',
  'Mga anino ng kahapon': 'https://www.imdb.com/title/tt3209894/?ref_=ev_nom',
  'Kaleh and Mbaki': 'https://www.imdb.com/title/tt3101052/?ref_=ev_nom',
  'Ang lalong ni Kulakog': 'https://www.imdb.com/title/tt3411784/?ref_=ev_nom',
  'Gayuma ni Maria': 'https://www.imdb.com/title/tt3411806/?ref_=ev_nom',
  'Mamang pulis': 'https://www.imdb.com/title/tt3411818/?ref_=ev_nom',
  'Origin of Mang Jose': 'https://www.imdb.com/title/tt3411776/?ref_=ev_nom',
  'Ang maestra': 'https://www.imdb.com/title/tt3411756/?ref_=ev_nom',
  'Island Dreams': 'https://www.imdb.com/title/tt2728672/?ref_=ev_nom',
  'Saka-saka': 'https://www.imdb.com/title/tt3411772/?ref_=ev_nom',
  'Ang paglisan': 'https://www.imdb.com/title/tt3535432/?ref_=ev_nom',
  '#NoFilter': 'https://www.imdb.com/title/tt3411820/?ref_=ev_nom',
  'Ang huling balikbayan box': 'https://www.imdb.com/title/tt3535436/?ref_=ev_nom',
  'Gapos': 'https://www.imdb.com/title/tt3411830/?ref_=ev_nom',
  'Hintayin mo sa Seq. 24': 'https://www.imdb.com/title/tt3411828/?ref_=ev_nom'},
 2014: {'English Only, Please': 'https://www.imdb.com/title/tt4304418/?ref_=ev_nom',
  'Feng Shui 2': 'https://www.imdb.com/title/tt3837198/?ref_=ev_nom',
  'Kubot: The Aswang Chronicles 2': 'https://www.imdb.com/title/tt3748466/?ref_=ev_nom',
  'Bonifacio: Ang unang pangulo': 'https://www.imdb.com/title/tt4304364/?ref_=ev_nom',
  'Shake Rattle & Roll XV': 'https://www.imdb.com/title/tt4304450/?ref_=ev_nom',
  'My Big Bossing': 'https://www.imdb.com/title/tt4304440/?ref_=ev_nom',
  'Magnum Muslim .357': 'https://www.imdb.com/title/tt4304426/?ref_=ev_nom',
  'The Amazing Praybeyt Benjamin': 'https://www.imdb.com/title/tt3828220/?ref_=ev_nom',
  'The Coffin Maker': 'https://www.imdb.com/title/tt3989186/?ref_=ev_nom',
  "M: Mother's Maiden Name": 'https://www.imdb.com/title/tt4078780/?ref_=ev_nom',
  'An maogmang lugar': 'https://www.imdb.com/title/tt4316808/?ref_=ev_nom',
  'Cherry': 'https://www.imdb.com/title/tt4316816/?ref_=ev_nom',
  'Gymsnatch': 'https://www.imdb.com/title/tt4316840/?ref_=ev_nom',
  'Isip-bata': 'https://www.imdb.com/title/tt4316832/?ref_=ev_nom',
  'Shifter': 'https://www.imdb.com/title/tt4316850/?ref_=ev_nom',
  'Awaken': 'https://www.imdb.com/title/tt4288802/?ref_=ev_nom',
  'Gemini': 'https://www.imdb.com/title/tt3764970/?ref_=ev_nom',
  'Maratabat': 'https://www.imdb.com/title/tt4288788/?ref_=ev_nom',
  'Bundok chubibo': 'https://www.imdb.com/title/tt4316782/?ref_=ev_nom',
  'Ang soltera': 'https://www.imdb.com/title/tt4316926/?ref_=ev_nom',
  'Bimyana': 'https://www.imdb.com/title/tt4141484/?ref_=ev_nom',
  'Kalaw': 'https://www.imdb.com/title/tt4316796/?ref_=ev_nom',
  'Kubli': 'https://www.imdb.com/title/tt4316882/?ref_=ev_nom',
  'Siyanawa': 'https://www.imdb.com/title/tt4316900/?ref_=ev_nom'},
 2015: {'#WalangForever': 'https://www.imdb.com/title/tt5293858/?ref_=ev_nom',
  'All You Need Is Pag-ibig': 'https://www.imdb.com/title/tt4697134/?ref_=ev_nom',
  'Honor Thy Father': 'https://www.imdb.com/title/tt4498800/?ref_=ev_nom',
  'My Bebe Love: #KiligPaMore': 'https://www.imdb.com/title/tt5201424/?ref_=ev_nom',
  'Beauty and the Bestie': 'https://www.imdb.com/title/tt4842270/?ref_=ev_nom',
  'Buy Now, Die Later': 'https://www.imdb.com/title/tt5293828/?ref_=ev_nom',
  'Geisha of Death': 'https://www.imdb.com/title/tt5216674/?ref_=ev_nom',
  'Haunted Mansion': 'https://www.imdb.com/title/tt3875444/?ref_=ev_nom',
  'Geo': 'https://www.imdb.com/title/tt5294140/?ref_=ev_nom',
  'Ari: My Life with a King': 'https://www.imdb.com/title/tt5166946/?ref_=ev_nom',
  'Momento': 'https://www.imdb.com/title/tt5294214/?ref_=ev_nom',
  'Tandem': 'https://www.imdb.com/title/tt4838342/?ref_=ev_nom',
  'Toto': 'https://www.imdb.com/title/tt4933106/?ref_=ev_nom',
  'Buttons': 'https://www.imdb.com/title/tt5294160/?ref_=ev_nom',
  '8': 'https://www.imdb.com/title/tt5294118/?ref_=ev_nom',
  'Little Lights': 'https://www.imdb.com/title/tt5294148/?ref_=ev_nom',
  "Marvino's League of Superheroes": 'https://www.imdb.com/title/tt5294128/?ref_=ev_nom',
  'Mandirigma': 'https://www.imdb.com/title/tt5293878/?ref_=ev_nom',
  'Turo turo': 'https://www.imdb.com/title/tt5294092/?ref_=ev_nom',
  'Mumu': 'https://www.imdb.com/title/tt5294200/?ref_=ev_nom',
  'Daisy': 'https://www.imdb.com/title/tt5294232/?ref_=ev_nom',
  'Ding mangasyas': 'https://www.imdb.com/title/tt5294184/?ref_=ev_nom',
  'Lapis': 'https://www.imdb.com/title/tt5294192/?ref_=ev_nom'},
 2016: {'Die Beautiful': 'https://www.imdb.com/title/tt6186430/?ref_=ev_nom',
  'Oro': 'https://www.imdb.com/title/tt6271528/?ref_=ev_nom',
  'Vince & Kath & James': 'https://www.imdb.com/title/tt6271264/?ref_=ev_nom',
  'Ang babae sa septic tank 2: #ForeverIsNotEnough': 'https://www.imdb.com/title/tt6271404/?ref_=ev_nom',
  'Kabisera': 'https://www.imdb.com/title/tt6271638/?ref_=ev_nom',
  'Seklusyon': 'https://www.imdb.com/title/tt6271180/?ref_=ev_nom',
  'Saving Sally': 'https://www.imdb.com/title/tt5569560/?ref_=ev_nom',
  'Sunday Beauty Queen': 'https://www.imdb.com/title/tt6102288/?ref_=ev_nom',
  'Mga bitoon sa siyudad': 'https://www.imdb.com/title/tt6375200/?ref_=ev_nom',
  'Mitatang': 'https://www.imdb.com/title/tt6378216/?ref_=ev_nom',
  'EJK': 'https://www.imdb.com/title/tt6378194/?ref_=ev_nom',
  'Birds': 'https://www.imdb.com/title/tt6378178/?ref_=ev_nom',
  'Manila Scream': 'https://www.imdb.com/title/tt6378212/?ref_=ev_nom',
  'Momo': 'https://www.imdb.com/title/tt6273266/?ref_=ev_nom',
  'Passage of Life': 'https://www.imdb.com/title/tt6378222/?ref_=ev_nom',
  'Sitsiritsit': 'https://www.imdb.com/title/tt6378230/?ref_=ev_nom'},
 2017: {'All of You': 'https://www.imdb.com/title/tt7736536/?ref_=ev_nom',
  'Ang Panday': 'https://www.imdb.com/title/tt6947630/?ref_=ev_nom',
  'Siargao': 'https://www.imdb.com/title/tt7736564/?ref_=ev_nom',
  'Deadma Walking': 'https://www.imdb.com/title/tt7736550/?ref_=ev_nom',
  'The Portrait': 'https://www.imdb.com/title/tt6099554/?ref_=ev_nom',
  'Meant to Beh': 'https://www.imdb.com/title/tt7736492/?ref_=ev_nom',
  'Gandarrapiddo: The Revenger Squad': 'https://www.imdb.com/title/tt7736488/?ref_=ev_nom',
  'Haunted Forest': 'https://www.imdb.com/title/tt7248324/?ref_=ev_nom',
  'Anong nangyari kay Nicanor Dante?': 'https://www.imdb.com/title/tt7815952/?ref_=ev_nom',
  'Anino': 'https://www.imdb.com/title/tt8317374/?ref_=ev_nom',
  'Black Market': 'https://www.imdb.com/title/tt8317584/?ref_=ev_nom',
  'Crab Mental': 'https://www.imdb.com/title/tt8317600/?ref_=ev_nom',
  'Gunita': 'https://www.imdb.com/title/tt8317626/?ref_=ev_nom',
  'Isang gabi': 'https://www.imdb.com/title/tt8278578/?ref_=ev_nom',
  'Kinalimutan natin ang mga bata': 'https://www.imdb.com/title/tt8317686/?ref_=ev_nom',
  'Noel': 'https://www.imdb.com/title/tt8317690/?ref_=ev_nom'},
 2018: {'One Great Love': 'https://www.imdb.com/title/tt9395384/?ref_=ev_nom',
  "Rainbow's Sunset": 'https://www.imdb.com/title/tt9389924/?ref_=ev_nom',
  'The Girl In the Orange Dress': 'https://www.imdb.com/title/tt9493038/?ref_=ev_nom',
  'Aurora': 'https://www.imdb.com/title/tt8821182/?ref_=ev_nom',
  'Otlum': 'https://www.imdb.com/title/tt9494300/?ref_=ev_nom',
  'Jack Em Popoy: The Puliscredibles': 'https://www.imdb.com/title/tt9139664/?ref_=ev_nom',
  'Mary, Marry Me': 'https://www.imdb.com/title/tt9337248/?ref_=ev_nom',
  'Fantastica': 'https://www.imdb.com/title/tt9270416/?ref_=ev_nom',
  'Kasilyas': 'https://www.imdb.com/title/tt10397958/?ref_=ev_nom',
  'Balita': 'https://www.imdb.com/title/tt10491674/?ref_=ev_nom',
  'Binibini 14. Beverly Lagdameo': 'https://www.imdb.com/title/tt10491818/?ref_=ev_nom',
  'Dalawampung pisong pag-asa': 'https://www.imdb.com/title/tt10492368/?ref_=ev_nom',
  'Padyak': 'https://www.imdb.com/title/tt10492558/?ref_=ev_nom',
  'Paraiso': 'https://www.imdb.com/title/tt10492402/?ref_=ev_nom',
  'Sasagot ka pa?': 'https://www.imdb.com/title/tt10492180/?ref_=ev_nom',
  'Tahanan': 'https://www.imdb.com/title/tt10492228/?ref_=ev_nom'},
 2019: {'Mindanao': 'https://www.imdb.com/title/tt11519610/?ref_=ev_nom',
  'Miracle in Cell No. 7': 'https://www.imdb.com/title/tt10845262/?ref_=ev_nom',
  'Write About Love': 'https://www.imdb.com/title/tt11531544/?ref_=ev_nom',
  'Sunod': 'https://www.imdb.com/title/tt11531410/?ref_=ev_nom',
  '3pol Trobol Huli Ka Balbon': 'https://www.imdb.com/title/tt11531234/?ref_=ev_nom',
  'Culion': 'https://www.imdb.com/title/tt11531248/?ref_=ev_nom',
  'M&M: The Mall The Merrier': 'https://www.imdb.com/title/tt10619522/?ref_=ev_nom',
  'Mission Unstapabol: The Don Identity': 'https://www.imdb.com/title/tt10827078/?ref_=ev_nom',
  'Pamana ni Lola': 'https://www.imdb.com/title/tt11582904/?ref_=ev_nom',
  'Ang huling kembot ni Fernando': 'https://www.imdb.com/title/tt11582832/?ref_=ev_nom',
  'Bronze, silver, gold, at anting-anting': 'https://www.imdb.com/title/tt11583034/?ref_=ev_nom',
  'Dating App': 'https://www.imdb.com/title/tt11583092/?ref_=ev_nom',
  'Hipos': 'https://www.imdb.com/title/tt11582818/?ref_=ev_nom',
  'Manggagalaw': 'https://www.imdb.com/title/tt11582954/?ref_=ev_nom',
  'Tabako': 'https://www.imdb.com/title/tt11582794/?ref_=ev_nom',
  'The Lost Sitty': 'https://www.imdb.com/title/tt11582866/?ref_=ev_nom'},
 2020: {'Fan Girl': 'https://www.imdb.com/title/tt11091664/?ref_=ev_nom',
  'The Boy Foretold by the Stars': 'https://www.imdb.com/title/tt13397632/?ref_=ev_nom',
  'Suarez: The Healing Priest': 'https://www.imdb.com/title/tt13723386/?ref_=ev_nom',
  'One More Rainbow': 'https://www.imdb.com/title/tt13572612/?ref_=ev_nom',
  'Rendezvous': 'https://www.imdb.com/title/tt13702138/?ref_=ev_nom',
  'The Missing': 'https://www.imdb.com/title/tt13713640/?ref_=ev_nom',
  'Coming Home': 'https://www.imdb.com/title/tt13724106/?ref_=ev_nom',
  'Magikland': 'https://www.imdb.com/title/tt13582504/?ref_=ev_nom',
  'Lipstick': 'https://www.imdb.com/title/tt13727604/?ref_=ev_nom',
  'Paano maging babae': 'https://www.imdb.com/title/tt13727686/?ref_=ev_nom',
  'AaBaKaDa': 'https://www.imdb.com/title/tt13723436/?ref_=ev_nom',
  'Balik-bayan': 'https://www.imdb.com/title/tt13727488/?ref_=ev_nom',
  'Garbo': 'https://www.imdb.com/title/tt13727570/?ref_=ev_nom',
  'Kyokok': 'https://www.imdb.com/title/tt13727596/?ref_=ev_nom',
  'Laruang baril': 'https://www.imdb.com/title/tt13727634/?ref_=ev_nom',
  'Paraisong parisukat': 'https://www.imdb.com/title/tt13727702/?ref_=ev_nom',
  'Sala, salin-laway': 'https://www.imdb.com/title/tt13727732/?ref_=ev_nom',
  'Sina Alexa, Xander at ang Universe': 'https://www.imdb.com/title/tt13727746/?ref_=ev_nom',
  'Mang Kepweng: The Mystery of the Black Scarf': 'https://www.imdb.com/title/tt10845704/?ref_=ev_nom'},
 2021: {'Big Night!': 'https://www.imdb.com/title/tt16024312/?ref_=ev_nom',
  'Whether the Weather Is Fine': 'https://www.imdb.com/title/tt14962622/?ref_=ev_nom',
  'A Hard Day': 'https://www.imdb.com/title/tt16024152/?ref_=ev_nom',
  'The Exorsis': 'https://www.imdb.com/title/tt16729508/?ref_=ev_nom',
  'Love at First Stream': 'https://www.imdb.com/title/tt15006444/?ref_=ev_nom',
  'Huwag kang lalabas': 'https://www.imdb.com/title/tt16024722/?ref_=ev_nom',
  'Nelia': 'https://www.imdb.com/title/tt16024470/?ref_=ev_nom',
  'Huling ulan sa tag-araw': 'https://www.imdb.com/title/tt16024362/?ref_=ev_nom',
  'Kandado': 'https://www.imdb.com/title/tt16730442/?ref_=ev_nom',
  'Dito ka lang': 'https://www.imdb.com/title/tt16730544/?ref_=ev_nom',
  'Let It Simmer': 'https://www.imdb.com/title/tt16730426/?ref_=ev_nom',
  'Pepe sa lockdown': 'https://www.imdb.com/title/tt16730636/?ref_=ev_nom',
  'Then & Now': 'https://www.imdb.com/title/tt16730494/?ref_=ev_nom'},
 2022: {'Nanahimik ang gabi': 'https://www.imdb.com/title/tt25147662/?ref_=ev_nom',
  'My Father, Myself': 'https://www.imdb.com/title/tt23771510/?ref_=ev_nom',
  'Family Matters': 'https://www.imdb.com/title/tt25145532/?ref_=ev_nom',
  'Deleter': 'https://www.imdb.com/title/tt21237578/?ref_=ev_nom',
  'Partners in Crime': 'https://www.imdb.com/title/tt22178922/?ref_=ev_nom',
  'My Teacher': 'https://www.imdb.com/title/tt25147606/?ref_=ev_nom',
  'Mamasapano: Now It Can Be Told': 'https://www.imdb.com/title/tt25146782/?ref_=ev_nom',
  'Labyu with an Accent': 'https://www.imdb.com/title/tt22170224/?ref_=ev_nom'},
 2023: {'GomBurZa': 'https://www.imdb.com/title/tt27203116/?ref_=ev_nom',
  'Family of Two (A Mother and Son Story)': 'https://www.imdb.com/title/tt29544849/?ref_=ev_nom',
  'When I Met You in Tokyo': 'https://www.imdb.com/title/tt28929276/?ref_=ev_nom',
  'Kampon': 'https://www.imdb.com/title/tt29544860/?ref_=ev_nom',
  'Rewind': 'https://www.imdb.com/title/tt28152455/?ref_=ev_nom',
  'Mallari': 'https://www.imdb.com/title/tt29544868/?ref_=ev_nom',
  'Becky and Badette': 'https://www.imdb.com/title/tt29544878/?ref_=ev_nom',
  'Firefly': 'https://www.imdb.com/title/tt29173414/?ref_=ev_nom',
  'Broken Hearts Trip': 'https://www.imdb.com/title/tt29544938/?ref_=ev_nom',
  'Penduko': 'https://www.imdb.com/title/tt7248350/?ref_=ev_nom'}}

In [19]:
movies_list = list(all_movies['Title'].to_dict().values())

duplicates = [item for item in movies_list if movies_list.count(item) > 1 ]
duplicates_unique = []
for item in duplicates:
    if not(item in duplicates_unique):
        duplicates_unique.append(item)

all_movies['Title'] = all_movies.apply(lambda x: f'{x.Title} ({str(x.Year)})'if x.Title in duplicates_unique else x.Title, axis=1)

In [20]:
last_year = 2023
years_list = [num for num in range(1975,last_year+1)]

all_movies_list = [pd.DataFrame(data[year]) for year in years_list]
new_concat_list = []

for i in range(len(years_list)):
    year = years_list[i]
    links_dict = filmdict[year]
    movies_links_list = list(links_dict.keys())
    all_links_list = list(links_dict.values())

    year_df = all_movies_list[i]
    real_movies_list = list(year_df['Title'].to_dict().values())

    for i2 in range(len(movies_links_list)):
        for real_movie in real_movies_list:
            real_movie_lower = real_movie.lower()
            change_movie = movies_links_list[i2].lower()
            if change_movie in real_movie_lower or real_movie_lower in change_movie:
                del movies_links_list[i2]
                movies_links_list.insert(i2, real_movie)

    score_df_dict = {'Movie':movies_links_list, 'Link 2':all_links_list}
    score_df = pd.DataFrame(score_df_dict)

    new_df = year_df.merge(score_df, how='left', left_on='Title', right_on='Movie')
    new_concat_list.append(new_df)


all_movies_clean_links = pd.concat(new_concat_list, ignore_index=True)
all_movies_clean_links = all_movies_clean_links[['Year','Title', 'Link 2']]
all_movies_clean_links['Title'] = all_movies_clean_links.apply(lambda x: f'{x.Title} ({str(x.Year)})'if x.Title in duplicates_unique else x.Title, axis=1)
del all_movies_clean_links['Year']

### merge attempts, get correct link

In [21]:
all_movies_merge = all_movies.merge(all_movies_clean_links, on='Title', how='outer')
all_movies_merge['Link 2'] = all_movies_merge['Link 2'].apply(lambda x: 'None' if type(x) != str else x)
all_movies_merge['Link'] = all_movies_merge.apply(lambda x: x['Link 1'] if x['Link 2'] == 'None' or x['Link 1'][:x['Link 1'].find('ref')] == x['Link 2'][:x['Link 2'].find('ref')] else x['Link 2'], axis=1)
all_movies_merge = all_movies_merge.drop(columns=['Link 1','Link 2'])
all_movies = all_movies_merge.copy()

In [22]:
# correct_links_list = list(all_movies['Link'].to_dict().values())
# score_column = []

# for url in correct_links_list:
#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
#     }
    
#     response = requests.get(url, headers=headers)
    
#     if response.status_code == 200:
#         soup = BeautifulSoup(response.text, "html.parser")
#         # Now you can use soup.find_all
#     else:
#         print(f"Error: {response.status_code}")
    
#     ########################
    
#     score_list = list(soup.find_all('span'))
#     score_list = [item.get_text().strip() for item in score_list]
    
#     if '/10' in score_list:
#         cut_index = score_list.index('/10')
#         score_list = score_list[cut_index-1:cut_index+1]
#         score = score_list[0]
#         print(score)
#         score_column.append(score)
#     else:
#         score = 'No IMDb Rating'
#         print(score)
#         score_column.append(score)

# score_column

In [23]:
score_column = ['No IMDb Rating', 'No IMDb Rating', '6.0', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', '7.5', '7.5', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', '8.0', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', '8.0', '7.0', '8.2', '7.4', 'No IMDb Rating', 'No IMDb Rating', '7.3', '6.6', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', '8.6', 'No IMDb Rating', 'No IMDb Rating', '8.7', 'No IMDb Rating', '8.3', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', '7.5', '6.3', 'No IMDb Rating', '7.4', 'No IMDb Rating', '8.4', '4.9', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', '8.5', '7.7', '5.4', '6.4', '8.7', 'No IMDb Rating', 'No IMDb Rating', '6.6', '8.0', 'No IMDb Rating', '7.4', '5.9', '7.7', 'No IMDb Rating', 'No IMDb Rating', '7.5', 'No IMDb Rating', 'No IMDb Rating', '6.0', '7.5', '7.2', 'No IMDb Rating', 'No IMDb Rating', '5.6', '8.6', 'No IMDb Rating', '7.1', 'No IMDb Rating', '7.2', 'No IMDb Rating', '6.3', '6.5', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', '6.4', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', 'No IMDb Rating', '7.5', 'No IMDb Rating', 'No IMDb Rating', '7.4', '7.4', '7.4', '5.8', 'No IMDb Rating', '7.4', '5.1', 'No IMDb Rating', '5.9', '6.4', 'No IMDb Rating', '6.8', 'No IMDb Rating', '7.3', 'No IMDb Rating', 'No IMDb Rating', '6.5', 'No IMDb Rating', '5.5', 'No IMDb Rating', '5.6', '5.2', 'No IMDb Rating', 'No IMDb Rating', '7.5', 'No IMDb Rating', '7.7', '4.5', 'No IMDb Rating', '8/10', '6.3', '6.2', 'No IMDb Rating', '6.2', '7.3', '6.5', '5.4', '7.8', '8.1', '5.3', '6.4', '6.4', 'No IMDb Rating', 'No IMDb Rating', '6.8', '7.4', '7.4', '2.4', '8.5', 'No IMDb Rating', 'No IMDb Rating', '7.7', '6.1', '6.9', '6.5', 'No IMDb Rating', 'No IMDb Rating', '6.2', '7.1', '7.4', '5.2', '7.4', 'No IMDb Rating', '7.5', '5.4', 'No IMDb Rating', '7.6', '6.9', '7.3', '6.9', '8.0', 'No IMDb Rating', '6.5', 'No IMDb Rating', '7.6', '7.2', '6.2', '6.3', 'No IMDb Rating', '6.8', '7.6', '7.1', '6.0', '7.3', '6.2', '6.9', '4.8', '6.1', '6.6', '7.6', '7.0', '5.6', 'No IMDb Rating', '6.6', 'No IMDb Rating', '5.5', '5.6', '6.0', '5.5', '7.7', '8.3', '8.1', '5.9', '6.3', 'No IMDb Rating', '6.0', '5.0', '6.7', '6.2', '6.9', '5.4', '5.9', '6.0', '6.7', 'No IMDb Rating', '7.0', '6.2', '5.3', '5.7', '7.1', '5.8', '6.3', '5.2', '7.4', '5.1', '6.0', '5.7', '5.6', '5.7', 'No IMDb Rating', '7.8', '8.0', '5.2', '5.2', '7.1', '4.9', '5.1', '5.8', '5.6', '5.8', '3.3', '5.4', '7.1', '6.6', '5.8', '2.9', '7.2', '5.3', '4.1', '6.6', '5.2', '6.9', '7.0', '6.6', '4.6', '7.3', '6.6', '6.6', '5.6', '7.0', '5.8', '5.1', '6.1', '4.0', '4.5', '5.4', '5.4', '5.3', '6.0', '7.6', '7.3', '5.9', '7.1', '6.0', '3.5', '7.4', '5.6', '6.5', '4.6', '6.3', '7.2', '6.1', '5.9', '6.0', '5.2', '5.2', '5.3', '5.9', '6.9', '5.8', '6.1', '5.3', '6.9', '5.5', '2.6', '5.4', '4.3', '4.2', '6.5', '7.1', '5.2', '6.4', '5.2', '4.7', '5.5', '6.2', '4.7', '4.9', '4.3', '7.5', '3.9', '4.3', '7.0', '6.6', '7.5', '7.7', '4.6', '7.1', '6.1', '7.9', '6.8', '5.6', '3.4', '3.8', '5.5', '7.8', '3.6', '7.1', '5.9', '4.4', '3.6', '6.7', '5.9', '6.0', '6.4', '2.6', '7.3', '7.5', '3.9', '3.3', '5.7', '3.4', '6.0', '6.4', '4.7', '3.7', '6.6', '4.2', '5.5', '7.9', '7.1', '7.3', '3.6', '6.7', '3.6', '6.2', '7.0', '5.0', '2.8', '6.0', '5.7', '5.8', '4.1', '4.3', '6.0', '4.3', '2.1', '5.2', '6.5', '4.6', '4.2', '6.2', '4.8', '5.8', '6.7', '6.5', '3.5', '7.7', '7.6', '6.1', '3.6']

#### IF INFO NEEDS TO BE UPDATED FOR A NEW YEAR
change `start_year` and `end_year` to the years needed

In [24]:
# new_score_column = []

# start_year = 2023
# all_years_list = list(all_movies['Year'].to_dict().values())
# cut_off_index = all_years_list.index(start_year)

# correct_links_list = list(all_movies['Link'].to_dict().values())
# correct_links_list = correct_links_list[cut_off_index:]

# for url in correct_links_list:
#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
#     }
    
#     response = requests.get(url, headers=headers)
    
#     if response.status_code == 200:
#         soup = BeautifulSoup(response.text, "html.parser")
#         # Now you can use soup.find_all
#     else:
#         print(f"Error: {response.status_code}")
    
#     ########################
    
#     score_list = list(soup.find_all('span'))
#     score_list = [item.get_text().strip() for item in score_list]
    
#     if '/10' in score_list:
#         cut_index = score_list.index('/10')
#         score_list = score_list[cut_index-1:cut_index+1]
#         score = score_list[0]
#         print(score)
#         new_score_column.append(score)
#     else:
#         score = 'No IMDb Rating'
#         print(score)
#         new_score_column.append(score)

# score_column = score_column + new_score_column

In [25]:
all_movies['IMDb Rating'] = score_column

all_movies['IMDb Rating'] = all_movies['IMDb Rating'].apply(lambda x: x[:x.find('/')] if '/' in x else x)
all_movies['IMDb Rating'] = all_movies['IMDb Rating'].apply(lambda x: float(x) if x != 'No IMDb Rating' else x)

## AWARDS DATAFRAME

In [26]:
# MAKE LIST OF ALL STUDIOS

all_studios = list(all_movies['Studio'].to_dict().values())
remove_studios = [item for item in all_studios if item.find('and') != -1]

for item in all_studios:
    if item.find('and') != -1:
        indiv_studios = item.split('and')
        for split_studio in indiv_studios:
            all_studios.append(split_studio.strip())

for item in remove_studios:
    all_studios.remove(item)


remove_studios = [item for item in all_studios if item.find(',') != -1]
for item in all_studios:
    if item.find(',') != -1:
        indiv_studios = item.split(',')
        indiv_studios_clean = []
        # print(indiv_studios)
        for index in range(1,len(indiv_studios)):
            if indiv_studios[index].strip() == 'Inc.':
                indiv_studios_clean.append(indiv_studios[index-1]+indiv_studios[index])
            else:
                indiv_studios_clean.append(indiv_studios[index].strip())
        for split_studio in indiv_studios_clean:
            all_studios.append(split_studio.strip())

In [27]:
all_years = []
all_awards = []
all_winners_artist = []
all_winners_movie = []

last_year = 2023
years_list = [num for num in range(1975,2023+1)]

award_dict = {}
for year in years_list:
    awards_per_year = awards_data[year]
    awards = list(awards_per_year.keys())
    year_column = [year]*len(awards)

    for item in year_column:
        all_years.append(item)
    for item in awards:
        all_awards.append(item)

    winners = list(awards_per_year.values())
    for item in winners:
        if type(item) == str:
            all_winners_movie.append(item)
            all_winners_artist.append('—')
        else:
            all_winners_movie.append(list(item.values())[0])
            all_winners_artist.append(list(item.keys())[0])


award_dict.update({'Year':all_years, 'Award':all_awards, 'Movie Winner':all_winners_movie, 'Artist Winner':all_winners_artist})
awards_df = pd.DataFrame(award_dict)
awards_clean = awards_df.copy()

# DATA CLEANING
## remove productions
awards_clean['Movie Winner'] = awards_clean['Movie Winner'].apply(lambda x: x[:x.find(' – ')] if ' – ' in x else x)
awards_clean['Movie Winner'] = awards_clean['Movie Winner'].apply(lambda x: x[:x.find(' - ')] if ' - ' in x else x)

## separate movie and artist (if not yet)
awards_clean['Artist Winner'] = awards_clean.apply(lambda x: x['Movie Winner'][:x['Movie Winner'].find('- ')] if '- ' in x['Movie Winner'] else x['Artist Winner'], axis=1)
awards_clean['Artist Winner'] = awards_clean.apply(lambda x: x['Movie Winner'][:x['Movie Winner'].find(' -')] if ' -' in x['Movie Winner'] else x['Artist Winner'], axis=1)
awards_clean['Movie Winner'] = awards_clean['Movie Winner'].apply(lambda x: x[x.find('- ')+2:] if '- ' in x else x)
awards_clean['Movie Winner'] = awards_clean['Movie Winner'].apply(lambda x: x[x.find(' -')+2:] if ' -' in x else x)

## remove productions, move movie from artist to movie winner
awards_clean['Movie Winner'] = awards_clean.apply(lambda x: x['Artist Winner'] if x['Movie Winner'] in all_studios else x['Movie Winner'], axis=1)
awards_clean['Artist Winner'] = awards_clean.apply(lambda x: '—' if x['Movie Winner'] == x['Artist Winner'] else x['Artist Winner'], axis=1)

## song cleaning
awards_clean['Move to Movie'] = awards_clean['Artist Winner'].apply(lambda x: x if 'from' in x else '—')
awards_clean['Move to Artist'] = awards_clean['Movie Winner'].apply(lambda x: x if 'by' in x else '—')
awards_clean['Move to Artist'] = awards_clean.apply(lambda x: x['Movie Winner'] if x['Move to Movie'] != '—' else '—', axis=1)

awards_clean['Movie Winner'] = awards_clean.apply(lambda x: x['Move to Movie'] if x['Move to Movie'] != '—' else x['Movie Winner'], axis=1)
awards_clean['Artist Winner'] = awards_clean.apply(lambda x: x['Move to Artist'] if x['Move to Artist'] != '—' else x['Artist Winner'], axis=1)

del awards_clean['Move to Movie']
del awards_clean['Move to Artist']

awards_clean['Artist Winner'] = awards_clean['Artist Winner'].apply(lambda x: x[1:-1] if x[0] == '(' and x[-1] == ')' else x)

# REASSIGN VARIABLE
awards_df = awards_clean.copy()

# ***RUN WEBSITE***

In [28]:
import pickle

with open('all_movies.pkl', 'wb') as f:
    pickle.dump(all_movies, f)

with open('awards_df.pkl', 'wb') as g:
    pickle.dump(awards_df, g)

In [29]:
# !pip install voila
# !pip install -U ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

In [ ]:
!voila --debug "[ITMGT] Final Project.ipynb"